# Python Deep Dive Part 4 - Object Oriented Programming

## Introduction

### Course topics

* Focus on Object Oriented Programming (OOP) concepts in Python
  * classes (types) and objects
  * functions, methods, and properties
  * polymorphism and special dunder methods
  * single inheritance
  * descriptors
  * enumerations
  * exceptions

## Classes

### Objects and classes

What is an object

* An object is any data with state (attributes or value) and defined behavior (methods).
* An object is an instance of a class.
* Characteristics of an object:
  * Identity: each object has a unique name or identity.
  * State: objects can have attributes or value that are initialized (often using constructors).
  * Behavior: objects perform specific actions or tasks through functions (methods).
* `object` is the ultimate base class of any new-style class in Python.
  * `class object`
    * Return a new featureless object. `object` is a base for all classes.
    * It has methods that are common to all instances of Python classes.
    * This function does not accept any arguments.

What is a class

* Classes provide a means of bundling data and functionality together, i.e. a template for creating user-defined objects.
* Classes are themselves objects that are created from the `type` [metaclass](https://docs.python.org/3/reference/datamodel.html#metaclasses).
* Creating a new class creates a new *type* of object, allowing new instances of that type to be made.
* Each class instance can have *attributes* attached to it for maintaining its state.
* Class instances can also have *methods* (defined by its class) for modifying its state.
* Python classes provide all the standard features of Object Oriented Programming:
  * the class inheritance mechanism allows multiple base classes
  * a derived class can override any methods of its base class or classes
  * a method can call the method of a base class with the same name.

`type`

* `class type(object)`, `class type(name, bases, dict, **kwds)`
  * With one argument, return the type of an object. The return value is a `type` object and generally the same object as returned by `object.__class__`.
    * The `isinstance()` built-in function is recommended for testing the type of an object, because it takes subclasses into account.
  * With three arguments, return a new `type` object. This is essentially a dynamic form of the `class` statement.
    * The `name` string is the class name and becomes the `__name__` attribute.
    * The `bases` tuple contains the base classes and becomes the `__bases__` attribute; if empty, `object`, the ultimate base of all classes, is added.
    * The `dict` dictionary contains attribute and method definitions for the class body; it may be copied or wrapped before becoming the `__dict__` attribute.

In [6]:
from pprint import pprint

class Program:
    pass

pprint(Program.__dict__)
# pprint(vars(MyClass)) # same as above
pprint(dir(Program))
print(Program.__name__)
print(type(Program)) # class type
print(type(type)) # class type
print(type(Program())) # class __main__.MyClass

mappingproxy({'__dict__': <attribute '__dict__' of 'MyClass' objects>,
              '__doc__': None,
              '__module__': '__main__',
              '__weakref__': <attribute '__weakref__' of 'MyClass' objects>})
['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__']
MyClass
<class 'type'>
<class 'type'>
<class '__main__.MyClass'>


### Class variables / attributes

* A variable defined in a class and intended to be modified only at class level (i.e. not in an instance of the class).
* Class variables are for attributes and methods shared by all instances of the class.

`getattr`

* `getattr(object, name)`, `getattr(object, name, default)`
  * Return the value of the named attribute of `object`.
  * `name` must be a string. If the string is the name of one of the object's attributes, the result is the value of that attribute.
  * `name` need *not* be a Python identifier.
  * For example, `getattr(x, 'foobar')` is equivalent to `x.foobar`. If the named attribute does not exist, `default` is returned if provided, otherwise `AttributeError` is raised.
* Note Since private name mangling happens at compilation time, one must manually mangle a private attribute's (attributes with two leading underscores) name in order to retrieve it with `getattr()`.
  * Private name mangling
    * When an identifier that textually occurs in a class definition begins with *two or more* underscore characters and does *not* end in *two or more* underscores, it is considered a private name of that class.
    * Private names are transformed to a longer form before code is generated for them. The transformation inserts the class name, with leading underscores removed and a single underscore inserted, in front of the name.
    * For example, the identifier `__spam` occurring in a class named `Ham` will be transformed to `_Ham__spam`.
    * This transformation is independent of the syntactical context in which the identifier is used.
    * If the transformed name is extremely long (longer than 255 characters), implementation defined truncation may happen.
    * If the class name consists only of underscores, no transformation is done.

`setattr`

* `setattr(object, name, value)`
  * This is the counterpart of `getattr()`.
  * The arguments are an object, a string, and an arbitrary value. The string may name an existing attribute or a new attribute.
  * `name` need not be a Python identifier unless the object chooses to enforce that, for example in a custom `__getattribute__()` or via `__slots__`. An attribute whose name is not an identifier will not be accessible using the dot notation, but is accessible through `getattr()`.
  * The function assigns the value to the attribute, provided the object allows it.
  * For example, `setattr(x, 'foobar', 123)` is equivalent to `x.foobar = 123`.
  * Note Since private name mangling happens at compilation time, one must manually mangle a private attribute's name in order to set it with `setattr()`.

`delattr`

* `delattr(object, name)`
  * This is a relative of `setattr()`.
  * The arguments are an object and a string. The string must be the name of one of the object's attributes.
  * `name` need not be a Python identifier.
  * The function deletes the named attribute, provided the object allows it.
  * For example, `delattr(x, 'foobar')` is equivalent to `del x.foobar`.

`object.__dict__`

* A dictionary or other mapping object used to store an object's (writable) attributes.
* The `__dict__` attribute of a class returns a mappingproxy object.

### Callable class attributes

Setting an attribute value to a callable

* Attribute values can be any object (e.g. other classes, callables, etc.).

In [25]:
class Program:
    language = 'Python'

    def say_hello(): # no self, cls or decorator needed here
        print(f'Hello from {Program.language}')

print(Program.__dict__)

# different ways to call say_hello
Program.say_hello()
getattr(Program, 'say_hello')()
Program.__dict__['say_hello']()
print(type(Program.say_hello)) # function
# MyClass().say_hello() # does not work, TypeError, extra arguments is passed to the function

{'__module__': '__main__', 'language': 'Python', 'say_hello': <function MyClass.say_hello at 0x00000122B0CE40E0>, '__dict__': <attribute '__dict__' of 'MyClass' objects>, '__weakref__': <attribute '__weakref__' of 'MyClass' objects>, '__doc__': None}
Hello from Python
Hello from Python
Hello from Python
<class 'function'>


### Classes are callables

* When we create a class using the `class` keyword, Python automatically adds behaviors to the class.
  * It adds something to make the class callable.
  * The return value of that callable is an object
  * The type of the returned object is the class object, i.e. the returned object is an instance of the class.

Instantiating classes

* When we call a class, a class instance object is created.
* The class instance has its own namespace, which is distinct from the namespace of the class.
* The instance has some attributes Python automatically implements for us:
  * `__dict__`: the instance's local namespace
  * `__class__`: the class used to instantiate the object
    * prefer using `type(obj)` instead of `obj.__class__`

In [10]:
class Program:
    # __class__ = str # can be modified manually, this attribute affects isinstance()
    language = 'Python'

program = Program()
print(type(program))
print(program.__class__) # __class__ can be modified inside the class manually
print(type(program) is program.__class__)
print(isinstance(program, Program))
print(program.__dict__)
print(program.language) # find the language attribute in the class

<class '__main__.Program'>
<class '__main__.Program'>
True
True
{}
Python


### Class objects and instance objects

Class objects

* When a class definition is left normally (via the end), a class object is created. This is basically a wrapper around the contents of the namespace created by the class definition. The original local scope (the one in effect just before the class definition was entered) is reinstated, and the class object is bound here to the class name given in the class definition header.
* Class objects support two kinds of operations:
  * Attribute references
    * Attribute references use the standard syntax used for all attribute references in Python: `obj.name`.
  * Instantiation
    * Class instantiation uses function notation: `MyClass()`.
    * The instantiation operation creates an empty object.
    * Many classes like to create objects with instances customized to a specific initial state. Therefore a class may define a special method named `__init__() that can have arguments for flexibility.
    * When a class defines an `__init__()` method, class instantiation automatically invokes `__init__()` for the newly created class instance.

Instance objects

* An instance is an actual object created from a class.
* When you create an instance, you're essentially making a copy of the class with its attributes and methods.
* Each instance has its own unique data and can perform actions defined by the class.
* The only operations understood by instance objects are attribute references.
* There are two kinds of valid attribute names:
  * Data attributes
    * Data attributes are variables bound to an object. They represent information or state associated with an *instance* of a class.
    * Data attributes need not be declared; like local variables, they spring into existence when they are first assigned to.
  * Methods
    * A method is a function that bounds to an object.
    * `method` is an actual object type in Python.
    * An instance method object combines a class, a class instance and any callable object (normally a user-defined function).
      * Special read-only attributes:
        * `method.__self__`
        * `method.__func__`
        * `method.__doc__`
        * `method.__name__`
        * `method.__module__`
      * Methods also support accessing (but not setting) the arbitrary function attributes on the underlying function object.
      * When an instance method object is created by retrieving a user-defined function object from a class via one of its instances, its `__self__` attribute is the instance, and the method object is said to be *bound*. The new method's `__func__` attribute is the original function object.
      * When an instance method object is created by retrieving a classmethod object from a class or instance, its `__self__` attribute is the class itself, and its `__func__` attribute is the function object underlying the class method.
      * When an instance method object is called, the underlying function (`__func__`) is called, inserting the class instance (`__self__`) in front of the argument list. For instance, when `C` is a class which contains a definition for a function `f()`, and `x` is an instance of `C`, calling `x.f(1)` is equivalent to calling `C.f(x, 1)`.
      * When an instance method object is derived from a classmethod object, the class instance stored in `__self__` will actually be the class itself, so that calling either `x.f(1)` or `C.f(1)` is equivalent to calling `f(C,1)` where `f` is the underlying function.
      * Note that the transformation from function object to instance method object happens each time the attribute is retrieved from the instance.
      * Also notice that this transformation only happens for *user-defined functions*; other callable objects (and all non-callable objects) are retrieved without transformation.
      * It is also important to note that user-defined functions which are attributes of a class instance are *not* converted to bound methods; this only happens when the function is an attribute of the class.

In [27]:
class Program:
    language = 'Python'

    def say_hello(self):
        print(f'Hello from {self.language}')

print(f'{type(Program.__dict__)=}')
print(Program.__dict__)
program = Program()
print(f'{type(program.__dict__)=}')
print(program.__dict__)
print(program.language)
program.language = 'C'
print(program.__dict__)
print(Program.language)

Program.version = 3.12
print(program.version)

print(type(Program.say_hello)) # function
print(type(program.say_hello)) # method
Program.say_hello(Program)
Program.say_hello(program)
program.say_hello() # equivalent to the above one
print(program.say_hello.__func__)
print(program.say_hello.__self__)
print(program.say_hello.__name__)
print(program.say_hello.__doc__)
print(program.say_hello.__module__)

type(Program.__dict__)=<class 'mappingproxy'>
{'__module__': '__main__', 'language': 'Python', 'say_hello': <function Program.say_hello at 0x00000122B0CE44A0>, '__dict__': <attribute '__dict__' of 'Program' objects>, '__weakref__': <attribute '__weakref__' of 'Program' objects>, '__doc__': None}
type(program.__dict__)=<class 'dict'>
{}
Python
{'language': 'C'}
Python
3.12
<class 'function'>
<class 'method'>
Hello from Python
Hello from C
Hello from C
<function Program.say_hello at 0x00000122B0CE44A0>
say_hello
None
__main__


### Initializing class instances

* When we instantiate a class, by default Python does two separate things:
  * creates a new instance of the class
  * initialize the namespace of the instance
* We can provide a custom initializer method (`__init__`) Python will use after the instance and its namespace are created.

### Creating attributes at runtime

* We can add attributes to an instance namespace directly at runtime by using dot notation or `setattr`.
* If we create a new attribute whose value is a function, then that function will not be converted to bound method when it is retrieved, i.e. that function has no access to the instance namespace.
* We can also create and bind a method to an instance at runtime manually by using `types.MethodType`.

In [4]:
from types import MethodType

class Program:
    def __init__(self, language):
        self.language = language

program = Program('Python')
program.say_hi = lambda: 'Hi'
print(program.say_hi())
print(program.__dict__)
program.say_hello = MethodType(lambda self: f'Hello from {self.language}', program)
print(program.say_hello())
print(program.__dict__)

Hi
{'language': 'Python', 'say_hi': <function <lambda> at 0x0000017E70773BA0>}
Hello from Python
{'language': 'Python', 'say_hi': <function <lambda> at 0x0000017E70773BA0>, 'say_hello': <bound method <lambda> of <__main__.Program object at 0x0000017E7066F110>>}


In [20]:
from types import MethodType

class Person:
    def __init__(self, name):
        self.name = name

    def register_do_work(self, func):
        # self._do_work = MethodType(func, self)
        setattr(self, '_do_work', MethodType(func, self))

    def do_work(self):
        try:
            do_work_method = self._do_work
        except AttributeError as err:
            raise AttributeError('AttributeError: do_work method must be registered first') from err
            # raise AttributeError('AttributeError: do_work method must be registered first').with_traceback(err.__traceback__)
        else:
            return do_work_method()
        # do_work_method = getattr(self, '_do_work', None)
        # if do_work_method:
        #     return do_work_method()
        # else:
        #     raise AttributeError('do_work method must be registered first')
        
def work_math(person):
    return f'{person.name} will teach differentials today'

def work_english(person):
    return f'{person.name} will teach writing today'

math_teacher = Person('John')
english_teacher = Person('Peter')
# print(math_teacher.do_work()) # AttributeError
math_teacher.register_do_work(work_math)
english_teacher.register_do_work(work_english)
print(math_teacher.do_work())
print(english_teacher.do_work())

John will teach differentials today
Peter will teach writing today


### Properties

* We can define bare attributes in classes and instances. However, in many languages direct access to attributes is highly discouraged. Instead, the convention is to make the attribute private, and create public getter and setter methods. Although we don't have private attributes in Python, we can simulate a similar way of dealing with attributes.
* Properties are a special kind of attributes. They allow you to control access to an object's attributes using getter and setter methods.
* Ways to Define Properties:
  * Using the `property()` function
  * Using the `@property` decorator
* `class property(fget=None, fset=None, fdel=None, doc=None)`
  * Return a `property` attribute.
  * `@getter`, `@setter`, `@deleter`
    * A `property` object has `getter`, `setter`, and `deleter` methods usable as *decorators* that create a copy of the property with the corresponding accessor function set to the decorated function.
        ```Python
        x = property()
        x = x.getter(get_x)
        x = x.setter(set_x)
        x = x.deleter(delete_x)
        # or the short way as follows
        x = property(get_x)
        x = x.setter(set_x)
        x = x.deleter(delete_x)
        ```

In [28]:
class Person:
    def __init__(self, name):
        self.name = name

    def get_name(self):
        return self._name
    
    def set_name(self, name):
        self._name = name

    def delete_name(self):
        del self._name

    name = property(get_name, set_name, delete_name) # name is a class attribute

person = Person('John')
print(person.name) # the name attribute is first retrieved in the class namespace, then the getter method of the instance is called
person.name = 'Eric'
print(person.name)
print(Person.__dict__)
print(person.__dict__)
person.__dict__['name'] = 'Tony'
print(person.name) # still get Eric

John
Eric
{'__module__': '__main__', '__init__': <function Person.__init__ at 0x0000017E70DF7920>, 'get_name': <function Person.get_name at 0x0000017E70DF63E0>, 'set_name': <function Person.set_name at 0x0000017E70DF6C00>, 'name': <property object at 0x0000017E70BF8E00>, '__dict__': <attribute '__dict__' of 'Person' objects>, '__weakref__': <attribute '__weakref__' of 'Person' objects>, '__doc__': None}
{'_name': 'Eric'}
Eric


In [30]:
class Person:
    def __init__(self, name):
        self.name = name

    @property
    def name(self):
        return self._name
    
    @name.setter
    def name(self, name):
        self._name = name

    @name.deleter
    def name(self):
        del self._name

person = Person('John')
print(person.name)
person.name = 'Eric'
print(person.name)
print(Person.__dict__)
print(person.__dict__)
person.__dict__['name'] = 'Tony'
print(person.name) # still get Eric
del person.name # _name attribute gets deleted from person
# delattr(person, name) # same as above
# print(person.name) # AttributeError

John
Eric
{'__module__': '__main__', '__init__': <function Person.__init__ at 0x0000017E70DF6DE0>, 'name': <property object at 0x0000017E70E967F0>, '__dict__': <attribute '__dict__' of 'Person' objects>, '__weakref__': <attribute '__weakref__' of 'Person' objects>, '__doc__': None}
{'_name': 'Eric'}
Eric


In [6]:
def getter_func(self):
    return 'getter called'

def setter_func(self):
    return 'setter called'

def deleter_func(self):
    return 'deleter called'

prop0 = property()
# print(prop.__dict__) # AttributeError

# we can use dir() to introspect an object, i.e. to explore and discover its attributes, methods, and other members
# it returns a list of names representing the attributes and methods associated with the object
# dir() is analogous to a directory listing
# just as a directory provides a list of files and subdirectories, dir() provides a list of valid names (attributes) in the scope (object)
# dir(prop)
print(prop0.fget, prop0.fset, prop0.fdel)
prop1 = prop0.getter(getter_func) # the getter method returns a new property object
print(prop0 is prop1)
print(prop1.fget, prop1.fset, prop1.fdel)
prop2 = prop1.setter(setter_func)
prop3 = prop2.deleter(deleter_func)
print(prop3.fget, prop3.fset, prop3.fdel)

None None None
False
<function getter_func at 0x00000146113BB6A0> None None
<function getter_func at 0x00000146113BB6A0> <function setter_func at 0x00000146113BB880> <function deleter_func at 0x00000146113BAA20>


### Read-only and computed properties

Read-only properties

* To create a read-only property, we just need to create a property with only the getter defined.
* It is not truly read-only since underlying storage variable could be accessed directly.
* It is useful for computed properties.

Caching computed properties

* Using property setters is useful for controlling how other computed properties are cached.

In [11]:
import math
import numbers

class Circle:
    def __init__(self, radius):
        self.radius = radius # use the setter to set the radius

    @property
    def radius(self):
        return self._radius
    
    @radius.setter
    def radius(self, radius):
        if not isinstance(radius, numbers.Real):
            raise TypeError('radius must be a number')
        if radius < 0:
            raise ValueError('radius must greater than 0')
        self._radius = radius
        self._area = None # initialize or invalidate the old cache of area

    @property
    def area(self): # cached computed read-only property
        if self._area is None: # recalculate area if not available
            print('calculating area...')
            self._area = math.pi * self._radius * self._radius
        return self._area
    

circle = Circle(1)
print(circle.area)
circle.radius = 2
print(circle.area)
print(circle.area) # using cache this time   

calculating area...
3.141592653589793
calculating area...
12.566370614359172
12.566370614359172


In [1]:
import urllib
from time import perf_counter

class WebPage:
    def __init__(self, url):
        self.url = url

    @property
    def url(self):
        return self._url

    @url.setter
    def url(self, url):
        self._url = url

        # initialize or reset computed properties
        self._page = None
        self._page_size = None
        self._page_load_time = None

    @property
    def page(self):
        if self._page is None:
            self.download_page()
        return self._page

    @property
    def page_size(self):
        if self._page is None:
            self.download_page()
        return self._page_size

    @property
    def page_load_time(self):
        if self._page is None:
            self.download_page()
        return self._page_load_time

    def download_page(self):
        start = perf_counter()
        with urllib.request.urlopen(self._url) as f:
            self._page = f.read()
        end = perf_counter()
        self._page_size = len(self._page)
        self._page_load_time = end - start


urls = [
    'https://www.google.com/',
    'https://www.python.org/',
    'https://www.bing.com/'
]

for url in urls:
    webpage = WebPage(url)
    print(f'{webpage.url}\tsize: {webpage.page_size}\ttime:{webpage.page_load_time}')

https://www.google.com/	size: 18293	time:0.4083212000041385
https://www.python.org/	size: 50871	time:0.03663319999759551
https://www.bing.com/	size: 113456	time:0.2199103000020841


### Static and class methods

Static method objects

* Static method objects provide a way of defeating the transformation of function objects to method objects described above.
* A static method object is a wrapper around any other object, usually a user-defined method object. When a static method object is retrieved from a class or a class instance, the object actually returned is the wrapped object, which is not subject to any further transformation.
* Static method objects are also callable. Static method objects are created by the built-in `staticmethod()` constructor.
* `@staticmethod`
  * Transform a method into a static method.
  * A static method does not receive an implicit first argument.
  * A static method can be called either on the class (such as `C.f()`) or on an instance (such as `C().f()`). Moreover, they can be called as regular functions (such as `f()`) within the class definition, not in code blocks of methods.
    * A class definition is an executable statement that may use and define names. These references follow the normal rules for name resolution with an exception that unbound local variables are looked up in the global namespace. The namespace of the class definition becomes the attribute dictionary of the class.
    * [The scope of names defined in a class block is limited to the class block; it does not extend to the code blocks of methods.](https://docs.python.org/3/reference/executionmodel.html#resolution-of-names) This includes comprehensions and generator expressions, but it does not include annotation scopes, which have access to their enclosing class scopes.
  * Like all decorators, it is also possible to call `staticmethod` as a regular function and do something with its result. This is needed in some cases where you need a reference to a function from a class body and you want to avoid the automatic transformation to instance method.

Class method objects

* A class method object, like a static method object, is a wrapper around another object that alters the way in which that object is retrieved from classes and class instances.
* Class method objects are created by the built-in `classmethod()` constructor.
* `@classmethod`
  * Transform a method into a class method.
  * A class method receives the class as an implicit first argument, just like an instance method receives the instance.
  * A class method can be called either on the class (such as `C.f()`) or on an instance (such as `C().f()`). The instance is ignored except for its class. If a class method is called for a derived class, the derived class object is passed as the implied first argument.

In [41]:
from datetime import datetime, timezone, timedelta
import time

class Timer:
    tz = timezone.utc

    @classmethod
    def set_tz(cls, offset, name=None):
        cls.tz = timezone(timedelta(hours=offset)) if name is None else timezone(timedelta(hours=offset), name)

    @staticmethod
    def current_datetime_utc():
        return datetime.now(timezone.utc)
    
    @classmethod
    def current_datetime(cls):
        return datetime.now(cls.tz)
    
    def start(self):
        self._start_at = self.current_datetime_utc()

        # initialize or reset instance attributes, better initializing in __init__
        self._stop_at = None
        self._time_elapsed = None

    def stop(self):
        if getattr(self, '_start_at', None) is None:
            raise TimerError('timer must be started before it can be stopped')
        self._stop_at = self.current_datetime_utc()

    @property
    def start_at(self):
        if getattr(self, '_start_at', None) is None:
            raise TimerError('timer has not been started yet')
        return self._start_at.astimezone(self.tz)
    
    @property
    def stop_at(self):
        if getattr(self, '_start_at', None) is None:
            raise TimerError('timer has not been started yet')
        if getattr(self, '_stop_at', None) is None:
            raise TimerError('timer has not been stopped')
        return self._stop_at.astimezone(self.tz)
    
    @property
    def time_elapsed(self): # cached read-only property
        if getattr(self, '_start_at', None) is None:
            raise TimerError('timer has not been started yet')
        if getattr(self, '_stop_at', None) is None:
            raise TimerError('timer has not been stopped')
        if getattr(self, '_time_elapsed', None) is None:
            self._time_elapsed = self._stop_at - self._start_at
        return self._time_elapsed.total_seconds()
    

class TimerError(Exception):
    pass

th1 = Timer()
th2 = Timer()
print(th1.tz.__repr__())
th2.set_tz(11, 'AET')
print(th1.tz.__repr__())
print(th1.current_datetime())
print(th1.current_datetime_utc())
th1.start()
time.sleep(0.5)
th1.stop()
print(f'\nstart: {th1.start_at}\nstop: {th1.stop_at}\ntime elapsed: {th1.time_elapsed}s')

datetime.timezone.utc
datetime.timezone(datetime.timedelta(seconds=39600), 'AET')
2024-02-17 16:51:14.620508+11:00
2024-02-17 05:51:14.620508+00:00

start: 2024-02-17 16:51:14.620508+11:00
stop: 2024-02-17 16:51:15.121468+11:00
time elapsed: 0.50096s


### Python built-in and standard types

References

* https://docs.python.org/3/library/stdtypes.html
* https://docs.python.org/3/reference/datamodel.html#types

Built-in types

* Numerics
  * `int`
    * `bool`
  * `float`
  * `complex`
* Sequences
  * `list`
  * `tuple`
  * `range`
  * `str`
  * `bytes`
  * `bytearray`
  * `memoryview`
* Sets
  * `set`
  * `frozenset`
* Mappings
  * `dict`
* Classes
* Instances
* Exceptions
* Not all types are in the builtins.

The `types` module

* This module defines utility functions to assist in dynamic creation of new types.
* It also defines names for some object types that are used by the standard Python interpreter, but not exposed as builtins like `int` or `str` are.
* Finally, it provides some additional type-related utility classes and functions that are not fundamental enough to be builtins.

Standard interpreter types

* `types` module provides names for many of the types that are required to implement a Python interpreter.
* It deliberately avoids including some of the types that arise only incidentally during processing such as the `listiterator` type.
* Typical use of these names is for `isinstance()` or `issubclass()` checks.
* Standard names are defined for the following types:
  * `types.NoneType`
  * `types.FunctionType`
  * `types.LambdaType`
  * `types.GeneratorType`
  * `types.MethodType`
  * `types.BuiltinFunctionType`
  * `types.BuiltinMethodType`
  * `types.NotImplementedType`
  * `types.ModuleType`
  * `types.EllipsisType`
  * `types.MappingProxyType`
  * etc.

### Class body scope

* The scopes of functions defined in the body of a class are not nested in the class body scope.
* Instead, the functions are nested in the class's containing scope.
* So when Python looks for a symbol in a function in a class, it will not use the class body scope.
* Reference: https://docs.python.org/3/reference/executionmodel.html#resolution-of-names
* Rationale behind the limitation:
  * When you define a class, it's essentially syntactic sugar for constructing a dictionary of attributes. This dictionary is then used by the metaclass (usually `type`) to create the actual class object.
  * For example, the following class definition

    ```Python
    class A:
        i = 1
        def f(self):
            print(i)
    ```

    is roughly equivalent to
    
    ```Python
    def f(self):
        print(i)
    attributes = {"f": f, "i": 1}
    A = type("A", (object,), attributes)
    ```

    In this context, there is no outer scope from which the name `i` can be accessed directly. The temporary scope for executing statements within the class block doesn't provide access to class-level attributes.
  * Python treats objects as containers for attributes. There's not really any concept of "variables" other than local variables in functions.
  * Methods within a class are implemented using function scope.
  * This includes comprehensions and generator expressions, which are also implemented using function scope.
  * Consequently, class variables are not visible from methods unless explicitly accessed via the class name.

## Project 1

Basic Information

* We need to design and implement a class that will be used to represent bank accounts with the following functionality and characteristics:
  * accounts are uniquely identified by an **account number** (assume it will just be passed in the initializer)
  * account holders have a **first** and **last** name
  * accounts have an associated **preferred time zone offset** (e.g. -7 for MST)
  * **balances** need to be zero or higher, and should not be directly settable.
  * but, **deposits** and **withdrawals** can be made (given sufficient funds)
    * if a withdrawal is attempted that would result in negative funds, the transaction should be declined.
  * a **monthly interest rate** exists and is applicable to all accounts **uniformly**. There should be a method that can be called to calculate the interest on the current balance using the current interest rate, and **add it** to the balance.
  * each deposit and withdrawal must generate a **confirmation number** composed of:
    * the transaction type: `'D'` for deposit, and `'W'` for withdrawal, `'I'` for interest deposit, and `'X'` for declined (in which case the balance remains unaffected)
    * the account number
    * the time the transaction was made, using UTC
    * an incrementing number (that increments across all accounts and transactions)
    * for simplicity assume that the transaction id starts at zero (or whatever number you choose) whenever the program starts
    * the confirmation number should be returned from any of the transaction methods (deposit, withdraw, etc)
  * create a **method** that, given a confirmation number, returns:
    * the account number, transaction code (D, W, etc), datetime (UTC format), date time (in the timezone specified in the argument, but more human readable), the transaction id
    * make it so it is a nicely structured object (so can use dotted notation to access these three attributes)
  * You should also remember to test your code (using Python's `unittest` package).

Example

* We may have an account with:
  * account number `140568`
  * preferred time zone offset of -7 (MST) 
  * an existing balance of `100.00`
* Suppose the last transaction ID in the system was `123`, and a deposit is made for `50.00` on `2019-03-15T14:59:00` (UTC) on that account (or `2019-03-15T07:59:00` in account's preferred time zone offset)
  * The new balance should reflect `150.00` and the confirmation number returned should look something like this: ```D-140568-20190315145900-124```
* We also want a method that given the confirmation number returns an object with attributes:
  * `result.account_number` --> `140568`
  * `result.transaction_code` --> `'D'`
  * `result.transaction_id` --> `124`
  * `result.time` --> `2019-03-15 07:59:00 (MST)`
  * `result.time_utc` --> `2019-03-15T14:59:00`
* Furthermore, if current interest rate is `0.5%`, and the account's balance is `1000.00`, then the result of calling the `deposit_interest` method, should result in a new transaction and a new balance of `1005.00`. Calling this method should also return a confirmation number.
* For simplicity, just use floats, but be aware that for these types of situations you'll probably want to use `Decimal` objects instead of floats.

In [1]:
from datetime import datetime, timezone, timedelta
from decimal import Decimal
import os
import re
from collections import namedtuple

Transaction = namedtuple('Transaction', 'transaction_type account_id transaction_datetime transaction_datetime_preferred random_code')

class BankAccount:
    _TRANSACTION_TYPES = {'deposit': 'D', 'withdraw': 'W', 'pay_interest': 'I', 'declined': 'X'}
    _monthly_interest_rate = Decimal('0.005')
    _account_ids = set()

    @classmethod
    def get_monthly_interest_rate(cls):
        return str(cls._monthly_interest_rate)
    
    @classmethod
    def set_monthly_interest_rate(cls, monthly_interest_rate):
        if cls.validate_interest_rate(monthly_interest_rate): # exclude strings like 'inf', '-Infinity', 'nan', 'snan', 'snan1' that are acceptable to Decimal()
            cls._monthly_interest_rate = Decimal(monthly_interest_rate)
        else:
            raise ValueError('Internal error: the interest rate should be a numeric string')

    @classmethod
    def generate_account_id(cls, length=18):
        # generate a string of digits with designated length (default to 18), and check the 
        while True:
            account_id = cls.generate_random_digits(length)

            if account_id not in cls._account_ids:
                cls._account_ids.add(account_id)
                break
        return account_id
    
    @classmethod
    def parse_transaction_code(cls, transaction_code, preferred_timezone=None):
        transaction_parameters = transaction_code.split('-')
        # find corresponding transaction_type according to type_code
        for k, v in cls._TRANSACTION_TYPES.items():
            if v == transaction_parameters[0]:
                transaction_type = k
                break
        account_id = transaction_parameters[1]
        transaction_datetime = datetime.strptime(transaction_parameters[2], '%Y%m%d%H%M%S%f').replace(tzinfo=timezone.utc)
        if preferred_timezone is None:
            transaction_datetime_preferred = transaction_datetime
        else:
            transaction_datetime_preferred = transaction_datetime.astimezone(preferred_timezone)
        random_code = transaction_parameters[3]
        return Transaction(transaction_type, account_id, transaction_datetime, transaction_datetime_preferred, random_code)

    @staticmethod
    def generate_random_digits(length):
        # random bytes -> int -> str of digits -> slice the last n digits -> fill with 0 from left if not enough digits
        return f'{str(int.from_bytes(os.urandom(8)))[-length:]:0>{length}}'
    
    # @staticmethod
    # def validate_name(name):
    #     pattern = r'^[a-zA-Z]+$'
    #     return bool(re.match(pattern, name))

    @staticmethod
    def formalize_name(name, field):
        name = name.strip().capitalize()
        if name.isalpha():
            return name
        else:
            raise ValueError(f'{field} should be non-empty and can only contains alphabets')
    
    @staticmethod
    def validate_interest_rate(interest_rate):
        # match a string of integer or float with arbitrary decimal places, allowing prefixing or trailing whitespaces
        pattern = r'^\s*[+-]?\d+(\.\d+)?\s*$'
        return bool(re.match(pattern, interest_rate))

    @staticmethod
    def validate_amount(amount):
        # match a string of positive integer or float with 2 decimal places at most, allowing prefixing or trailing whitespaces
        pattern = r'^\s*\d+(\.\d{1,2})?\s*$'
        return re.match(pattern, amount) and float(amount) > 0
    
    @staticmethod
    def get_timezone(hours_offset):
        if hours_offset is None:
            return timezone.utc
        return timezone(timedelta(hours=hours_offset))

    def __init__(self, firstname, lastname, hours_offset=None):
        self._account_id = self.generate_account_id()
        self.firstname = firstname
        self.lastname = lastname
        self.preferred_timezone = hours_offset
        self._balance = Decimal(0)

    @property
    def account_id(self):
        return self._account_id
    
    @property
    def firstname(self):
        return self._firstname
    
    @firstname.setter
    def firstname(self, firstname):
        # firstname = firstname.strip().capitalize()
        # # if self.validate_name(firstname):
        # if firstname.isalpha():
        #     self._firstname = firstname
        #     self._fullname = None
        # else:
        #     raise ValueError('name should be non-empty and can only contain alphabets')
        self._firstname = self.formalize_name(firstname, 'first name')
        self._fullname = None
        
    @property
    def lastname(self):
        return self._lastname
    
    @lastname.setter
    def lastname(self, lastname):
        # lastname = lastname.strip().capitalize()
        # if lastname.isalpha():
        #     self._lastname = lastname
        #     self._fullname = None
        # else:
        #     raise ValueError('name should be non-empty and can only contain alphabets')
        self._lastname = self.formalize_name(lastname, 'last name')
        self._fullname = None
        
    @property
    def fullname(self):
        if self._fullname is None:
            self._fullname = f'{self._firstname} {self._lastname}'
        return self._fullname
    
    @property
    def preferred_timezone(self):
        return self._preferred_timezone
    
    @preferred_timezone.setter
    def preferred_timezone(self, hours_offset):
        self._preferred_timezone = self.get_timezone(hours_offset)

    @property
    def preferred_timezone_name(self):
        return str(self._preferred_timezone)
    
    @property
    def balance(self):
        return f'{self._balance:.2f}' # stringified and rounded
    
    # use class method instead
    # @property
    # def monthly_interest_rate(self):
    #     return str(self._monthly_interest_rate)
    
    def deposit(self, amount):
        if self.validate_amount(amount):
            self._balance += Decimal(amount)
            return self.generate_transaction_code('deposit')
        else:
            raise ValueError('Internal error: the amount should be a string of positive integer or float with 2 decimal places at most')

    def withdraw(self, amount):
        if self.validate_amount(amount):
            amount = Decimal(amount)
            if self._balance >= amount:
                self._balance -= amount
                return self.generate_transaction_code('withdraw')
            else:
                return self.generate_transaction_code('declined')
        else:
            raise ValueError('Internal error: the amount should be a string of positive integer or float with 2 decimal places at most')

    def pay_interest(self):
        self._balance *= (Decimal(1) + self._monthly_interest_rate)
        return self.generate_transaction_code('pay_interest')

    def generate_transaction_code(self, transaction_type): # generate code like transaction_type - account_id - datetime - random_digits
        if transaction_type not in self._TRANSACTION_TYPES:
            raise ValueError('Internal error: invalid transaction type')
        type_code = self._TRANSACTION_TYPES[transaction_type]
        datetime_code = datetime.now(timezone.utc).strftime('%Y%m%d%H%M%S%f')
        random_code = self.generate_random_digits(6)
        return f'{type_code}-{self._account_id}-{datetime_code}-{random_code}'

In [2]:
import unittest

def run_tests(test_case_class):
    suite = unittest.TestLoader().loadTestsFromTestCase(test_case_class)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(suite)

class BankAccountTestCase(unittest.TestCase):
    # to define instructions (test fixture) that will be executed immediately before each test method
    # def setUp(self):
    #     pass

    # to define instructions (test fixture) that will be executed immediately after each test method even if the test method raise an exception
    # def tearDown(self):
    #     pass

    def test_get_monthly_interest_rate(self):
        self.assertEqual(BankAccount.get_monthly_interest_rate(), '0.005')

    def test_set_monthly_interest_rate(self):
        # exception cases
        self.assertRaises(ValueError, BankAccount.set_monthly_interest_rate, '')
        self.assertRaises(ValueError, BankAccount.set_monthly_interest_rate, '.1')
        self.assertRaises(ValueError, BankAccount.set_monthly_interest_rate, 'a')
        self.assertRaises(ValueError, BankAccount.set_monthly_interest_rate, 'inf')
        self.assertRaises(ValueError, BankAccount.set_monthly_interest_rate, '-inf')
        self.assertRaises(ValueError, BankAccount.set_monthly_interest_rate, 'infinity')
        self.assertRaises(ValueError, BankAccount.set_monthly_interest_rate, 'nan')
        self.assertRaises(ValueError, BankAccount.set_monthly_interest_rate, '1a')
        self.assertRaises(ValueError, BankAccount.set_monthly_interest_rate, '1..2')
        self.assertRaises(TypeError, BankAccount.set_monthly_interest_rate, 0.001)

        # success cases
        BankAccount.set_monthly_interest_rate('0.001')
        self.assertEqual(BankAccount.get_monthly_interest_rate(), '0.001')
        BankAccount.set_monthly_interest_rate('-0.001')
        self.assertEqual(BankAccount.get_monthly_interest_rate(), '-0.001')

    def test_generate_account_id(self):
        account_id = BankAccount.generate_account_id()
        self.assertEqual(len(account_id), 18)
        self.assertTrue(account_id.isdigit())
        self.assertTrue(account_id in BankAccount._account_ids)

        account_id = BankAccount.generate_account_id(length=16)
        self.assertEqual(len(account_id), 16)

    def test_get_timezone(self):
        self.assertEqual(BankAccount.get_timezone(None), timezone.utc)
        self.assertEqual(BankAccount.get_timezone(10), timezone(timedelta(hours=10)))

    def test_instance_properties(self):
        firstname = 'john'
        lastname = 'cleese'
        hours_offset = 10
        ba1 = BankAccount(firstname, lastname, hours_offset)

        self.assertTrue(ba1.account_id in BankAccount._account_ids)
        self.assertEqual(ba1.firstname, firstname.capitalize())
        self.assertEqual(ba1.lastname, lastname.capitalize())
        self.assertEqual(ba1.fullname, f'{firstname.capitalize()} {lastname.capitalize()}')
        self.assertEqual(ba1.preferred_timezone, timezone(timedelta(hours=hours_offset)))
        self.assertEqual(ba1.balance, '0.00')

        firstname = ' john '
        lastname = ' cleese '
        ba2 = BankAccount(firstname, lastname)

        self.assertTrue(ba2.account_id in BankAccount._account_ids)
        self.assertNotEqual(ba2.account_id, ba1.account_id)
        self.assertEqual(ba2.firstname, firstname.strip().capitalize())
        self.assertEqual(ba2.lastname, lastname.strip().capitalize())
        self.assertEqual(ba2.preferred_timezone, timezone.utc)
        self.assertEqual(ba2.preferred_timezone_name, str(timezone.utc))
        self.assertEqual(ba2.balance, '0.00')

        # exception cases
        self.assertRaises(ValueError, setattr, ba1, 'firstname', '')
        self.assertRaises(ValueError, setattr, ba1, 'firstname', ' ')
        self.assertRaises(ValueError, setattr, ba1, 'firstname', '123')
        self.assertRaises(ValueError, setattr, ba1, 'firstname', '123aa')
        self.assertRaises(ValueError, setattr, ba1, 'firstname', 'john cleese')
        self.assertRaises(ValueError, setattr, ba1, 'lastname', '')
        self.assertRaises(ValueError, setattr, ba1, 'lastname', ' ')
        self.assertRaises(ValueError, setattr, ba1, 'lastname', '123')
        self.assertRaises(ValueError, setattr, ba1, 'lastname', '123aa')
        self.assertRaises(ValueError, setattr, ba1, 'lastname', 'john cleese')

    def test_instance_methods(self):
        ba = BankAccount('john', 'cleese')
        balance = Decimal(ba.balance)

        # deposit
        ba.deposit('100.3')
        self.assertEqual(ba.balance, f'{balance + Decimal('100.3'):.2f}')
        # exception cases
        self.assertRaises(ValueError, ba.deposit, '')
        self.assertRaises(ValueError, ba.deposit, ' ')
        self.assertRaises(ValueError, ba.deposit, '0')
        self.assertRaises(ValueError, ba.deposit, '0.0')
        self.assertRaises(ValueError, ba.deposit, 'inf')
        self.assertRaises(ValueError, ba.deposit, 'nan')
        self.assertRaises(ValueError, ba.deposit, '-1')
        self.assertRaises(TypeError, ba.deposit, 100)

        # withdraw
        ba.withdraw('10.29')
        self.assertEqual(ba.balance, f'{balance + Decimal('100.3') - Decimal('10.29'):.2f}')

        transaction_code = ba.withdraw('1000')
        self.assertEqual(transaction_code[0], 'X')
        
        # exception cases
        self.assertRaises(ValueError, ba.withdraw, '')
        self.assertRaises(ValueError, ba.withdraw, ' ')
        self.assertRaises(ValueError, ba.withdraw, '0')
        self.assertRaises(ValueError, ba.withdraw, '0.0')
        self.assertRaises(ValueError, ba.withdraw, 'inf')
        self.assertRaises(ValueError, ba.withdraw, 'nan')
        self.assertRaises(ValueError, ba.withdraw, '-1')
        self.assertRaises(TypeError, ba.withdraw, 100)

    def test_generate_transaction_code(self):
        ba = BankAccount('john', 'cleese')

        self.assertRaises(ValueError, ba.generate_transaction_code, 'other')

    def test_parse_transaction_code(self):
        ba = BankAccount('ben', 'son')
        transaction_code = ba.deposit('100')
        transaction = BankAccount.parse_transaction_code(transaction_code, timezone.utc)

        self.assertTrue(isinstance(transaction, Transaction))
        self.assertTrue(transaction.transaction_type in BankAccount._TRANSACTION_TYPES)
        self.assertEqual(transaction.account_id, ba.account_id)
        self.assertEqual(transaction.transaction_datetime, transaction.transaction_datetime_preferred)
        self.assertEqual(len(transaction.random_code), 6)
        self.assertTrue(transaction.random_code.isdigit())

run_tests(BankAccountTestCase)

test_generate_account_id (__main__.BankAccountTestCase.test_generate_account_id) ... ok
test_generate_transaction_code (__main__.BankAccountTestCase.test_generate_transaction_code) ... ok
test_get_monthly_interest_rate (__main__.BankAccountTestCase.test_get_monthly_interest_rate) ... ok
test_get_timezone (__main__.BankAccountTestCase.test_get_timezone) ... ok
test_instance_methods (__main__.BankAccountTestCase.test_instance_methods) ... ok
test_instance_properties (__main__.BankAccountTestCase.test_instance_properties) ... ok
test_parse_transaction_code (__main__.BankAccountTestCase.test_parse_transaction_code) ... ok
test_set_monthly_interest_rate (__main__.BankAccountTestCase.test_set_monthly_interest_rate) ... ok

----------------------------------------------------------------------
Ran 8 tests in 0.021s

OK


## Polymorphism and special methods

### Introduction

Polymorphism

* Polymorphism is a fundamental concept in object-oriented programming (OOP).
* It refers to the ability to define a generic type of behavior that will potentially behave differently when applied to different types.
* Essentially, polymorphism creates a structure that can adapt to various forms of objects.
* Python is polymorphic in nature.
  * duck typing: if it walks like a duck and quacks like a duck then it is a duck
  * operators are polymorphic
* In Python, polymorphism manifests in several ways:
  * function polymorphism
  * class polymorphism
  * inheritance class polymorphism

Special methods

* Special methods, also known as "dunder" (double-underscore) methods, are instance methods reserved by Python.
* Special methods allow you to define custom behavior for your classes when interacting with operators (operator overloading) or other language constructs.
* A class can implement certain operations that are invoked by special syntax (such as arithmetic operations or subscripting and slicing) by defining methods with special names.
* The naming convention for special methods is `__<name>__`, where the double underscores precede and succeed the method name.
* Setting a special method to `None` indicates that the corresponding operation is not available.
  * For example, if a class sets `__iter__()` to `None`, the class is not iterable, so calling `iter()` on its instances will raise a `TypeError` (without falling back to `__getitem__()`).

### `__str__` and `__repr__` methods

* `__str__` and `__repr__` are special methods related to string representation of objects.
  * `__repr__(self)`
    * Returns a string for a printable representation of the object.
    * Called by the `repr()` built-in function or when an object is displayed in a console or notebook.
    * This should look like a valid Python expression that could be used to recreate an object with the same value.
  * `__str__(self)`
    * Called by `str()` and the built-in functions `format()` and `print()` to compute the printable string representation of an object. The return value must be a string object.
    * This method differs from `__repr__()` in that there is no expectation that `__str__()` return a valid Python expression: a more convenient or concise representation can be used.
    * If `__str__` is not implemented, Python will look for `__repr__` instead.
    * If neither is implemented, the default implementation defined by the built-in type `object` calls `object.__repr__()`.

In [3]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age

    def __repr__(self):
        return f'Person(name={self.name}, age={self.age})'
    
    def __str__(self):
        return f'A person called {self.name} at age {self.age}'
    
person = Person('John', 20)
print(person)
print(repr(person))
print(f'Person: {person}') # use __str__

class Point:
    pass

point = Point()
print(point) # use __repr__ from object

A person called John at age 20
Person(name=John, age=20)
Person: A person called John at age 20


### Arithmetic operators

*  Methods corresponding to operations that are not supported by the particular kind of object implemented should be left undefined.
* If one of those methods does not support the operation with the supplied arguments, it should return `NotImplemented`.
* The following methods are called to implement the binary arithmetic operations.
  * `__add__(self, other)`: `+`
  * `__sub__(self, other)`: `-`
  * `__mul__(self, other)`: `*`
  * `__truediv__(self, other)`: `\`
  * `__floordiv__(self, other)`: `\\`
  * `__mod__(self, other)`: `%`
  * `__pow__(self, other[, modulo])`: `**`, `pow()`
  * `__matmul__(self, other)`: `@` (matrix multiplication)
  * `__divmod__(self, other)`: `divmod()`
  * `__lshift__(self, other)`: `<<`
  * `__rshift__(self, other)`: `>>`
  * `__and__(self, other)`: `&`
  * `__xor__(self, other)`: `^`
  * `__or__(self, other)`: `|`
* The following methods are called to implement the binary arithmetic operations with reflected (swapped) operands.
  * `__radd__(self, other)`: `+`
  * `__rsub__(self, other)`: `-`
  * `__rmul__(self, other)`: `*`
  * `__rmatmul__(self, other)`: `@`
  * `__rtruediv__(self, other)`: `\`
  * `__rfloordiv__(self, other)`: `\\`
  * `__rmod__(self, other)`: `%`
  * `__rdivmod__(self, other)`: `divmod()`
  * `__rpow__(self, other[, modulo])`: `pow()`, `**`
  * `__rlshift__(self, other)`: `<<`
  * `__rrshift__(self, other)`: `>>`
  * `__rand__(self, other)`: `&`
  * `__rxor__(self, other)`: `^`
  * `__ror__(self, other)`: `|`
  * These functions are only called if the left operand does not support the corresponding operation and the operands are of different types.
    * For instance, to evaluate the expression `x - y`, where `y` is an instance of a class that has an `__rsub__()` method, `type(y).__rsub__(y, x)` is called if `type(x).__sub__(x, y)` returns `NotImplemented`.
  * Note that ternary `pow()` will not try calling `__rpow__()` (the coercion rules would become too complicated).
  * If the right operand's type is a subclass of the left operand's type and that subclass provides a different implementation of the reflected method for the operation, this method will be called before the left operand's non-reflected method. This behavior allows subclasses to override their ancestors' operations.
* The following methods are called to implement the augmented arithmetic assignments.
  * `__iadd__(self, other)`: `+=`
  * `__isub__(self, other)`: `-=`
  * `__imul__(self, other)`: `*=`
  * `__imatmul__(self, other)`: `@=`
  * `__itruediv__(self, other)`: `\=`
  * `__ifloordiv__(self, other)`: `\\=`
  * `__imod__(self, other)`: `%=`
  * `__ipow__(self, other[, modulo])`: `**=`
  * `__ilshift__(self, other)`: `<<=`
  * `__irshift__(self, other)`: `>>=`
  * `__iand__(self, other)`: `&=`
  * `__ixor__(self, other)`: `^=`
  * `__ior__(self, other)`: `|=`
  * These methods should attempt to do the operation in-place (modifying `self`) and return the result (which could be, but does not have to be, `self`).
  * If a specific method is not defined, the augmented assignment falls back to the normal methods.
    * For instance, if `x` is an instance of a class with an `__iadd__()` method, `x += y` is equivalent to `x = x.__iadd__(y)` . Otherwise, `x.__add__(y)` and `y.__radd__(x)` are considered, as with the evaluation of `x + y`.
  * In certain situations, augmented assignment can result in unexpected errors (see [Why does a_tuple[i] += ['item'] raise an exception when the addition works?](https://docs.python.org/3/faq/programming.html#faq-augmented-assignment-tuple-error)), but this behavior is in fact part of the data model.
* The following methods are called to implement the unary arithmetic operations.
  * `__neg__(self)`: `-`
  * `__pos__(self)`: `+`
  * `__abs__(self)`: `abs()`
  * `__invert__(self)`: `~`
* The following methods are called to implement the built-in functions.
  * `__complex__`: `complex()`
  * `__float__`: `float()`
  * `__int__`: `int()`
  * Should return a value of the appropriate type.
* `__index__(self)`
  * Called to implement `operator.index()`, and whenever Python needs to losslessly convert the numeric object to an integer object (such as in slicing, or in the built-in `bin()`, `hex()` and `oct()` functions).
  * Presence of this method indicates that the numeric object is an integer type. Must return an integer.
  * If `__int__()`, `__float__()` and `__complex__()` are not defined then corresponding built-in functions `int()`, `float()` and `complex()` fall back to `__index__()`.
* The following methods are called to implement the built-in function.
  * `__round__(self[, ndigits])`: `round()`
  * `__trunc__(self)`: `math.trunc()`
  * `__floor__(self)`: `math.floor()`
  * `__ceil__(self)`: `math.ceil()`
  * The built-in function `int()` falls back to `__trunc__()` if neither `__int__()` nor `__index__()` is defined.
  * Changed in version 3.11: The delegation of `int()` to `__trunc__()` is deprecated.

In [35]:
from numbers import Real
from operator import add, sub, mul, neg
class Vector:
    def __init__(self, *components):
        if len(components) < 1:
            raise ValueError('vector cannot be empty')
        # for component in components:
        #     if not isinstance(component, Real):
        #         raise ValueError(f'Vector components must be all real numbers ({component} is invalid)')
        if not all(map(lambda x: isinstance(x, Real), components)):
            raise ValueError('vector components must be all real numbers')
        self._component = components

    @property
    def component(self):
        return self._component

    def __repr__(self):
        return f'Vector{self._component}'
    
    def __len__(self):
        return len(self._component)
    
    def __add__(self, other):
        if isinstance(other, Vector):
            if len(other) == len(self):
                return Vector(*map(add, self._component, other._component)) # can also use generator comprehension here
            else:
                raise ValueError('vectors must be of the same length')
        else:
            return NotImplemented

    def __sub__(self, other):
        if isinstance(other, Vector):
            if len(other) == len(self):
                return Vector(*map(sub, self._component, other._component)) # can also use generator comprehension here
            else:
                raise ValueError('vectors must be of the same length')
        else:
            return NotImplemented

    def __mul__(self, other):
        if isinstance(other, Real): # scalar product
            return Vector(*(component * other for component in self._component)) # can also use generator comprehension here
        if isinstance(other, Vector): # dot product
            return sum(map(mul, self._component, other._component))
        return NotImplemented
        
    def __rmul__(self, other):
        # return self.__mul__(other)
        return self * other

    def __iadd__(self, other):
        # version 1: the id of self will be changed
        # return self + other
    
        # version 2: the id of self will remain the same
        if isinstance(other, Vector):
            if len(other) == len(self):
                self._component = tuple(map(add, self._component, other._component))
                return self
            else:
                raise ValueError('vectors must be of the same length')
        else:
            return NotImplemented
        
    def __isub__(self, other):
        if isinstance(other, Vector):
            if len(other) == len(self):
                self._component = tuple(map(sub, self._component, other._component))
                return self
            else:
                raise ValueError('vectors must be of the same length')
        else:
            return NotImplemented
        
    def __neg__(self):
        return Vector(*map(neg, self._component))
    
    def __abs__(self):
        return sum(map(lambda component: component ** 2, self._component)) ** 0.5
    

v1 = Vector(1, 2, 3)
print(v1)
print(v1.component)
print(len(v1))
v2 = Vector(4, 5, 6)
print(v1 + v2)
v3 = Vector(1)
# print(v1 + 1) # TypeError, unsupported operand type
# print(v1 + v3) # ValueError, different lengths
print(v2 - v1)
print(v1 * 2)
# print(v1 * 1j) # TypeError, unsupported operand type
print(2 * v1)
print(v1 * v2)
print(id(v1))
v1 += v2
print(id(v1), v1)
# v1 += v3
# v1 += 1
v2 -= v1
print(v2)
print(-v2)
print(abs(Vector(1, 1)))
print(abs(Vector(2, 2)))

Vector(1, 2, 3)
(1, 2, 3)
3
Vector(5, 7, 9)
Vector(3, 3, 3)
Vector(2, 4, 6)
Vector(2, 4, 6)
32
1889462894960
1889462894960 Vector(5, 7, 9)
Vector(-1, -2, -3)
Vector(1, 2, 3)
1.4142135623730951
2.8284271247461903


### Rich comparisons

* The following methods are the so-called rich comparison methods.
  * `__lt__(self, other)`: `<`
  * `__le__(self, other)`: `<=`
  * `__eq__(self, other)`: `==`
  * `__ne__(self, other)`: `!=`
  * `__gt__(self, other)`: `>`
  * `__ge__(self, other)`: `>=`
* A rich comparison method may return the singleton `NotImplemented` if it does not implement the operation for a given pair of arguments.
* By convention, `False` and `True` are returned for a successful comparison. However, these methods can return any value, so if the comparison operator is used in a Boolean context (e.g. in the condition of an `if` statement), Python will call `bool()` on the value to determine if the result is true or false.
* By default, object implements `__eq__()` by using `is`, returning `NotImplemented` in the case of a false comparison: `True if x is y else NotImplemented`. For `__ne__()`, by default it delegates to `__eq__()` and inverts the result unless it is `NotImplemented`.
* There are no other implied relationships among the comparison operators or default implementations.
  * For example, the truth of `(x<y or x==y)` does not imply `x<=y`. To automatically generate ordering operations from a single root operation, see `functools.total_ordering()`.
* There are no swapped-argument versions of these methods; rather, `__lt__()` and `__gt__()` are each other's reflection, `__le__()` and `__ge__()` are each other's reflection, and `__eq__()` and `__ne__()` are their own reflection.
* If the operands are of different types, and right operand's type is a direct or indirect subclass of the left operand's type, the reflected method of the right operand has priority, otherwise the left operand's method has priority. Virtual subclassing is not considered.

In [39]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __repr__(self):
        return f'Point({self.x}, {self.y})'
    
    def __abs__(self):
        return (self.x ** 2 + self.y ** 2) ** 0.5
    
    def __eq__(self, other):
        if isinstance(other, (list, tuple)):
            other = Point(*other)
        if isinstance(other, Point):
            return self.x == other.x and self.y == other.y
        return NotImplemented
    
    def __lt__(self, other):
        if isinstance(other, (list, tuple)):
            other = Point(*other)
        if isinstance(other, Point):
            return abs(self) < abs(other)
        
    def __le__(self, other):
        # method 1:
        # if isinstance(other, (list, tuple)):
        #     other = Point(*other)
        # if isinstance(other, Point):
        #     return abs(self) <= abs(other)

        # method 2:
        return self < other or self == other
        
p1 = Point(0, 0)
p2 = Point(1, 1)
p3 = Point(1, 1)
print(p1 == p2)
print(p1 != p2)
print(p3 == p2)
print(p1 < p2)
print(p1 > p2)
print(p1 <= p2)
print(p1 >= p2)
# print((0, 0) < p2) # TypeError, not implemented in tuple, python will try flip the comparison, i.e. p2 > (0, 0), but __gt__ is not implemented      

False
True
True
True
False
True
False


In [41]:
from functools import total_ordering

@total_ordering
class Number:
    def __init__(self, number):
        self.number = number

    def __repr__(self):
        return f'Number({self.number})'
    
    def __eq__(self, other): # if __eq__ is not implemented, python will use the default `is` to check equality
        if isinstance(other, Number):
            return self.number == other.number
        return NotImplemented
    
    def __lt__(self, other):
        if isinstance(other, Number):
            return self.number < other.number
        
number1 = Number(1)
number2 = Number(2)
number3 = Number(1)
print(number1 < number2)
print(number1 <= number2)
print(number1 > number2)
print(number1 >= number2)
print(number1 >= number3)
print(number1 == number3)
print(number1 != number3)

True
True
False
False
True
True
False


### Hashing and equality

* `__hash__(self)`
  * Called by built-in function `hash()` and for operations on members of hashed collections including `set`, `frozenset`, and `dict`.
  * The `__hash__()` method should return an integer.
  * The only required property is that objects which compare equal have the same hash value; it is advised to mix together the hash values of the components of the object that also play a part in comparison of objects by packing them into a tuple and hashing the tuple.
  * If a class does not define an `__eq__()` method it should not define a `__hash__()` operation either.
  * If a class defines `__eq__()` but not `__hash__()`, its instances will not be usable as items in hashable collections.
  * If a class defines mutable objects and implements an `__eq__()` method, it should not implement `__hash__()`, since the implementation of hashable collections requires that a key's hash value is immutable (if the object's hash value changes, it will be in the wrong hash bucket).
  * User-defined classes have `__eq__()` and `__hash__()` methods by default; with them, all objects compare unequal (except with themselves) and `x.__hash__()` returns an appropriate value such that `x == y` implies both that `x is y` and `hash(x) == hash(y)`.
  * A class that overrides `__eq__()` and does not define `__hash__()` will have its `__hash__()` implicitly set to `None`. When the `__hash__()` method of a class is `None`, instances of the class will raise an appropriate `TypeError` when a program attempts to retrieve their hash value, and will also be correctly identified as unhashable when checking `isinstance(obj, collections.abc.Hashable)`.
  * If a class that overrides `__eq__()` needs to retain the implementation of `__hash__()` from a parent class, the interpreter must be told this explicitly by setting `__hash__ = <ParentClass>.__hash__`.
  * If a class that does not override `__eq__()` wishes to suppress hash support, it should include `__hash__ = None` in the class definition. A class which defines its own `__hash__()` that explicitly raises a `TypeError` would be incorrectly identified as hashable by an `isinstance(obj, collections.abc.Hashable)` call.
  * By default, the `__hash__()` values of `str` and `bytes` objects are *salted* with an unpredictable random value. Although they remain constant within an individual Python process, they are *not* predictable between repeated invocations of Python.
    * Changing hash values affects the iteration order of sets. Python has never made guarantees about this ordering (and it typically varies between 32-bit and 64-bit builds).

### Booleans

* `__bool__(self)`
  * Called to implement truth value testing and the built-in operation `bool()`; should return `False` or `True`.
  * When this method is not defined, `__len__()` is called, if it is defined, and the object is considered true if its result is nonzero.
  * If a class defines neither `__len__()` nor `__bool__()`, all its instances are considered true.

### Callables

* `__call__(self[, args...])`
  * Called when the instance is called as a function. 
  * If this method is defined, `x(arg1, arg2, ...)` roughly translates to `type(x).__call__(x, arg1, ...)`.
  * Useful for creating function-like objects that need to maintain state.
  * Useful for creating decorator classes.

In [46]:
from functools import partial

print(f'{type(partial)=}') # type, partial is actually a class

def func(a, b, c):
    return a, b, c

partial_func = partial(func, 1, 2)
print(f'{type(partial_func)=}') # an instance of partial class which is callable
print(f'{callable(partial_func)=}')
print(f'{partial_func(3)=}') # (1, 2, 3)

type(partial)=<class 'type'>
type(partial_func)=<class 'functools.partial'>
callable(partial_func)=True
partial_func(3)=(1, 2, 3)


In [51]:
from collections import defaultdict

class DefaultValue:
    def __init__(self, default_value):
        self.default_value = default_value
        self.count = 0

    def __call__(self):
        self.count += 1
        return self.default_value

default_value1 = DefaultValue(0)
default_value2 = DefaultValue(None)

dd1 = defaultdict(default_value1)
dd2 = defaultdict(default_value2)

print(dd1['a'], dd1['b'], dd2['a'], dd2['b'], dd2['c'])
print(default_value1.count, default_value2.count)

0 0 None None None
2 3


In [3]:
from functools import wraps
from time import perf_counter, sleep
import random

def profiler(fn):
    count = 0
    total_time = 0
    average_time = 0

    @wraps(fn)
    def inner(*args, **kwargs):
        nonlocal count
        nonlocal total_time
        nonlocal average_time
        count += 1
        start = perf_counter()
        result = fn(*args, **kwargs)
        end = perf_counter()
        total_time += end - start
        average_time = total_time / count
        return result
    
    # this approach doesn't work, because the value of inner.count is defined when defining the profiler function
    inner.count = count
    inner.total_time = total_time
    inner.average_time = average_time
    return inner

@profiler
def random_sleep():
    sleep(random.random())

random.seed(0)
print(random_sleep(), random_sleep())
print(random_sleep.count, random_sleep.total_time, random_sleep.average_time)

None None
0 0 0


In [17]:
from functools import wraps
from time import perf_counter, sleep
import random

def profiler(fn):
    count = 0
    total_time = 0

    @wraps(fn)
    def inner(*args, **kwargs):
        nonlocal count
        nonlocal total_time
        count += 1
        start = perf_counter()
        result = fn(*args, **kwargs)
        end = perf_counter()
        total_time += end - start
        return result
    
    def get_count():
        return count
    
    def get_total_time():
        return total_time
    
    def get_average_time():
        return total_time / count
    
    inner.count = get_count
    inner.total_time = get_total_time
    inner.average_time = get_average_time

    return inner

@profiler
def random_sleep():
    sleep(random.random())

random.seed(0)
print(random_sleep(), random_sleep())
print(random_sleep.count(), random_sleep.total_time(), random_sleep.average_time())

None None
2 1.6031541999836918 0.8015770999918459


In [16]:
from time import perf_counter, sleep
import random
# from functools import wraps

class Profiler:
    def __init__(self, fn):
        self.count = 0
        self.total_time = 0
        self.fn = fn

    def __call__(self, *args, **kwargs):
        self.count += 1
        start = perf_counter()
        result = self.fn(*args, **kwargs)
        self.total_time += perf_counter() - start
        return result
    
    @property
    def average_time(self): # only calculate average_time when needed
        return self.total_time / self.count

@Profiler
def random_sleep():
    sleep(random.random())

random.seed(0)
print(type(random_sleep)) # random_sleep function becomes an instance of Profiler class
print(random_sleep(), random_sleep())
print(random_sleep.count, random_sleep.total_time, random_sleep.average_time)

<class '__main__.Profiler'>
None None
2 1.6031223999743816 0.8015611999871908


## `__del__` method

* The garbage collector destroys objects that are no longer referenced anywhere.
* The `__del__` method will get called right before the object is destroyed by the garbage collector. So the GC determines when this method is called.
* `__del__(self)`
  * Called when the instance is about to be destroyed.
  * This is also called a finalizer or improperly a destructor, because it is GC that destroys an instance, all `__del__` doing is injecting code that runs immediately before the instance being destroyed.
  * If a base class has a `__del__()` method, the derived class's `__del__()` method, if any, must explicitly call it to ensure proper deletion of the base class part of the instance.
  * It is possible (though not recommended!) for the `__del__()` method to postpone destruction of the instance by creating a new reference to it. This is called *object resurrection*. It is implementation-dependent whether `__del__()` is called a second time when a resurrected object is about to be destroyed; the current CPython implementation only calls it once.
  * It is not guaranteed that `__del__()` methods are called for objects that still exist when the interpreter exits.
  * `del x` doesn't directly call `x.__del__()`. The former decrements the reference count for `x` by one, and the latter is only called when `x`'s reference count reaches zero.
  * Due to the precarious circumstances under which `__del__()` methods are invoked, exceptions that occur during their execution are *ignored*, and a warning is printed to `sys.stderr` instead.
* Maybe using context managers instead of `__del__` is better to clean up resources.

In [4]:
import ctypes

def ref_count(address):
    return ctypes.c_long.from_address(address).value

class Person:
    def __init__(self, name):
        self.name = name

    def __repr__(self):
        return f'Person({self.name})'
    
    def __del__(self):
        print(f'__del__ called for {self}')

    def raise_exception(self):
        raise ValueError

p = Person('Alex')
ref_count(id(p))
p = None
p = Person('John')
ref_count(id(p))
del p
p = Person('Bonny')
ref_count(id(p))
try:
    p.raise_exception()
except ValueError as ex:
    error = ex
ref_count(id(p)) # 2, because p is referenced in the error object
for k, v in error.__traceback__.tb_frame.f_locals.copy().items():
    if isinstance(v, Person):
        print(k, v, id(v) == id(p))
del p
del error # the __del__ finally get called here when p's reference count becomes 0

__del__ called for Person(Alex)
__del__ called for Person(John)
p Person(Bonny) True
__del__ called for Person(Bonny)


In [2]:
import sys

class Person:
    def __del__(self):
        raise ValueError('ValueError occurred in __del__, but it is ignored, a warning printed to stderr instead')

class ErrToFile:
    def __init__(self, filename):
        self._filename = filename
        self._stderr = sys.stderr
        self._file = None

    def __enter__(self):
        self._file = open(self._filename, 'w')
        sys.stderr = self._file

    def __exit__(self, exc_type, exc_value, traceback):
        sys.stderr = self._stderr
        if self._file:
            self._file.close()
        return False
    
p = Person()
with ErrToFile('temp.txt'):
    del p
    print(1)
print(2)

with open('temp.txt') as f:
    print(f.readlines())

1
2
['Exception ignored in: <function Person.__del__ at 0x000001389302E7A0>\n', 'Traceback (most recent call last):\n', '  File "C:\\Users\\FREEDEMPIRE-XL\\AppData\\Local\\Temp\\ipykernel_2244\\3856971142.py", line 5, in __del__\n', 'ValueError: ValueError occurred in __del__, but it is ignored, a warning printed to stderr instead\n']


### `__format__` method

* `__format__(self, format_spec)`
  * Called by the `format()` built-in function, and by extension, evaluation of formatted string literals and the `str.format()` method, to produce a formatted string representation of an object.
  * The `format_spec` argument is a string that contains a description of the formatting options desired. The interpretation of the `format_spec` argument is up to the type implementing `__format__()`, however most classes will either delegate formatting to one of the built-in types, or use a similar formatting option syntax.
  * See [Format Specification Mini-Language](https://docs.python.org/3/library/string.html#formatspec) for a description of the standard formatting syntax.
  * The return value must be a string object.
  * Changed in version 3.4: The `__format__` method of `object` itself raises a `TypeError` if passed any non-empty string.
  * Changed in version 3.7: `object.__format__(x, '')` is now equivalent to `str(x)` rather than `format(str(x), '')`.
* `format(value, format_spec='')`
  * Convert a value to a formatted representation, as controlled by `format_spec`. The interpretation of format_spec will depend on the type of the value argument; however, there is a standard formatting syntax that is used by most built-in types: Format Specification Mini-Language.
  * The default `format_spec` is an empty string which usually gives the same effect as calling `str(value)`.
  * A call to `format(value, format_spec)` is translated to `type(value).__format__(value, format_spec)` which bypasses the instance dictionary when searching for the value's `__format__()` method. A `TypeError` exception is raised if the method search reaches `object` and the `format_spec` is non-empty, or if either the `format_spec` or the return value are not strings.
  * Changed in version 3.4: `object().__format__(format_spec)` raises `TypeError` if `format_spec` is not an empty string.

In [4]:
from datetime import datetime

num = 1.1
print(format(num, '.20f'))
current_dt = datetime.now()
print(format(current_dt, '%a %Y-%m-%d %I:%M %p'))

1.10000000000000008882
Wed 2024-02-21 08:20 AM


In [3]:
from datetime import date

class Person:
    def __init__(self, name, dob):
        self.name = name
        self.dob = dob

    def __repr__(self):
        return f'Person(name={self.name}, dob={self.dob})'
    
    def __str__(self):
        return f'A person named {self.name}'
    
    def __format__(self, date_format_spec):
        dob = format(self.dob, date_format_spec) # delegate formatting to Date type
        return f'Person({self.name}, {dob})'
    
p = Person('John', date(2020, 1, 1))
print(p) # __str__ get called
print(repr(p)) # __repr__ get called
print(format(p)) # __format__ get called, dob = str(self.dob)
print(format(p, '%B %d, %Y')) # __format__ get called with non-empty date_format_spec

A person named John
Person(name=John, dob=2020-01-01)
Person(John, 2020-01-01)
Person(John, January 01, 2020)


## Project 2

Modular arithmetic

* Modular arithmetic, also known as clock arithmetic, is a system of arithmetic for integers.
* Given an integer m (where m ≥ 1), modular arithmetic involves two integers a and b. We say that a and b are congruent modulo m if m is a divisor of their difference. In other words, there exists an integer k such that a − b = km.

Project

* Create a class `Mod` initializing with `value` and `modulus` arguments.
* `modulus` and ` value` should be read-only, integral, and moreover, `modulus` should be positive.
* Store the value as the remainder.
* Implement congruence for the `==` operator.
  * Allow comparison of a `Mod` object to an `int` (in which case use the residue / remainder of the `int`).
  * Allow comparison of two `Mod` objects only if they have the same modulus.
  * Ensure objects remain hashable.
* Provide an implementation so that `int(mod_object)` will return the `value` attribute (i.e. the remainder).
* Provide a proper representation (`repr`).
* Implement the operators: `+`, `-`, `*`, `**`
  * Support other operand to be a `Mod` object (with same modulus).
  * Support other operand to be an `int` (use the same modulus).
  * Always return a `Mod` instance.
  * Perform the operations on the values.
* Implement the corresponding in-place arithmetic operators.
* Implement ordering.
  * Support other operand to be a `Mod` (with same modulus), or an `int`.

In [1]:
from functools import total_ordering
import operator

@total_ordering
class Mod:
    def __init__(self, value, modulus):
        if not isinstance(modulus, int):
            raise TypeError('modulus should be of int type')
        if modulus <= 0:
            raise ValueError('modulus should be positive')
        if not isinstance(value, int):
            raise TypeError('value should be of int type')
        
        self._modulus = modulus
        self._value = value % modulus

    @property
    def value(self):
        return self._value
    
    @property
    def modulus(self):
        return self._modulus
    
    def _get_value(self, other):
        if isinstance(other, int):
            return other % self._modulus
        if isinstance(other, Mod):
            if self._modulus == other._modulus:
                return other._value
            else:
                raise ValueError('operation unsupported between Mod instances with different moduli')
        raise TypeError(f'unsupported operand type(s) for +: \'Mod\' and {type(other).__name__!r}')
    
    def _arithmetic_operation(self, other, operation, *, in_place=False):
        other_value = self._get_value(other)
        result = operation(self._value, other_value)
        if in_place:
            self._value = result % self._modulus
            return self
        return Mod(result, self._modulus)

    def __repr__(self):
        return f'Mod(value={self._value}, modulus={self._modulus})'
    
    def __lt__(self, other):
        return self._value < self._get_value(other)

    def __eq__(self, other):
        return self._value == self._get_value(other)
    
    def __hash__(self):
        return hash((self._value, self._modulus))
    
    def __int__(self):
        return self._value
    
    def __neg__(self):
        return Mod(-self._value, self._modulus)
    
    def __add__(self, other):
        return self._arithmetic_operation(other, operator.add)

    def __sub__(self, other):
        return self._arithmetic_operation(other, operator.sub)

    def __mul__(self, other):
        return self._arithmetic_operation(other, operator.mul)

    def __pow__(self, other):
        return self._arithmetic_operation(other, operator.pow)

    def __iadd__(self, other):
        return self._arithmetic_operation(other, operator.add, in_place=True)

    def __isub__(self, other):
        return self._arithmetic_operation(other, operator.sub, in_place=True)


    def __imul__(self, other):
        return self._arithmetic_operation(other, operator.mul, in_place=True)


    def __ipow__(self, other):
        return self._arithmetic_operation(other, operator.pow, in_place=True)

In [2]:
import unittest

def run_tests(test_case_class):
    suite = unittest.TestLoader().loadTestsFromTestCase(test_case_class)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(suite)

class ModTestCase(unittest.TestCase):
    def test_create_mod_instance(self):
        # exception cases
        self.assertRaises(TypeError, Mod, 1.1, 10)
        self.assertRaises(TypeError, Mod, 2, 1.1)
        self.assertRaises(TypeError, Mod, 2, 'a')
        self.assertRaises(ValueError, Mod, 2, 0)
        self.assertRaises(ValueError, Mod, 2, -2)

        # success cases
        self.assertEqual(Mod(15, 12).value, 3)
        self.assertEqual(Mod(15, 12).modulus, 12)

        # exception cases
        self.assertRaises(AttributeError, setattr, Mod(1, 1), 'value', 2)
        self.assertRaises(AttributeError, setattr, Mod(1, 1), 'modulus', 2)

    def test_eq(self):
        self.assertEqual(Mod(10, 12), Mod(22, 12))
        self.assertEqual(Mod(10, 12), 10)
        self.assertEqual(Mod(10, 12), 22)
        self.assertNotEqual(Mod(10, 12), 12)
        self.assertNotEqual(Mod(10, 12), Mod(9, 12))
        self.assertRaises(TypeError, lambda: Mod(10, 12) == 'a')
        self.assertRaises(ValueError, lambda: Mod(10, 12) == Mod(10, 11))

    def test_hash(self):
        self.assertEqual(hash(Mod(1, 2)), hash(Mod(3, 2)))
        self.assertNotEqual(hash(Mod(1, 2)), hash(Mod(2, 2)))

    def test_int(self):
        self.assertEqual(int(Mod(12, 12)), 0)
        self.assertEqual(int(Mod(13, 12)), 1)

    def test_neg(self):
        self.assertEqual(-Mod(1, 12), Mod(-1, 12))

    def test_add(self):
        self.assertEqual(Mod(10, 12) + 2, Mod(0, 12))
        self.assertEqual(Mod(10, 12) + Mod(2, 12), Mod(0, 12))
        self.assertRaises(ValueError, lambda: Mod(10, 12) + Mod(10, 2))
        self.assertRaises(TypeError, lambda: Mod(10, 12) + 3.1)
        self.assertRaises(TypeError, lambda: Mod(10, 12) + 'a')

    def test_iadd(self):
        mod1 = Mod(10, 12)
        mod2 = Mod(2, 12)
        id_mod = id(mod1)
        mod1 += 2
        self.assertEqual(mod1, Mod(0, 12))
        self.assertEqual(id_mod, id(mod1))
        mod1 += mod2
        self.assertEqual(mod1, Mod(2, 12))
        self.assertRaises(ValueError, lambda: Mod(10, 12).__iadd__(Mod(10, 2)))
        self.assertRaises(TypeError, lambda: Mod(10, 12).__iadd__(3.1))
        self.assertRaises(TypeError, lambda: Mod(10, 12).__iadd__('a'))

    def test_sub(self):
        self.assertEqual(Mod(10, 12) - 2, Mod(8, 12))
        self.assertEqual(Mod(10, 12) - Mod(2, 12), Mod(8, 12))
        self.assertRaises(ValueError, lambda: Mod(10, 12) - Mod(10, 2))
        self.assertRaises(TypeError, lambda: Mod(10, 12) - 3.1)
        self.assertRaises(TypeError, lambda: Mod(10, 12) - 'a')

    def test_isub(self):
        mod1 = Mod(10, 12)
        mod2 = Mod(2, 12)
        id_mod = id(mod1)
        mod1 -= 2
        self.assertEqual(mod1, Mod(8, 12))
        self.assertEqual(id_mod, id(mod1))
        mod1 -= mod2
        self.assertEqual(mod1, Mod(6, 12))
        self.assertRaises(ValueError, lambda: Mod(10, 12).__isub__(Mod(10, 2)))
        self.assertRaises(TypeError, lambda: Mod(10, 12).__isub__(3.1))
        self.assertRaises(TypeError, lambda: Mod(10, 12).__isub__('a'))

    def test_mul(self):
        self.assertEqual(Mod(10, 12) * 2, Mod(20, 12))
        self.assertEqual(Mod(10, 12) * Mod(2, 12), Mod(20, 12))
        self.assertRaises(ValueError, lambda: Mod(10, 12) * Mod(10, 2))
        self.assertRaises(TypeError, lambda: Mod(10, 12) * 3.1)
        self.assertRaises(TypeError, lambda: Mod(10, 12) * 'a')

    def test_imul(self):
        mod1 = Mod(10, 12)
        mod2 = Mod(2, 12)
        id_mod = id(mod1)
        mod1 *= 2
        self.assertEqual(mod1, Mod(8, 12))
        self.assertEqual(id_mod, id(mod1))
        mod1 *= mod2
        self.assertEqual(mod1, Mod(4, 12))
        self.assertRaises(ValueError, lambda: Mod(10, 12).__imul__(Mod(10, 2)))
        self.assertRaises(TypeError, lambda: Mod(10, 12).__imul__(3.1))
        self.assertRaises(TypeError, lambda: Mod(10, 12).__imul__('a'))

    def test_pow(self):
        self.assertEqual(Mod(10, 12) ** 2, Mod(100, 12))
        self.assertEqual(Mod(10, 12) ** Mod(2, 12), Mod(100, 12))
        self.assertRaises(ValueError, lambda: Mod(10, 12) ** Mod(10, 2))
        self.assertRaises(TypeError, lambda: Mod(10, 12) ** 3.1)
        self.assertRaises(TypeError, lambda: Mod(10, 12) ** 'a')

    def test_ipow(self):
        mod1 = Mod(10, 12)
        mod2 = Mod(2, 12)
        id_mod = id(mod1)
        mod1 **= 2
        self.assertEqual(mod1, Mod(4, 12))
        self.assertEqual(id_mod, id(mod1))
        mod1 **= mod2
        self.assertEqual(mod1, Mod(4, 12))
        self.assertRaises(ValueError, lambda: Mod(10, 12).__ipow__(Mod(10, 2)))
        self.assertRaises(TypeError, lambda: Mod(10, 12).__ipow__(3.1))
        self.assertRaises(TypeError, lambda: Mod(10, 12).__ipow__('a'))

    def test_ordering(self):
        mod1 = Mod(3, 12)
        mod2 = Mod(5, 12)
        mod3 = Mod(30, 12)
        mod4 = Mod(15, 12)
        self.assertRaises(TypeError, lambda: mod1 > 1.1)
        self.assertRaises(TypeError, lambda: mod1 < 1.1)
        self.assertRaises(TypeError, lambda: mod1 <= 1.1)
        self.assertRaises(TypeError, lambda: mod1 != 1.1)
        self.assertRaises(TypeError, lambda: mod1 >= 1.1)
        self.assertRaises(ValueError, lambda: mod1 >= Mod(3, 11))
        self.assertTrue(mod1 < mod2)
        self.assertTrue(mod1 <= mod2)
        self.assertTrue(mod2 > mod1)
        self.assertTrue(mod2 >= mod1)
        self.assertTrue(mod3 > mod2)
        self.assertTrue(mod3 != mod2)
        self.assertTrue(mod1 == mod4)


run_tests(ModTestCase)

test_add (__main__.ModTestCase.test_add) ... ok
test_create_mod_instance (__main__.ModTestCase.test_create_mod_instance) ... ok
test_eq (__main__.ModTestCase.test_eq) ... ok
test_hash (__main__.ModTestCase.test_hash) ... ok
test_iadd (__main__.ModTestCase.test_iadd) ... ok
test_imul (__main__.ModTestCase.test_imul) ... ok
test_int (__main__.ModTestCase.test_int) ... ok
test_ipow (__main__.ModTestCase.test_ipow) ... ok
test_isub (__main__.ModTestCase.test_isub) ... ok
test_mul (__main__.ModTestCase.test_mul) ... ok
test_neg (__main__.ModTestCase.test_neg) ... ok
test_ordering (__main__.ModTestCase.test_ordering) ... ok
test_pow (__main__.ModTestCase.test_pow) ... ok
test_sub (__main__.ModTestCase.test_sub) ... ok

----------------------------------------------------------------------
Ran 14 tests in 0.041s

OK


## Single inheritance

### Introduction

* Python supports both single and multiple inheritance.
* Technically, all classes inherit from the `object` class.

### Single inheritance

Fundamental concept in OOP

* Classes define attributes and behaviors.
* Classes can form a natural hierarchy.
* Inheritance establishes an "is a" relationship. When a derived class inherits from a base class, it means that the derived class is a specialized version of the base class.
* The base class provides a blueprint for common behavior and attributes.
* The derived class extends or modifies the functionality of the base class by adding new features or overriding existing ones.
* Benefits of inheritance
  * Code reusability: inheritance allows you to reuse existing code from the base class.
  * Hierarchical organization: you can create class hierarchies with different levels of specialization.
  * Polymorphism: derived classes can be used interchangeably with the base class.
* Drawbacks and considerations
  * Class explosion: excessive inheritance levels can lead to a complex class hierarchy.
  * Multiple inheritance: Python supports multiple inheritance, but it can be tricky to manage when multiple base classes are involved.

`type()`, `isinstance()` and `issubclass()` 

* `class type(object)`, `class type(name, bases, dict, **kwds)`
  * With one argument, return the type of an object. The return value is a type object and generally the same object as returned by `object.__class__`.
  * With three arguments, return a new type object. This is essentially a dynamic form of the `class` statement.
* `isinstance(object, classinfo)`
  * Return `True` if the object argument is an instance of the `classinfo` argument, or of a (*direct*, *indirect*, or *virtual*) subclass thereof.
  * If `object` is not an object of the given type, the function always returns `False`. If `classinfo` is a *tuple* of type objects (or recursively, other such tuples) or a `Union` Type of multiple types, return `True` if object is an instance of any of the types.
  * If `classinfo` is not a type or tuple of types and such tuples, a `TypeError` exception is raised. `TypeError` may not be raised for an invalid type if an earlier check succeeds.
  * The `isinstance()` built-in function is recommended for testing the type of an object, because it takes subclasses into account.
* `issubclass(class, classinfo)`
  * Return `True` if class is a subclass (*direct*, *indirect*, or *virtual*) of `classinfo`.
  * A class is considered a subclass of itself.
  * `classinfo` may be a *tuple* of class objects (or recursively, other such tuples) or a `Union` Type, in which case return `True` if `class` is a subclass of any entry in `classinfo`. In any other case, a `TypeError` exception is raised.

In [7]:
class Shape:
    pass

class Ellipse(Shape):
    pass

class Circle(Ellipse):
    pass

class Polygon(Shape):
    pass

class Rectangle(Polygon):
    pass

class Square(Rectangle):
    pass

class Triangle(Polygon):
    pass

print(issubclass(Triangle, Polygon))
print(issubclass(Triangle, Shape))
print(issubclass(Triangle, Ellipse))
print(issubclass(Polygon, Shape))
print(issubclass(Rectangle, Polygon))
print(issubclass(Square, Polygon))
print(issubclass(Circle, Ellipse))
print(issubclass(Ellipse, Ellipse))
print(issubclass(Ellipse, Shape))
print(issubclass(Ellipse, Polygon))

s = Shape()
t = Triangle()
p = Polygon()
c = Circle()
e = Ellipse()
print(isinstance(s, Shape))
print(isinstance(c, Shape))
print(isinstance(c, Circle))
print(isinstance(c, Ellipse))
print(isinstance(e, Ellipse))
print(isinstance(t, Polygon))
print(isinstance(p, Ellipse))
print(isinstance(t, Square))

print(type(Shape))
print(type(s))
print(s.__class__)
print(type(e))
print(issubclass(Shape, object))

True
True
False
True
True
True
True
True
True
False
True
True
True
True
True
True
False
False
<class 'type'>
<class '__main__.Shape'>
<class '__main__.Shape'>
<class '__main__.Ellipse'>
True


### The `object` class

* `class object`
  * Return a new featureless object. `object` is a base for all classes.
  * It has methods that are common to all instances of Python classes.
  * This function does not accept any arguments.
  * Note object does not have a `__dict__`, so you can't assign arbitrary attributes to an instance of the object class.
* Every class (including built-in classes) implicitly inherits from `object`, and hence automatically inherits behaviors and attributes from it.
  * `__new__`
  * `__init__`
  * `__name__`
  * `__class__`
  * `__repr__`
  * `__str__`
  * `__hash__`
  * `__eq__`
  * etc.
* It serves as the foundation for object-oriented programming in Python.

In [12]:
import math
import types

class Person:
    pass

def func():
    pass

# dir(object)
# dir(types)

print(f'{issubclass(Person, object)=}')
print(f'{type(math)=}')
print(f'{isinstance(math, object)=}')
print(f'{isinstance(math, types.ModuleType)=}')
print(f'{issubclass(type(math), object)=}')
print(f'{issubclass(type(math), types.ModuleType)=}')
print(f'{(type(math) is types.ModuleType)=}')
print(f'{type(func)=}')
print(f'{isinstance(func, object)=}')
print(f'{isinstance(func, types.FunctionType)=}')
print(f'{issubclass(type(func), types.FunctionType)=}')
print(f'{(type(func) is types.FunctionType)=}')

issubclass(Person, object)=True
type(math)=<class 'module'>
isinstance(math, object)=True
isinstance(math, types.ModuleType)=True
issubclass(type(math), object)=True
issubclass(type(math), types.ModuleType)=True
(type(math) is types.ModuleType)=True
type(func)=<class 'function'>
isinstance(func, object)=True
isinstance(func, types.FunctionType)=True
issubclass(type(func), types.FunctionType)=True
(type(func) is types.FunctionType)=True


### Overriding

* When a class inherits from a base class, it inherits all the attributes, including callables.
* Method references are resolved as follows: the corresponding class attribute is searched, descending down the chain of base classes if necessary, and the method reference is valid if this yields a function object.
* Derived classes may override / redefine methods of their base classes.
  * Because methods have no special privileges when calling other methods of the same object, a method of a base class that calls another method defined in the same base class may end up calling a method of a derived class that overrides it.
* An overriding method in a derived class may in fact want to extend rather than simply replace the base class method of the same name.
  * There is a simple way to call the base class method directly: just call `BaseClassName.methodname(self, arguments)`. Note that this only works if the base class is accessible as `BaseClassName` in the global scope.

`super`

* `class super`, `class super(type, object_or_type=None)`
  * Return a proxy object that delegates method calls to a *parent* or *sibling* class of `type`. This is useful for accessing inherited methods that have been overridden in a class.
  * The `object_or_type` determines the method resolution order to be searched. The search starts from the class right after the `type`.
    * For example, if `__mro__` of `object_or_type` is `D -> B -> C -> A -> object` and the value of `type` is `B`, then `super()` searches `C -> A -> object`.
    * The `__mro__` attribute of the `object_or_type` lists the method resolution search order used by both `getattr()` and `super()`. The attribute is dynamic and can change whenever the inheritance hierarchy is updated.
  * If the second argument is omitted, the `super` object returned is unbound. If the second argument is an object, `isinstance(obj, type)` must be true. If the second argument is a type, `issubclass(type2, type)` must be true (this is useful for classmethods).
* There are two typical use cases for `super`.
  * In a class hierarchy with *single inheritance*, `super` can be used to refer to parent classes without naming them explicitly, thus making the code more maintainable.
  * The second use case is to support cooperative *multiple inheritance* in a dynamic execution environment.
    * This use case is unique to Python and is not found in statically compiled languages or languages that only support single inheritance.
    * This makes it possible to implement "diamond diagrams" where multiple base classes implement the same method.
    * Good design dictates that such implementations have the same calling signature in every case (because the order of calls is determined at runtime, because that order adapts to changes in the class hierarchy, and because that order can include sibling classes that are unknown prior to runtime).
  * In addition to method lookups, `super()` also works for attribute lookups. One possible use case for this is calling descriptors in a parent or sibling class.
* Note that `super()` is implemented as part of the binding process for explicit dotted attribute lookups such as `super().__getitem__(name)`. It does so by implementing its own `__getattribute__()` method for searching classes in a predictable order that supports cooperative multiple inheritance. Accordingly, `super()` is undefined for implicit lookups using statements or operators such as `super()[name]`.
* Also note that, aside from the zero argument form, `super()` is not limited to use inside methods. The two argument form specifies the arguments exactly and makes the appropriate references. The zero argument form only works inside a class definition, as the compiler fills in the necessary details to correctly retrieve the class being defined, as well as accessing the current instance for ordinary methods.
* For practical suggestions on how to design cooperative classes using super(), see [guide to using `super()`](https://rhettinger.wordpress.com/2011/05/26/super-considered-super/).

In [16]:
class Person:
    def __str__(self):
        print('Person.__str__ called')
        return self.__repr__()
    
    def __repr__(self):
        print('Person.__repr__ called')
        return super().__repr__()
    
class Student(Person):
    def __repr__(self):
        print('Student.__repr__ called')
        return super().__repr__()
    
p = Person()
s = Student()
print('str(p):')
print(str(p))
print('\nrepr(p):')
print(repr(p))
print('\nstr(s):')
print(str(s))
print('\nrepr(s):')
print(repr(s))

str(p):
Person.__str__ called
Person.__repr__ called

repr(p):
Person.__repr__ called

str(s):
Person.__str__ called
Student.__repr__ called
Person.__repr__ called

repr(s):
Student.__repr__ called
Person.__repr__ called


### Extending

* A subclass can extend the functionality of its superclass by adding new methods.

In [18]:
class Account:
    APR = 3.0

    def __init__(self, account_number, balance):
        self.account_number = account_number
        self.balance = balance
    
    def calculate_interest(self):
        # return self.balance * self.APR / 100 # could be mistakenly shadowed by instance attributes instead of class attributes
        # return self.balance * Account.APR / 100 # not compatible with inheritance
        # return self.balance * type(self).APR / 100
        return self.balance * self.__class__.APR / 100

class Savings(Account):
    APR = 5.0

    def __init__(self, account_number, balance):
        super().__init__(account_number, balance)

s = Savings('12345', 100)
print(s.calculate_interest())

5.0


### Delegating to parent

* Often when overriding methods, we need to call a method specifically in the ancestry hierarchy (or a sibling class), i.e. delegate back to the parent or sibling class.
* We can explicitly call a method from the parent class by using the `super` built-in function.
* When delegating, you don't have to delegate first, although it is usually safer to do so.
  * Executing the delegate method may modify something you've already set in the instance.
* When we call a method from an instance, that method is bound to the instance; when we delegate from an instance to parent method, that method is also bound to the instance it was called from.
* Since delegated method are bound to the calling instance, any method called from the parent class will use the calling instance's version of the method.

In [9]:
from numbers import Real
import math

class Circle:
    def __init__(self, radius):
        self.radius = radius
        self._perimeter = None
        self._area = None

    @property
    def radius(self):
        return self._radius

    @radius.setter
    def radius(self, radius):
        if isinstance(radius, Real) and radius > 0:
            self._radius = radius
            self._perimeter = None
            self._area = None
        else:
            raise ValueError('radius must be a positive real number')
        
    @property
    def perimeter(self):
        if self._perimeter is None:
            self._perimeter =  2 * math.pi * self._radius
        return self._perimeter
    
    @property
    def area(self):
        if self._area is None:
            self._area = math.pi * self._radius ** 2
        return self._area
    

class UnitCircle(Circle):
    def __init__(self): # can't use super().__init__ here, because radius property has been changed to read-only
        self._radius = 1
        self._perimeter = None
        self._area = None

    @property
    def radius(self):
        return super().radius


uc = UnitCircle()
print(uc.radius)
# uc.radius = 2 # unit circle's radius is read-only
print(uc.perimeter)
print(uc.area)

1
6.283185307179586
3.141592653589793


### Slots

* By default, Python objects use a dynamic dictionary (`__dict__`) to store their attributes. However, this dictionary consumes memory, especially when you have many instances of the same class.
* `__slots__` allow us to explicitly declare *data members* (like properties) and deny the creation of `__dict__` and `__weakref__` in *instances* (unless explicitly declared in `__slots__` or available in a parent.)
* The space saved over using `__dict__` can be significant. Attribute lookup speed can be significantly improved as well.
  * `__slots__` helps reduce memory usage by preallocating memory for specific attributes.
* `object.__slots__`
  * This class variable can be assigned a string, iterable, or sequence of strings with variable names used by instances.
  * `__slots__` reserves space for the declared variables and prevents the automatic creation of `__dict__` and `__weakref__` for each *instance*.
    * Try to get `__dict__` attribute on an instance without it will result in `AttributeError`.
    * Similarly, calling `vars` on such an instance will cause `TypeError`.
    * `dir` still works.
* Notes on using `__slots__`
  * When inheriting from a class without `__slots__`, the `__dict__` and `__weakref__` attribute of the instances will always be accessible.
  * Without a `__dict__` variable, instances cannot be assigned new variables not listed in the `__slots__` definition. Attempts to assign to an unlisted variable name raises `AttributeError`. If dynamic assignment of new variables is desired, then add `'__dict__'` to the sequence of strings in the `__slots__` declaration (this will result in an empty dictionary for `__dict__`)
  * Without a `__weakref__` variable for each instance, classes defining `__slots__` do not support weak references to its instances. If weak reference support is needed, then add `'__weakref__'` to the sequence of strings in the `__slots__` declaration.
  * `__slots__` are implemented at the class level by creating *descriptors* for each variable name. As a result, class attributes cannot be used to set default values for instance variables defined by `__slots__`; otherwise, the class attribute would overwrite the descriptor assignment.
  * The action of a `__slots__` declaration is not limited to the class where it is defined. `__slots__` declared in parents are available in child classes. However, child subclasses will get a `__dict__` and `__weakref__` unless they also define `__slots__` (which should only contain names of any additional slots).
  * If a class defines a slot also defined in a base class, the instance variable defined by the base class slot is inaccessible (except by retrieving its descriptor directly from the base class). This renders the meaning of the program undefined. In the future, a check may be added to prevent this.
  * `TypeError` will be raised if nonempty `__slots__` are defined for a class derived from a "variable-length" built-in type such as `int`, `bytes`, and `tuple`.
  * Any non-string iterable may be assigned to `__slots__`.
  * If a dictionary is used to assign `__slots__`, the dictionary keys will be used as the slot names. The values of the dictionary can be used to provide per-attribute docstrings that will be recognized by `inspect.getdoc()` and displayed in the output of `help()`.
  * `__class__` assignment works only if both classes have the same `__slots__`.
  * Multiple inheritance with multiple slotted parent classes can be used, but only one parent is allowed to have attributes created by slots (the other bases must have empty slot layouts), violations raise `TypeError`.
  * If an iterator is used for `__slots__` then a descriptor is created for each of the iterator's values. However, the `__slots__` attribute will be an empty iterator.

In [14]:
class Location:
    # __slots__ = 'name', '_longitude', '_latitude'
    __slots__ = 'name', '_longitude', '_latitude', '__dict__'

    def __init__(self, name, longitude, latitude):
        self.name = name
        self._longitude = longitude
        self._latitude = latitude

    @property
    def longitude(self):
        return self._longitude
    
    @property
    def latitude(self):
        return self._latitude
    
print(Location.__dict__) # available
l = Location('Sydney', 151.20107055605533, -33.86491159041564)
# print(l.__dict__) # AttributeError, if '__dict__' not contained in __slots__
# print(vars(l)) # TypeError, if '__dict__' not contained in __slots__
print(l.longitude)
print(l.__dict__) # {}, if '__dict__' is contained in __slots__

{'__module__': '__main__', '__slots__': ('name', '_longitude', '_latitude', '__dict__'), '__init__': <function Location.__init__ at 0x0000020A330DBB00>, 'longitude': <property object at 0x0000020A3314FE20>, 'latitude': <property object at 0x0000020A3314EED0>, '_latitude': <member '_latitude' of 'Location' objects>, '_longitude': <member '_longitude' of 'Location' objects>, 'name': <member 'name' of 'Location' objects>, '__dict__': <attribute '__dict__' of 'Location' objects>, '__doc__': None}
151.20107055605533
{}


### Slots and single inheritance

* Subclasses wil use slots from the parents (if present), and will also use an instance dictionary.
* If we want subclasses to also just use slots, simply specify `__slots__` in them, but only specify the additional attributes, otherwise the same attributes defined in the parents will be shadowed.
* When inheriting from a class without `__slots__`, `__dict__` attribute of the instances will always be accessible, no matter the subclass itself having `__slots__` or not.
* `'__dict__'` can be added to `__slots__` so that adding attributes to instances at run-time is possible.

In [18]:
class Person:
    __slots__ = 'name'

class Student(Person):
    pass

# print(Person().__dict__) # AttributeError
print(Student().__dict__) # {}

{}


In [21]:
class Person:
    __slots__ = 'name'

    def __init__(self, name):
        self.name = name

class Student(Person):
    __slots__ = 'age'

    def __init__(self, name, age):
        super().__init__(name)
        self.age = age

s = Student('John', 'Doe')
print(s.name, s.age)
# print(s.__dict__) # AttributeError


John Doe


In [40]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age

class Student(Person):
    __slots__ = 'major'

    def __init__(self, name, age, gender, major):
        super().__init__(name, age)
        self.gender = gender
        self.major = major

s = Student('James', 18, 'male', 'computer science')
print(dir(s)) # __dict__ inside s
print(s.__dict__) # major not included
s.grade = 'A' # add grade attribute to __dict__
print(s.__dict__)

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', 'age', 'gender', 'major', 'name']
{'name': 'James', 'age': 18, 'gender': 'male'}
{'name': 'James', 'age': 18, 'gender': 'male', 'grade': 'A'}


## Project 3

Basic information

* You are writing an inventory application for a budding tech guy who has a video channel featuring computer builds. Basically they have a pool of inventory for builds. When they take a CPU from the pool, they will indicate this using the object that tracks that specific type of CPU. They may also purchase additional CPUs, or retire some.
* Technically we would want a database to back all this data, but here we're just going to build the classes we'll use while our program is running and not worry about retrieving or saving the state of the inventory.
* The base class is going to be a general `Resource`. This class should provide functionality common to all the actual resources (CPU, GPU, Memory, HDD, SSD) - for this exercise we're only going to implement CPU, HDD and SSD.
* It should provide this at a minimum:
  * `name`: user-friendly name of resource instance (e.g.`'Intel Core i9-9900K'`)
  * `manufacturer`: resource instance manufacturer (e.g. `'Nvidia'`)
  * `total`: inventory total
  * `allocated`: number allocated
  * a `__str__` representation that just returns the resource name
  * a more detailed `__repr__` implementation
  * `claim(n)`: method to take `n` resources from the pool (as long as inventory is available)
  * `freeup(n)`: method to return `n` resources to the pool (e.g. disassembled some builds)
  * `died(n)`: method to return and permanently remove inventory from the pool (e.g. they broke something) - as long as total available allows it
  * `purchased(n)`: method to add inventory to the pool
  * `category`: computed property that returns a lower case version of the class name
* Next we are going to define child classes for each of CPU, HDD and SDD.
  * For the `CPU` class:
    * `cores` (e.g. `8`)
    * `socket` (e.g. `'AM4'`)
    * `power_watts` (e.g. `94`)
  * For the HDD and SDD classes, we're going to create an intermediate class called `Storage` with these additional properties:
    * `capacity_GB` (e.g. `120`)
  * The `HDD` class extends `Storage` and has these additional properties:
    * `size` (e.g. `'2.5"'`)
    * `rpm` (e.g. `7000`)
  * The `SSD` class extends `Storage` and has these additional properties:
    * `interface` (e.g. `'PCIe NVMe 3.0 x4'`)
* For all your classes, implement a full constructor that can be used to initialize all the properties, some form of validation on numeric types, as well as customized `__repr__` as you see fit.
* For the `total` and `allocated` values in the `Resource` init, think of the arguments there as the *current* total and allocated counts. Those `total` and `allocated` attributes should be private *read-only* properties, but they are modifiable through the various methods such as `claim`, `return`, `died` and `purchased`. Other attributes like `name`, `manufacturer_name`, etc should be read-only.

In [1]:
class Resource:
    __slots__ = '_name', '_manufacturer', '_inventory', '_allocated', '_variables'

    def __init__(self, name, manufacturer, inventory, allocated):
        self._set_str_attribute('_name', name)
        self._set_str_attribute('_manufacturer', manufacturer)
        self._set_int_attribute('_inventory', inventory)
        self._set_int_attribute('_allocated', allocated)
        self._variables = list(locals())[1:]

    def _set_str_attribute(self, attribute_name, value):
        if isinstance(value, str):
            setattr(self, attribute_name, value)
        else:
            raise TypeError(f'{attribute_name.removeprefix('_')} should be a str')

    def _set_int_attribute(self, attribute_name, value, min=0, max=None):
        if not isinstance(value, int):
            raise TypeError(f'{attribute_name.removeprefix('_')} should be an int')
        
        if min is not None and value < min:
            raise ValueError(f'{attribute_name.removeprefix('_')} should be an int no less than {min}')

        if max is not None and value > max:
            raise ValueError(f'{attribute_name.removeprefix('_')} should be an int no greater than {max}')

        setattr(self, attribute_name, value)
        
    @property
    def name(self):
        return self._name
    
    @property
    def manufacturer(self):
        return self._manufacturer
    
    @property
    def inventory(self):
        return self._inventory
    
    @property
    def allocated(self):
        return self._allocated
        
    def __str__(self):
        return self._name
    
    def __repr__(self):
        attributes = map(lambda v: f'{v}={getattr(self, v)}', self._variables)
        return f'{self.__class__.__name__}({', '.join(attributes)})'
    
    def claim(self, n):
        if isinstance(n, int) and 0 < n <= self._inventory:
            self._inventory -= n
            self._allocated += n
        else:
            raise ValueError('n should be a positive integer no greater than inventory')
    
    def free_up(self, n):
        if self._allocated == 0:
            raise RuntimeError('0 allocated, nothing to free up')
        if isinstance(n, int) and 0 < n <= self._allocated:
            self._allocated -= n
            self._inventory += n
        else:
            raise ValueError('n should be a positive integer no greater than allocated')
        
    def remove_died(self, n):
        if self._inventory == 0:
            raise RuntimeError('0 inventory, nothing to remove')
        if isinstance(n, int) and 0 < n <= self._inventory:
            self._inventory -= n
        else:
            raise ValueError('n should be a positive integer no greater than inventory')

    def add_purchased(self, n):
        if isinstance(n, int) and n > 0:
            self._inventory += n
        else:
            raise ValueError('n should be a positive integer')
        
    def category(self):
        return self.__class__.__name__.lower()
    

class CPU(Resource):
    __slots__ = '_cores', '_socket', '_power_watts'

    def __init__(self, name, manufacturer, inventory, allocated, cores, socket, power_watts):
        super().__init__(name, manufacturer, inventory, allocated)
        self.cores = cores
        self.socket = socket
        self.power_watts = power_watts

    @property
    def cores(self):
        return self._cores
    
    @cores.setter
    def cores(self, cores):
        self._set_int_attribute('_cores', cores, 1)

    @property
    def socket(self):
        return self._socket
    
    @socket.setter
    def socket(self, socket):
        self._set_str_attribute('_socket', socket)

    @property
    def power_watts(self):
        return self._power_watts
    
    @power_watts.setter
    def power_watts(self, power_watts):
        self._set_int_attribute('_power_watts', power_watts, 1)


class Storage(Resource):
    __slots__ = '_capacity_GB'

    def __init__(self, name, manufacturer, inventory, allocated, capacity_GB):
        super().__init__(name, manufacturer, inventory, allocated)
        self.capacity_GB = capacity_GB

    @property
    def capacity_GB(self):
        return self._capacity_GB
    
    @capacity_GB.setter
    def capacity_GB(self, capacity_GB):
        self._set_int_attribute('_capacity_GB', capacity_GB, 1)


class HDD(Storage):
    __slots__ = '_size', '_rpm'

    def __init__(self, name, manufacturer, inventory, allocated, capacity_GB, size, rpm):
        super().__init__(name, manufacturer, inventory, allocated, capacity_GB)
        self.size = size
        self.rpm = rpm

    @property
    def size(self):
        return self._size
    
    @size.setter
    def size(self, size):
        self._set_str_attribute('_size', size)

    @property
    def rpm(self):
        return self._rpm
    
    @rpm.setter
    def rpm(self, rpm):
        self._set_int_attribute('_rpm', rpm, 1)


class SSD(Storage):
    __slots__ = '_interface'
    
    def __init__(self, name, manufacturer, inventory, allocated, capacity_GB, interface):
        super().__init__(name, manufacturer, inventory, allocated, capacity_GB)
        self.interface = interface

    @property
    def interface(self):
        return self._interface
    
    @interface.setter
    def interface(self, interface):
        self._set_str_attribute('_interface', interface)

In [3]:
import unittest

def run_tests(test_case_class):
    suite = unittest.TestLoader().loadTestsFromTestCase(test_case_class)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(suite)

class ResourceTestCase(unittest.TestCase):
    def setUp(self):
        self.name = 'resource'
        self.manufacturer = 'maker'
        self.inventory = 100
        self.allocated = 0
        self.r = Resource(self.name, self.manufacturer, self.inventory, self.allocated)

    def test_instance_creation(self):
        self.assertRaises(TypeError, Resource, 1, 1, 1, 1)
        self.assertRaises(TypeError, Resource, 'resource', 1, 1, 1)
        self.assertRaises(TypeError, Resource, 'resource', 'maker', '1', 1)
        self.assertRaises(TypeError, Resource, 'resource', 'maker', 10, '1')
        self.assertRaises(TypeError, Resource, 'resource', 'maker', 10, 1.0)
        self.assertRaises(ValueError, Resource, 'resource', 'maker', -10, 1)
        self.assertRaises(ValueError, Resource, 'resource', 'maker', 10, -1)

    def test_instance_attributes(self):
        self.assertEqual(self.r.name, self.name)
        self.assertEqual(self.r.manufacturer, self.manufacturer)
        self.assertEqual(self.r.inventory, self.inventory)
        self.assertEqual(self.r.allocated, self.allocated)
        self.assertRaises(AttributeError, setattr, self.r, 'new_attribute', 1)
        self.assertRaises(AttributeError, setattr, self.r, 'name', 'new_name')
        self.assertRaises(AttributeError, setattr, self.r, 'manufacturer', 'new_maker')
        self.assertRaises(AttributeError, setattr, self.r, 'inventory', 20)
        self.assertRaises(AttributeError, setattr, self.r, 'allocated', 10)

    def test_claim(self):
        self.assertRaises(ValueError, self.r.claim, -1)
        self.assertRaises(ValueError, self.r.claim, 0)
        self.assertRaises(ValueError, self.r.claim, self.r.inventory + 1)
        self.r.claim(self.r.inventory)
        self.assertEqual(self.r.inventory, 0)
        self.assertEqual(self.r.allocated, self.allocated + self.inventory)

    def test_free_up(self):
        self.assertRaises(RuntimeError, self.r.free_up, 1) # allocated 0
        self.r.claim(5)
        self.assertRaises(ValueError, self.r.free_up, -1)
        self.assertRaises(ValueError, self.r.free_up, 0)
        self.assertRaises(ValueError, self.r.free_up, self.r.allocated + 1)
        self.r.free_up(self.r.allocated)
        self.assertEqual(self.r.allocated, 0)
        self.assertEqual(self.r.inventory, self.inventory)

    def test_remove_died(self):
        self.assertRaises(ValueError, self.r.remove_died, -1)
        self.assertRaises(ValueError, self.r.remove_died, 0)
        self.assertRaises(ValueError, self.r.remove_died, self.r.inventory + 1)
        self.r.remove_died(self.r.inventory)
        self.assertAlmostEqual(self.r.inventory, 0)
        self.assertAlmostEqual(self.r.allocated, self.allocated)
        self.assertRaises(RuntimeError, self.r.remove_died, 1)

    def test_add_purchased(self):
        self.assertRaises(ValueError, self.r.add_purchased, 0)
        self.assertRaises(ValueError, self.r.add_purchased, -2)
        self.assertRaises(ValueError, self.r.add_purchased, 2.0)
        self.r.add_purchased(10)
        self.assertEqual(self.r.inventory, self.inventory + 10)

    def test_category(self):
        self.assertEqual(self.r.category(), 'resource')

class CPUTestCase(unittest.TestCase):
    def test_all(self):
        cpu = CPU('AMD Ryzen', 'AMD', 100, 10, 24, 'A10', 28)
        self.assertEqual(cpu.cores, 24)
        self.assertEqual(cpu.socket, 'A10')
        self.assertEqual(cpu.power_watts, 28)

class StorageTestCase(unittest.TestCase):
    def test_all(self):
        storage = Storage('WD 100T', 'WD', 20, 100, 100000)
        self.assertEqual(storage.capacity_GB, 100000)

class HDDTestCase(unittest.TestCase):
    def test_all(self):
        hdd = HDD('WD 100T', 'WD', 100, 20, 100000, '3.5"', 10000)
        self.assertEqual(hdd.size, '3.5"', 10000)

class SSDTestCase(unittest.TestCase):
    def test_all(self):
        ssd = SSD('SG 20T', 'SG', 200, 1000, 20000, 'NVMe')
        self.assertEqual(ssd.interface, 'NVMe')

run_tests(ResourceTestCase)
run_tests(CPUTestCase)
run_tests(StorageTestCase)
run_tests(HDDTestCase)
run_tests(SSDTestCase)

test_add_purchased (__main__.ResourceTestCase.test_add_purchased) ... ok
test_category (__main__.ResourceTestCase.test_category) ... ok
test_claim (__main__.ResourceTestCase.test_claim) ... ok
test_free_up (__main__.ResourceTestCase.test_free_up) ... ok
test_instance_attributes (__main__.ResourceTestCase.test_instance_attributes) ... ok
test_instance_creation (__main__.ResourceTestCase.test_instance_creation) ... ok
test_remove_died (__main__.ResourceTestCase.test_remove_died) ... ok

----------------------------------------------------------------------
Ran 7 tests in 0.020s

OK
test_all (__main__.CPUTestCase.test_all) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.002s

OK
test_all (__main__.StorageTestCase.test_all) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.003s

OK
test_all (__main__.HDDTestCase.test_all) ... ok

----------------------------------------------------------------------

## Descriptors

### Descriptors



* Reference
  * [Implementing Descriptors](https://docs.python.org/3/howto/descriptor.html#descriptor-guide)
  * [The Descriptor How To Guide](https://docs.python.org/3/reference/datamodel.html#descriptors)
* In general, a descriptor is an object attribute that defines any of the methods in the descriptor protocol: `__get__()`, `__set__()`, and `__delete__()`.
  * Optionally, descriptors can have a `__set_name__()` method (new in version 3.6). This is only used in cases where a descriptor needs to know either the class where it was created or the name of class variable it was assigned to. (This method, if present, is called even if the class is not a descriptor.)
* When an object attribute is a descriptor, it has special binding behavior, i.e. its attribute access has been overridden by methods in the descriptor protocol.
  * The default behavior for attribute access is to get, set, or delete the attribute from an object's dictionary. For instance, `a.x` has a lookup chain starting with `a.__dict__['x']`, then `type(a).__dict__['x']`, and continuing through the base classes of `type(a)` excluding metaclasses.
  * However, if the looked-up value is an object defining one of the descriptor methods, then Python may override the default behavior and invoke the descriptor method instead. Where this occurs in the precedence chain depends on which descriptor methods were defined and how they were called.
* Descriptors only work when used as *class variables*. When put in instances, they have no effect.
* The main motivation for descriptors is to provide a hook allowing objects stored in class variables to control what happens during attribute lookup.
  * Traditionally, the calling class controls what happens during lookup. Descriptors invert that relationship and allow the data being looked-up to have a say in the matter.
* Understanding descriptors is a key to a deep understanding of Python because they are the basis for many features including functions, methods, properties, class methods, static methods, and reference to super classes.
  * Descriptors are used throughout the language. It is how functions turn into bound methods.
  * Common tools like `classmethod()`, `staticmethod()`, `property()`, and `functools.cached_property()` are all implemented as descriptors.
* Two categories of descriptors:
  * Implement `__get__` only: non-data descriptors
  * Implement `__set__` and / or `__delete__`: data descriptors

In [1]:
from datetime import datetime, timezone

class CurrentDatetimeUTC:
    def __get__(self, instance, owner=None):
        return datetime.now(timezone.utc).isoformat()
    
class Logger:
    current_datetime_utc = CurrentDatetimeUTC()

print(Logger.current_datetime_utc)
print(Logger().current_datetime_utc)

2024-02-24T21:07:29.214140+00:00
2024-02-24T21:07:29.214140+00:00


In [7]:
from random import choice

class Choice:
    def __init__(self, *choices):
        self.choices = choices

    def __get__(self, instance, owner=None):
        return choice(self.choices)
    
class Deck:
    suit = Choice('Spade', 'Heart', 'Diamond', 'Club')
    rank = Choice(*'23456789JQKA', '10')

class Dice:
    die_1 = Choice(1, 2, 3, 4, 5, 6)
    die_2 = Choice(1, 2, 3, 4, 5, 6)
    die_3 = Choice(1, 2, 3, 4, 5, 6)

deck = Deck()
for _ in range(10):
    print(deck.rank, deck.suit)

dice = Dice()
for _ in range(6):
    print(dice.die_1, dice.die_2, dice.die_3)

7 Club
4 Heart
K Heart
7 Heart
10 Diamond
8 Club
A Heart
7 Club
7 Diamond
K Spade
3 5 5
1 3 2
4 4 1
2 3 6
1 3 5
2 4 6


### Getters and setters

* `__get__(self, instance, owner=None)`
  * Called to get the attribute of the *owner class* (**class attribute access**) or of an *instance* of that class (**instance attribute access**).
  * The optional `owner` argument is the owner class, while `instance` is the instance that the attribute was accessed through, or `None` when the attribute is accessed through the `owner`.
  * This method should return the computed attribute value or raise an `AttributeError` exception.
  * PEP 252 specifies that `__get__()` is callable with one or two arguments. Python's own built-in descriptors support this specification; however, it is likely that some third-party tools have descriptors that require both arguments. Python's own `__getattribute__()` implementation always passes in both arguments whether they are required or not.
* `__set__(self, instance, value)`
  * Called to set the attribute on an instance `instance` of the owner class to a new value, `value`.
  * Note, adding `__set__()` or `__delete__()` changes the kind of descriptor to a *data descriptor*.
* `__delete__(self, instance)`
  * Called to delete the *attribute* on an instance `instance` of the owner class.

In [8]:
from datetime import datetime, timezone

class CurrentDatetimeUTC:
    def __get__(self, instance, owner=None):
        if instance is None:
            return self, id(self)
        return datetime.now(timezone.utc).isoformat(), id(self)
    
class Logger:
    current_dt = CurrentDatetimeUTC()

logger_1 = Logger()
logger_2 = Logger()
print(Logger.current_dt)
print(logger_1.current_dt)
print(logger_2.current_dt)

(<__main__.CurrentDatetimeUTC object at 0x000002D978584CB0>, 3133050211504)
('2024-02-24T22:54:47.353466+00:00', 3133050211504)
('2024-02-24T22:54:47.353466+00:00', 3133050211504)


### Using data descriptors as instance properties

Where to store the instance attribute values

* Could be stored in the instance dictionary
  * instance dictionaries are not guaranteed because of `__slots__`
  * might overwrite an existing attribute
* Could be stored in a dictionary local to the data descriptor instance
  * key equals to the instance of the owning class, if it is hashable
  * value equals to the attribute value

In [14]:
import ctypes

def reference_count(address):
    return ctypes.c_long.from_address(id(address)).value

class IntegerValue:
    def __init__(self):
        self.data = {}

    def __get__(self, instance, owner=None):
        if instance is None:
            return self
        return self.data.get(instance) # get None if not found
    
    def __set__(self, instance, value):
        # `del instance` will not work because it is still being referenced
        # inside self.data, this can cause potential memory leak
        self.data[instance] = int(value)

class Point2D:
    x = IntegerValue()
    y = IntegerValue()

    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __repr__(self):
        return f'Point2D(x={self.x}, y={self.y})'

    def __eq__(self, other):
        if isinstance(other, Point2D):
            return self.x == other.x and self.y == other.y
        else:
            return NotImplemented
        
    def __hash__(self): # must be implemented if implemented __eq__ to keep instances hashable
        return hash((self.x, self.y))

p1 = Point2D(1.5, 1.5)
p2 = Point2D(2.0, 2.0)
print(p1, p2)
print(Point2D.x.data)
print(Point2D.y.data)
address_p1 = id(p1)
print(f'{reference_count(address_p1)=}')
del p1
print(f'{reference_count(address_p1)=}')

Point2D(x=1, y=1) Point2D(x=2, y=2)
{Point2D(x=1, y=1): 1, Point2D(x=2, y=2): 2}
{Point2D(x=1, y=1): 1, Point2D(x=2, y=2): 2}
reference_count(address_p1)=2
reference_count(address_p1)=2


### Strong and weak references

Strong references

* A strong reference is the most common type of reference in Python.
* When an object has one or more strong references, it remains alive in memory.
* Objects referenced by variables, data structures (like lists or dictionaries), and function arguments are considered strongly referenced.
* The reference count of an object increases when a strong reference is created.
* Garbage collection does not reclaim an object as long as there are strong references to it.
* In Python's C API, a strong reference is a reference to an object which is owned by the code holding the reference.
  * The strong reference is taken by calling `Py_INCREF()` when the reference is created and released with `Py_DECREF()` when the reference is deleted.
  * The `Py_NewRef()` function can be used to create a strong reference to an object.
  * Usually, the `Py_DECREF()` function must be called on the strong reference before exiting the scope of the strong reference, to avoid leaking one reference.
  See also borrowed reference.

Weak references

* A weak reference to an object is a reference that does not prevent an object from being garbage collected.
* When the only remaining references to a referent are weak references, garbage collection is free to destroy the referent and reuse its memory for something else.
* However, until the object is actually destroyed the weak reference may return the object even if there are no strong references to it.
* Use cases for weak references
  * A primary use for weak references is to implement *caches* or *mappings* holding large objects, where it's desired that a large object not be kept alive solely because it appears in a cache or mapping.
  * Reducing the impact of circular references (when objects reference each other in a loop).

Borrowed references

* A borrowed reference is a term used in the context of CPython's memory management.
* In Python's C API, a borrowed reference is a reference to an object, where the code using the object does not own the reference.
* A borrowed reference becomes a dangling pointer if the object is destroyed.
  * For example, a garbage collection can remove the last strong reference to the object and so destroy it.
* When a function passes ownership of a reference to its caller, the caller receives a new reference (i.e. a borrowed reference).
* No ownership transfer occurs for borrowed references; they do not affect the reference count.
* Borrowed references are common when passing objects between functions or methods.

`weakref` module

* The `weakref` module allows the Python programmer to create weak references to objects.
* The `WeakKeyDictionary` and `WeakValueDictionary` classes supplied by the `weakref` module use weak references to construct mappings that don't keep objects alive solely because they appear in the mapping objects.
* `WeakKeyDictionary` and `WeakValueDictionary` use weak references in their implementation, setting up callback functions on the weak references that notify the weak dictionaries when a key or value has been reclaimed by garbage collection.
* `WeakSet` implements the set interface, but keeps weak references to its elements, just like a `WeakKeyDictionary` does.
* `finalize` provides a straight forward way to register a cleanup function to be called when an object is garbage collected. This is simpler to use than setting up a callback function on a raw weak reference, since the module automatically ensures that the finalizer remains alive until the object is collected.
* Most programs should find that using one of these weak container types or `finalize` is all they need. It's usually unnecessary to create your own weak references directly. The low-level machinery is exposed by the `weakref` module for the benefit of advanced uses.
* Not all objects can be weakly referenced.
  * Objects which support weak references include class instances, functions written in Python (but not in C), instance methods, sets, frozensets, some file objects, generators, type objects, sockets, arrays, deques, regular expression pattern objects, and code objects.
  * Several built-in types such as `list` and `dict` do not directly support weak references but can add support through subclassing.
  * Other built-in types such as `tuple` and `int` do not support weak references even when subclassed.
  * When `__slots__` are defined for a given type, weak reference support is disabled unless a `'__weakref__'` string is also present in the sequence of strings in the `__slots__` declaration.

In [12]:
import ctypes
import weakref

def reference_count(address):
    return ctypes.c_long.from_address(address).value

class Person:
    def __init__(self, name):
        self.name = name

    def __repr__(self):
        return f'Person(name={self.name})'
    
p1 = Person('Guido')
address_p1 = id(p1)
print(f'{reference_count(address_p1)=}')
p2 = p1
print(f'{reference_count(address_p1)=}')
p3 = weakref.ref(p1) # reference count of p1 remains unchanged
print(f'{reference_count(address_p1)=}')
print(f'{weakref.getweakrefcount(p1)=}')
print(f'{p1.__weakref__=}')
print(f'{(p3() is p1)=}')
print(p3)
print(p3())

del p1
del p2
print(p3) # dead weakref
print(p3()) # None, the object it reference to has been garbage collected

reference_count(address_p1)=1
reference_count(address_p1)=2
reference_count(address_p1)=2
weakref.getweakrefcount(p1)=1
p1.__weakref__=<weakref at 0x000001FC59E1B880; to 'Person' at 0x000001FC59972B70>
(p3() is p1)=True
<weakref at 0x000001FC59E1B880; to 'Person' at 0x000001FC59972B70>
Person(name=Guido)
<weakref at 0x000001FC59E1B880; dead>
None


### `weakref` object

* `class weakref.ref(object[, callback])`
  * Return a weak reference to `object`.
  * The original object can be retrieved by *calling* the reference object if the referent is still alive; if the referent is no longer alive, calling the reference object will cause `None` to be returned.
  * If `callback` is provided and not `None`, and the returned `weakref` object is still alive, the callback will be called when the object is about to be finalized; the weak reference object will be passed as the only parameter to the callback; the referent will no longer be available.
  * It is allowable for many weak references to be constructed for the same object. Callbacks registered for each weak reference will be called from the most recently registered callback to the oldest registered callback.
  * Exceptions raised by the callback will be noted on the standard error output, but cannot be propagated; they are handled in exactly the same way as exceptions raised from an object's `__del__()` method.
  * Weak references are hashable if the object is hashable. They will maintain their hash value even after the object was deleted. If `hash()` is called the first time only after the object was deleted, the call will raise `TypeError`.
  * Weak references support tests for equality, but not ordering. If the referents are still alive, two references have the same equality relationship as their referents (regardless of the callback). If either referent has been deleted, the references are equal only if the reference objects are the same object.
  * This is a subclassable type rather than a factory function.
  * `__callback__`
    * This read-only attribute returns the callback currently associated to the `weakref`. If there is no callback or if the referent of the `weakref` is no longer alive then this attribute will have value `None`.

In [30]:
import weakref

class IntegerValue:
    def __init__(self):
        self.data = weakref.WeakKeyDictionary()

    def __get__(self, instance, owner=None):
        if instance is None:
            return self
        return self.data.get(instance)
    
    def __set__(self, instance, value):
        self.data[instance] = int(value)

class Point2D:
    x = IntegerValue()
    y = IntegerValue()

    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __repr__(self):
        return f'Point2D(x={self.x}, y={self.y})'
    
    def __eq__(self, other):
        if isinstance(other, Point2D):
            return self.x == other.x and self.y == other.y
        else:
            return NotImplemented
        
    def __hash__(self):
        return hash((self.x, self.y))
    
# p = Point2D(1.5, 2.0) # RecursionError when hashing the instance

In [25]:
import weakref

class IntegerValue:
    def __init__(self):
        self.data = {}

    def __get__(self, instance, owner=None):
        if instance is None:
            return self
        return self.data.get(id(instance))[1]
    
    def __set__(self, instance, value):
        self.data[id(instance)] = weakref.ref(instance, self._clear_data), int(value) # using id as the key of data dictionary could cause potential collision

    def _clear_data(self, weak_ref):
        for k, v in self.data.items():
            if weak_ref is v[0]:
                del self.data[k]
                break

class Point2D:
    x = IntegerValue()
    y = IntegerValue()

    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __repr__(self):
        return f'Point2D(x={self.x}, y={self.y})'
    
    def __eq__(self, other):
        # if isinstance(other, Point2D):
        #     return self.x == other.x and self.y == other.y
        # else:
        #     return NotImplemented
        return isinstance(other, Point2D) and self.x == other.x and self.y == other.y
        
    def __hash__(self):
        return hash((self.x, self.y))
    
p1 = Point2D(1.5, 2.0)
p2 = Point2D(3, 6)
print(Point2D.x.data)
print(Point2D.y.data)
del p1
print(Point2D.x.data)
del p2
print(Point2D.x.data)
print(Point2D.y.data)

{2183346635696: (<weakref at 0x000001FC5A0957B0; to 'Point2D' at 0x000001FC5999C3B0>, 1), 2183346649856: (<weakref at 0x000001FC5A094AE0; to 'Point2D' at 0x000001FC5999FB00>, 3)}
{2183346635696: (<weakref at 0x000001FC5A095A30; to 'Point2D' at 0x000001FC5999C3B0>, 2), 2183346649856: (<weakref at 0x000001FC5A094B30; to 'Point2D' at 0x000001FC5999FB00>, 6)}
{2183346649856: (<weakref at 0x000001FC5A094AE0; to 'Point2D' at 0x000001FC5999FB00>, 3)}
{}
{}


### `__set_name__` method

* `__set_name__(self, owner, name)`
  * Automatically called at the time the owning class `owner` is created. The descriptor object is assigned to `name` in that class.
  * If the class variable is assigned after the class is created, `__set_name__()` will not be called automatically. If needed, `__set_name__()` can be called directly.
  * New in version 3.6.
* This is helpful for descriptors that need to know their own name as defined in the owner class.
* By knowing the attribute name, a descriptor can tailor its behavior.
  * Descriptors can create additional attributes dynamically based on the attribute name.

In [36]:
class ValidString:
    def __init__(self, min_length=None, max_length=None):
        self.min_length = min_length
        self.max_length = max_length

    def __set_name__(self, owner, name):
        self.name = name

    def __set__(self, instance, value):
        if not isinstance(value, str):
            raise TypeError(f'{self.name} must be a str')
        value = value.strip()
        if self.min_length is not None and len(value) < self.min_length:
            raise ValueError(f'the length of {self.name} must be no less than {self.min_length}')
        if self.max_length is not None and len(value) > self.max_length:
            raise ValueError(f'the length of {self.name} must be no greater than {self.max_length}')
        # setattr(instance, self.name, value.capitalize()) # RecursionError, need to change to a different attribute name to use setattr
        instance.__dict__[self.name] = value.capitalize() # instance __dict__ can be modified manually
            
    def __get__(self, instance, owner=None):
        if instance is None:
            return self
        # return getattr(instance, self.name, None) # RecursionError
        return instance.__dict__.get(self.name)

class Person:
    firstname = ValidString(1, 128)
    lastname = ValidString(1, 128)

    def __init__(self, firstname, lastname):
        self.firstname = firstname
        self.lastname = lastname

p = Person('john ', ' smith')
print(repr(p.firstname))
print(repr(p.lastname))
print(p.__dict__)
p.firstname = ' invalid name ' # still works, same name attributes do not shadowing class attributes
print(repr(p.firstname))

'John'
'Smith'
{'firstname': 'John', 'lastname': 'Smith'}
'Invalid name'


### Property lookup resolution

Invocation from an instance

* Instance lookup scans through a chain of namespaces giving *data descriptors* the highest priority, followed by *instance variables*, then *non-data descriptors*, then *class variables*, and lastly `__getattr__()` if it is provided.
* If a descriptor is found for `a.x`, then it is invoked with: `desc.__get__(a, type(a))`.
* The logic for a dotted lookup is in `object.__getattribute__()`.

Invocation from a class

* The logic for a dotted lookup such as `A.x` is in `type.__getattribute__()`. The steps are similar to those for `object.__getattribute__()` but the instance dictionary lookup is replaced by a search through the class's method resolution order.
* If a descriptor is found, it is invoked with `desc.__get__(None, A)`.

Invocation from super

* The logic for super's dotted lookup is in the `__getattribute__()` method for object returned by `super()`.
* A dotted lookup such as `super(A, obj).m` searches `obj.__class__.__mro__` for the base class `B` immediately following `A` and then returns `B.__dict__['m'].__get__(obj, A)`. If not a descriptor, `m` is returned unchanged.

Summary of invocation logic

* Descriptors are invoked by the `__getattribute__()` method.
* Classes inherit this machinery from `object`, `type`, or `super()`.
* Overriding `__getattribute__()` prevents automatic descriptor calls because all the descriptor logic is in that method.
* `object.__getattribute__()` and `type.__getattribute__()` make different calls to `__get__()`. The first includes the instance and may include the class. The second puts in `None` for the instance and always includes the class.
* Data descriptors always override instance dictionaries.
* Non-data descriptors may be overridden by instance dictionaries.

### Properties and descriptors

* Calling `property()` is a succinct way of building a data descriptor that triggers a function call upon access to an attribute.

In [ ]:
class Property:
    "Emulate PyProperty_Type() in Objects/descrobject.c"

    def __init__(self, fget=None, fset=None, fdel=None, doc=None):
        self.fget = fget
        self.fset = fset
        self.fdel = fdel
        if doc is None and fget is not None:
            doc = fget.__doc__
        self.__doc__ = doc
        self._name = ''

    def __set_name__(self, owner, name):
        self._name = name

    def __get__(self, obj, objtype=None):
        if obj is None:
            return self
        if self.fget is None:
            raise AttributeError(
                f'property {self._name!r} of {type(obj).__name__!r} object has no getter'
             )
        return self.fget(obj)

    def __set__(self, obj, value):
        if self.fset is None:
            raise AttributeError(
                f'property {self._name!r} of {type(obj).__name__!r} object has no setter'
             )
        self.fset(obj, value)

    def __delete__(self, obj):
        if self.fdel is None:
            raise AttributeError(
                f'property {self._name!r} of {type(obj).__name__!r} object has no deleter'
             )
        self.fdel(obj)

    def getter(self, fget):
        prop = type(self)(fget, self.fset, self.fdel, self.__doc__)
        prop._name = self._name
        return prop

    def setter(self, fset):
        prop = type(self)(self.fget, fset, self.fdel, self.__doc__)
        prop._name = self._name
        return prop

    def deleter(self, fdel):
        prop = type(self)(self.fget, self.fset, fdel, self.__doc__)
        prop._name = self._name
        return prop

In [3]:
# Suppose we have a `Polygon` class that has a vertices property that needs to
# be defined as a sequence of `Point2D` instances. So here, not only do we want
# the `vertices` attribute of our `Polygon` to be an iterable of some kind, we
# also want the elements to all be instances of the `Point2D` class. In turn
# we'll also want to make sure that coordinates for `Point2D` are non-negative
# integer values (as might be expected in computer screen coordinates).

class RealNumber:
    def __set_name__(self, owner, name):
        self.name = name

    def __set__(self, instance, value):
        if not isinstance(value, (int, float)):
            raise TypeError(f'{self.name} must be a real number')
        instance.__dict__[self.name] = value
        
    def __get__(self, instance, owner=None):
        if instance is None:
            return self
        return instance.__dict__.get(self.name)
    
class Point2D:
    x = RealNumber()
    y = RealNumber()

    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.value = (float(self.x), float(self.y))

    def __repr__(self):
        return f'Point2D(x={self.x}, y={self.y})'
    
    def __eq__(self, other):
        if not isinstance(other, Point2D):
            try:
                other = Point2D(*other)
            except TypeError:
                return NotImplemented
        return self.value == other.value
    
    def __hash__(self):
        return hash(self.value)
    
    def __iter__(self):
        return iter((self.x, self.y))

    
class NonNegativeIntPoint2DIterable:
    def __set_name__(self, owner, name):
        self.name = name

    def __set__(self, instance, value):
        if not hasattr(value, '__iter__'):
            raise TypeError(f'{self.name} must be an iterable of Point2D objects')
        point2ds = []
        for element in value:
            if not isinstance(element, Point2D):
                try:
                    element = Point2D(*element)
                except TypeError:
                    raise TypeError(f'{element!r} should be a Point2D object or a compatible iterable')
            for coordinate in element:
                if coordinate < 0 or not isinstance(coordinate, int):
                    raise ValueError(f'coordinates of {element} should be non-negative integers')
            point2ds.append(element)
        instance.__dict__[self.name] = point2ds

    def __get__(self, instance, owner=None):
        if instance is None:
            return self
        return instance.__dict__.get(self.name)
        
class Polygon:
    vertices = NonNegativeIntPoint2DIterable()

    def __init__(self, vertices):
        self.vertices = vertices

p = Polygon([(1, 2), (3, 4), Point2D(0, 5)])
print(p.vertices)
print(Polygon.__dict__)
Polygon.vertices = 1 # class attribute vertices can be modified, may cause the class lose the descriptor attribute
print(Polygon.__dict__)
print(p.vertices) # still works, because it's in instance __dict__, but it will have problem when modifying the instance attribute value

[Point2D(x=1, y=2), Point2D(x=3, y=4), Point2D(x=0, y=5)]
{'__module__': '__main__', 'vertices': <__main__.NonNegativeIntPoint2DIterable object at 0x0000022743806570>, '__init__': <function Polygon.__init__ at 0x00000227437FF4C0>, '__dict__': <attribute '__dict__' of 'Polygon' objects>, '__weakref__': <attribute '__weakref__' of 'Polygon' objects>, '__doc__': None}
{'__module__': '__main__', 'vertices': 1, '__init__': <function Polygon.__init__ at 0x00000227437FF4C0>, '__dict__': <attribute '__dict__' of 'Polygon' objects>, '__weakref__': <attribute '__weakref__' of 'Polygon' objects>, '__doc__': None}


[Point2D(x=1, y=2), Point2D(x=3, y=4), Point2D(x=0, y=5)]

### Functions and descriptors

* Functions are objects that implement non-data descriptor protocol.

In [13]:
import sys

def add(self, a, b):
    return a + b

print(hasattr(add, '__get__'))
print(hasattr(add, '__set__'))
print(hasattr(type(add), '__get__'))
print(hasattr(type(add), '__set__'))

owner = sys.modules['__main__']
print(add.__get__(None, owner))
print(add)

instance = object()
bound_method = add.__get__(instance, owner)
print(bound_method) # when __get__ is called with instance, functions is bound to the instance
print(bound_method(1, 2))
print(bound_method.__func__)

True
False
True
False
<function add at 0x000001249DB05260>
<function add at 0x000001249DB05260>
<bound method add of <object object at 0x000001249DE96C20>>
3
<function add at 0x000001249DB05260>


In [11]:
class FuncDescriptor:
    def __init__(self, func):
        self.func = func

    def __get__(self, instance, owner=None):
        if instance is None:
            return self.func
        def method(*args, **kwargs):
            return self.func(instance, *args, **kwargs)
        return method

def work(self, subject):
    print(f'{self} is working on {subject}...')
    
class Person:
    work = FuncDescriptor(work)

    def __init__(self, name):
        self.name = name

    def __repr__(self):
        return f'Person({self.name})'

p = Person('John')
p.work('programming')
print(Person.work)
print(p.work)

Person(John) is working on programming...
<function work at 0x0000025A65B95D00>
<function FuncDescriptor.__get__.<locals>.method at 0x0000025A6607EFC0>


In [12]:
from types import MethodType

class FuncDescriptor:
    def __init__(self, func):
        self.func = func

    def __get__(self, instance, owner=None):
        if instance is None:
            return self.func
        return MethodType(self.func, instance)

def work(self, subject):
    print(f'{self} is working on {subject}...')
    
class Person:
    work = FuncDescriptor(work)

    def __init__(self, name):
        self.name = name

    def __repr__(self):
        return f'Person({self.name})'

p = Person('John')
p.work('programming')
print(Person.work)
print(p.work)

Person(John) is working on programming...
<function work at 0x0000025A6607E480>
<bound method work of Person(John)>


## Project 4

Project

* Define classes with fields that need to be validated before value setting.
* Write two data descriptors
  * `IntegerField`: allows only integral numbers, between minimum and maximum value
  * `CharField`: allows only strings with a minimum and maximum length
  * Make sure you can also omit one or both of the minimum and maximum values where it makes sense.
* Refactor your code and create a `BaseValidator` class that will handle the common functionality.
  * Change your `IntegerField` and `CharField` descriptors to inherit from `BaseValidator`.

In [4]:
class BaseValidator:
    def __init__(self, min_=None, max_=None):
        self.min = min_
        self.max = max_

    def __set_name__(self, owner, name):
        self.name = name

    def __get__(self, instance, owner):
        if instance is None:
            return self
        return instance.__dict__.get(self.name)
    
    def __set__(self, instance, value):
        value = self.validate(value)
        instance.__dict__[self.name] = value

    def validate(self, value):
        """need to be implemented by each subclass"""
        return value

class IntegerField(BaseValidator):
    def validate(self, value):
        if not isinstance(value, int):
            raise TypeError('{self.name} value must be an int')
        if self.min is not None and value < self.min:
            raise ValueError('{self.name} value cannot be less than {self.min}')
        if self.max is not None and value > self.max:
            raise ValueError('{self.name} value cannot be greater than {self.max}')
        return value

class CharField(BaseValidator):
    # def __init__(self, min_=None, max_=None):
    #       # no need to set self.min to non-negative, because it's impossible
    #     min_ = max(0, min_ or 0)
    #     super().__init__(min_, max_)

    def validate(self, value):
        if not isinstance(value, str):
            raise TypeError('{self.name} value must be a str')
        value = value.strip()
        if self.min is not None and len(value) < self.min:
            raise ValueError('{self.name} length cannot be less than {self.min}')
        if self.max is not None and len(value) > self.max:
            raise ValueError('{self.name} length cannot be greater than {self.max}')
        return value

In [5]:
import unittest

class TestIntegerField(unittest.TestCase):
    @staticmethod
    def create_test_class(min_=None, max_=None):
        obj = type('TestClass', (), {'age': IntegerField(min_, max_)})
        return obj()
        
    def test_set_age_ok(self):
        """Tests that valid values can be assigned/retrieved"""
        min_ = 5
        max_ = 10
        obj = self.create_test_class(min_, max_)
        valid_values = range(min_, max_)
        
        for i, value in enumerate(valid_values):
            with self.subTest(test_number=i):
                obj.age = value
                self.assertEqual(value, obj.age)
                
    def test_set_age_invalid(self):
        """Tests that invalid values raise ValueErrors or TypeError"""
        min_ = -10
        max_ = 10
        obj = self.create_test_class(min_, max_)
        bad_values = list(range(min_ - 5, min_)) + list(range(max_ + 1, max_ + 6))
        
        for i, value in enumerate(bad_values):
            with self.subTest(test_number=i):
                with self.assertRaises(ValueError):
                    obj.age = value

        bad_values = [10.5, 1 + 0j, 'abc', (1, 2)]      
        for i, value in enumerate(bad_values):
            with self.subTest(test_number=i):
                with self.assertRaises(TypeError):
                    obj.age = value

    def test_class_get(self):
        """Tests that class attribute retrieval returns the descriptor instance"""
        obj = self.create_test_class()
        obj_class = type(obj)
        self.assertIsInstance(obj_class.age, IntegerField)
        
    def test_set_age_min_only(self):
        """Tests that we can specify a min value only"""
        min_ = 0
        obj = self.create_test_class(min_)
        values = range(min_, min_ + 100, 10)
        for i, value in enumerate(values):
            with self.subTest(test_number=i):
                obj.age = value
                self.assertEqual(value, obj.age)
                
    def test_set_age_max_only(self):
        """Tests that we can specify a max value only"""
        max_ = 10
        obj = self.create_test_class(max_=max_)
        values = range(max_ - 100, max_, 10)
        for i, value in enumerate(values):
            with self.subTest(test_number=i):
                obj.age = value
                self.assertEqual(value, obj.age)
                
    def test_set_age_no_limits(self):
        """Tests that we can use IntegerField without any limits at all"""
        obj = self.create_test_class()
        values = range(-100, 100, 10)
        for i, value in enumerate(values):
            with self.subTest(test_number=i):
                obj.age = value
                self.assertEqual(value, obj.age)

class TestCharField(unittest.TestCase):
    @staticmethod
    def create_test_class(min_=None, max_=None):
        obj = type('TestClass', (), {'name': CharField(min_, max_)})
        return obj()
        
    def test_set_name_ok(self):
        """Tests that valid values can be assigned/retrieved"""
        min_ = 1
        max_ = 10
        obj = self.create_test_class(min_, max_)
        valid_lengths = range(min_, max_)
        
        for i, length in enumerate(valid_lengths):
            value = 'a' * length
            with self.subTest(test_number=i):
                obj.name = value
                self.assertEqual(value, obj.name)
            
    def test_set_name_invalid(self):
        """Tests that invalid values raise ValueErrors or TypeError"""
        min_ = 5
        max_ = 10
        obj = self.create_test_class(min_, max_)
        bad_lengths = list(range(min_ - 5, min_)) + list(range(max_ + 1, max_ + 6))
        for i, length in enumerate(bad_lengths):
            value = 'a' * length
            with self.subTest(test_number=i):
                with self.assertRaises(ValueError):
                    obj.name = value
        
        bad_values = [1, 1.1, ['a'], 5j]
        for i, value in enumerate(bad_values):
            with self.subTest(test_number=i):
                with self.assertRaises(TypeError):
                    obj.name = value
                    
    def test_class_get(self):
        """Tests that class attribute retrieval returns the descriptor instance"""
        obj = self.create_test_class(0, 0)
        obj_class = type(obj)
        self.assertIsInstance(obj_class.name, CharField)
        
    def test_set_name_min_only(self):
        """Tests that we can specify a min length only"""
        min_ = 0
        obj = self.create_test_class(min_)
        valid_lengths = range(min_, min_ + 100, 10)
        for i, length in enumerate(valid_lengths):
            value = 'a' * length
            with self.subTest(test_number=i):
                obj.name = value
                self.assertEqual(value, obj.name)
    
    # def test_set_name_min_negative_or_none(self):
    #     """Tests that setting a negative or None length results in a zero length"""
    #     obj = self.create_test_class(-10, 100)
    #     self.assertEqual(type(obj).name._min, 0)
    #     self.assertEqual(type(obj).name._max, 100)
        
    #     obj = self.create_test_class()
    #     self.assertEqual(type(obj).name._min, 0)
    #     self.assertIsNone(type(obj).name._max)
        
    def test_set_name_max_only(self):
        """Tests that we can specify a max length only"""
        max_ = 10
        obj = self.create_test_class(max_=max_)
        valid_lengths = range(max_ - 100, max_, 10)
        for i, length in enumerate(valid_lengths):
            value = 'a' * length # 'a' * -1 returns ''
            with self.subTest(test_number=i):
                obj.name = value
                self.assertEqual(value, obj.name)
                
    def test_set_name_no_limits(self):
        """Tests that we can use CharField without any limits at all"""
        obj = self.create_test_class()
        valid_lengths = range(0, 100, 10)
        for i, length in enumerate(valid_lengths):
            value = 'a' * length
            with self.subTest(test_number=i):
                obj.name = value
                self.assertEqual(value, obj.name)
                
    def test_set_name_strip(self):
        """Tests that values will be stripped when get stored"""
        obj = self.create_test_class()
        values = [' abc', 'ab ', ' add ddd ']
        for i, value in enumerate(values):
            with self.subTest(test_number=i):
                obj.name = value
                self.assertEqual(obj.name, value.strip())

def run_tests(test_class):
    suite = unittest.TestLoader().loadTestsFromTestCase(test_class)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(suite)

run_tests(TestIntegerField)
run_tests(TestCharField)

test_class_get (__main__.TestIntegerField.test_class_get)
Tests that class attribute retrieval returns the descriptor instance ... ok
test_set_age_invalid (__main__.TestIntegerField.test_set_age_invalid)
Tests that invalid values raise ValueErrors or TypeError ... ok
test_set_age_max_only (__main__.TestIntegerField.test_set_age_max_only)
Tests that we can specify a max value only ... ok
test_set_age_min_only (__main__.TestIntegerField.test_set_age_min_only)
Tests that we can specify a min value only ... ok
test_set_age_no_limits (__main__.TestIntegerField.test_set_age_no_limits)
Tests that we can use IntegerField without any limits at all ... ok
test_set_age_ok (__main__.TestIntegerField.test_set_age_ok)
Tests that valid values can be assigned/retrieved ... ok

----------------------------------------------------------------------
Ran 6 tests in 0.017s

OK
test_class_get (__main__.TestCharField.test_class_get)
Tests that class attribute retrieval returns the descriptor instance ... ok


## Enumerations

### Enumerations

* An enumeration:
  * is a set of symbolic names (members) bound to unique values
  * can be iterated over to return its canonical (i.e. non-alias) members in definition order
  * uses call syntax to return members by value
  * uses index syntax to return members by name
* Enumerations are created either by using class syntax, or by using function-call syntax:
    ```Python
    from enum import Enum

    # class syntax
    class Color(Enum):
        RED = 1
        GREEN = 2
        BLUE = 3

    # functional syntax
    Color = Enum('Color', ['RED', 'GREEN', 'BLUE'])
    ```
  * The class `Color` is an enumeration (or `enum`)
  * The attributes `Color.RED`, `Color.GREEN`, etc., are enumeration members (or members) and are functionally constants.
  * The type of a member is the enumeration (e.g. `Color` here) it belongs to.
  * The enum members have names and values (the name of `Color.RED` is `RED`, the value of `Color.BLUE` is `3`, etc.)
  * The member and its associated value are not equal.
  * Enumeration members are always hashable.
  * `Color(1)` -> `Color.RED`
  * `Color['RED']` -> `Color.RED`
  * Enumerations are iterable.
    * definition order is preserved
    * `list(Color)` -> `[Color.RED, Color.GREEN, Color.BLUE]`
  * Enumerations have a `__members__` property
    * return a mapping proxy (read-only dict) of every enum name to its member, including aliases
    * keys are the names, and values are the values
  * Once an enumeration has been declared
    * member list is immutable
    * member values are immutable
    * cannot be subclassed unless it contains no members

In [15]:
from enum import Enum

Color = Enum('Color', ('RED', 'GREEN', 'BLUE'))
print(Color)
print(Color.RED)
print(Color(1))
print(Color['RED'])
print(getattr(Color, 'RED'))
print(Color.RED.name)
print(Color.RED.value)
Color = Enum('Color', {'RED': 'red', 'GREEN': 'green', 'BLUE': 'blue'})
print(Color.RED.value)
print(type(Color.RED))
print(list(Color))
print(Color.__members__)

<enum 'Color'>
Color.RED
Color.RED
Color.RED
Color.RED
RED
1
red
<enum 'Color'>
[<Color.RED: 'red'>, <Color.GREEN: 'green'>, <Color.BLUE: 'blue'>]
{'RED': <Color.RED: 'red'>, 'GREEN': <Color.GREEN: 'green'>, 'BLUE': <Color.BLUE: 'blue'>}


### Aliases

* By definition, enumeration member values are unique. However, you can create different member names with the same values.
* When you define multiple members with the same values, Python treats them as aliases of the main member.
* When you look up a member by value, you'll always get the main member (not the aliases).
* When you iterate through the members of an enumeration with aliases, you'll only get the main members (not the aliases).
* To access all members (including aliases), use the `__members__` property.
* Enumeration aliases can be helpful in certain situations.
  * For instance, when dealing with APIs from different systems that use different status codes with the same meaning, you can standardize them using aliases.
* Python's `enum` module provides the `@enum.unique` decorator to enforce uniqueness of member values, preventing unintended aliases.

In [17]:
from enum import Enum

class NumSides(Enum):
    TRIANGLE = 3
    RECTANGLE = 4
    SQUARE = 4
    RHOMBUS = 4

print(NumSides.RECTANGLE is NumSides.SQUARE)
print(NumSides.RECTANGLE is NumSides.RHOMBUS)
print(NumSides.RHOMBUS in NumSides)
print(NumSides(4))
print(NumSides['RHOMBUS'])
print(list(NumSides))
print(NumSides.__members__)

True
True
True
NumSides.RECTANGLE
NumSides.RECTANGLE
[<NumSides.TRIANGLE: 3>, <NumSides.RECTANGLE: 4>]
{'TRIANGLE': <NumSides.TRIANGLE: 3>, 'RECTANGLE': <NumSides.RECTANGLE: 4>, 'SQUARE': <NumSides.RECTANGLE: 4>, 'RHOMBUS': <NumSides.RECTANGLE: 4>}


### Customizing and extending enumerations

Customizing enums

* Enums are classes, and their class attributes become instances (members) of that class.
* We can define functions in the enumeration classes, and they become methods when called from a member.
* Member truthyness
  * By default, every member of an enum is truthy, irrespective of its value.
  * We can implement the `__bool__` method to override the default behavior.

Extending enums

* Enumerations are classes so that they can be extended (subclassed), but only if they do not contain any members.
* We can create a base enum with functionality (methods), and use it as a base class for other enumerations.

In [20]:
from enum import Enum
from functools import total_ordering

@total_ordering
class Number(Enum):
    ONE = 1
    TWO = 2
    THREE = 3

    def __eq__(self, other):
        if isinstance(other, Number):
            return self is other
        if isinstance(other, (int, float)):
            return self.value == other
        return NotImplemented
    
    def __hash__(self):
        return hash(self.value)
    
    def __lt__(self, other):
        if isinstance(other, Number):
            return self.value < other.value
        if isinstance(other, (int, float)):
            return self.value < other
        return NotImplemented
    
print(Number.ONE < Number.TWO)
print(Number.ONE == 1)
print(Number.ONE == 1.0)
print(Number.THREE >= 2.0)
print(Number.THREE == Number(3))
print(Number.TWO == '2')

True
True
True
True
True
False


In [21]:
from enum import Enum
from functools import total_ordering

@total_ordering
class OrderedEnum(Enum):
    """Enumeration supporting ordering based on member values"""
    def __lt__(self, other):
        if isinstance(other, type(self)):
            return self.value < other.value
        return NotImplemented

class Number(OrderedEnum):
    ONE = 1
    TWO = 2
    THREE = 3

print(Number.ONE == Number.TWO)
print(Number.ONE <= Number.TWO)
print(Number.THREE > Number.TWO)

False
True
True


In [23]:
from enum import Enum

class TwoValueEnum(Enum):
    def __new__(cls, member_value, member_phrase):
        member = object.__new__(cls)
        member._value_ = member_value
        member.phrase = member_phrase
        return member
    
class AppStatus(TwoValueEnum):
    OK = (1, 'Running')
    FAILED = (0, 'Failed')

    def __bool__(self):
        return bool(self.value)

print(AppStatus.OK)
print(AppStatus.OK.name)
print(AppStatus.OK.value)
print(AppStatus.OK.phrase)
print(bool(AppStatus.OK))
print(bool(AppStatus.FAILED))

AppStatus.OK
OK
1
Running
True
False


### Automatic values

* `class enum.auto`
  * `auto` can be used in place of a value. If used, the `Enum` machinery will call an Enum's `_generate_next_value_()` to get an appropriate value.
  * For `Enum` and `IntEnum` that appropriate value will be the last value plus one; for `Flag` and `IntFlag` it will be the first power-of-two greater than the highest value; for `StrEnum` it will be the lower-cased version of the member's name. Care must be taken if mixing `auto()` with manually specified values.
  * `auto` instances are only resolved when at the top level of an assignment:
    * `FIRST = auto()` will work (`auto()` is replaced with `1`);
    * `SECOND = auto(), -2` will work `auto()` is replaced with `2`, so `2, -2` is used to create the `SECOND` enum member;
    * `THREE = [auto(), -3]` will not work (`<auto instance>, -3` is used to create the `THREE` enum member)
    * Changed in version 3.11.1: In prior versions, `auto()` had to be the only thing on the assignment line to work properly.
    * `_generate_next_value_` can be overridden to customize the values used by `auto`.
* `_generate_next_value_(name, start, count, last_values)`
  * A staticmethod that is used to determine the next value returned by `auto`:
  * `name`: the name of the member being defined (e.g. `'RED'`)
  * `start`: the start value for the `Enum`; the default is `1`
  * `count`: the number of members currently defined, not including this one.
  * `last_values`: a list of the previous values.

In [26]:
from enum import Enum, auto

class State(Enum):
    WAITING = auto()
    STARTED = auto()
    FINISHED = auto()

print(list(State))

class Status(Enum):
    def _generate_next_value_(name, start, count, last_values):
        return name.capitalize()
    
    WAITING = auto()
    STARTED = auto()
    FINISHED = auto()
    
print(list(Status))

[<State.WAITING: 1>, <State.STARTED: 2>, <State.FINISHED: 3>]
[<Status.WAITING: 'Waiting'>, <Status.STARTED: 'Started'>, <Status.FINISHED: 'Finished'>]


In [ ]:
from enum import Enum

class ValueMeaningless(Enum):
    """values of the members are meaningless and should be used or counted on"""
    Member1 = object()
    Member2 = object()
    Member3 = object()

## Project 5

Background information

* In many cases, especially in larger projects, we want to have an easy way to generate exceptions:
  * raise consistent exception types
  * associated exception code
  * associated default exception message
  * ability to easily list out all the possible exceptions
* This is very common when writing REST APIs for example.

Project

* Single enumeration `AppException`
* Exceptions have a name and three associated values
  * name
  * code
  * associated exception type
  * default message
* Lookup by exception name or code
* Method to raise an exception
* Ability to override default message when throwing exception

In [37]:
from enum import Enum, auto

class AppException(Enum):
    def __new__(cls, member_code, member_exc_type, member_default_msg):
        member = object.__new__(cls)
        member._value_ = member_code
        member.exc_type = member_exc_type
        member.default_msg = member_default_msg
        return member
    
    NotAnInteger = auto(), TypeError, 'value is not an integer'
    Timeout = auto(), RuntimeError, 'timeout error'

    @property
    def code(self):
        return self.value
    
    def throw(self, msg=None):
        msg = msg or self.default_msg
        raise self.exc_type(f'{self.name}: {msg}')
    
print(list(AppException))
print(AppException.Timeout)
print(AppException(2))
print(AppException['Timeout'])
print(AppException.Timeout.code)
print(AppException.Timeout.value)
print(AppException.Timeout.name)
print(AppException.Timeout.exc_type)
print(AppException.Timeout.default_msg)
try:
    # AppException.NotAnInteger.throw('custom message')
    AppException.NotAnInteger.throw()
except AppException.NotAnInteger.exc_type as e:
    print(e)

[<AppException.NotAnInteger: 1>, <AppException.Timeout: 2>]
AppException.Timeout
AppException.Timeout
AppException.Timeout
2
2
Timeout
<class 'RuntimeError'>
timeout error
NotAnInteger: value is not an integer


## Exceptions (single inheritance)

### Python exceptions

What are exceptions

* Exceptions are a means of breaking out of the normal flow of control of a code block in order to handle errors or other exceptional conditions.
* An exception is raised at the point where the error is detected; it may be handled by the surrounding code block or by any code block that directly or indirectly invoked the code block where the error occurred.
  * The Python interpreter raises an exception when it detects a run-time error (such as division by zero).
  * A Python program can also explicitly raise an exception with the `raise` statement.
  * If current call does not handle the exception, it is propagated to the caller.
    * call stack is maintained
      * document origin of exception
      * every call in the stack

Exception handling

* Exceptions are not necessarily fatal, i.e. do not necessarily result in program termination.
  * We can handle exceptions as they occur:
    * do something and let the program running normally
    * do something and let the original exception propagate
    * do something and raise a different exception
  * When an exception is not handled at all, the interpreter terminates execution of the program, or returns to its interactive main loop. In either case, it prints a stack traceback, except when the exception is `SystemExit`.
* Exception handlers are specified with the `try...except...else...finally` statement. The `finally` clause of such a statement can be used to specify cleanup code which does not handle the exception, but is executed whether an exception occurred or not in the preceding code.
  * Exceptions are identified by *class* instances. The `except` clause is selected depending on the class of the instance: it must reference the class of the instance or a non-virtual base class thereof.
  * The `except` clause that mentions a particular class also handles any exception classes derived from that class (but not exception classes from which it is derived).
  * Two exception classes that are not related via subclassing are never equivalent, even if they have the same name.

Built-in exceptions

* Two main categories of exceptions
  * Compilation exceptions (e.g. `SyntaxError`, `IndentationError`, `TabError`)
  * Execution exceptions (e.g. `ZeroDivisionError`, `IndexError`, `KeyError`, `FileNotFoundError`)
* In Python, all exceptions must be instances of a class that derives from `BaseException`.
  * [Exception hierarchy](https://docs.python.org/3/library/exceptions.html#exception-hierarchy)
  * The built-in exception classes can be subclassed to define new exceptions; programmers are encouraged to derive new exceptions from the `Exception` class or one of its subclasses, and not from `BaseException`.
  * More information on defining exceptions is available in the Python Tutorial under [User-defined Exceptions](https://docs.python.org/3/tutorial/errors.html#tut-userexceptions).
* Exception objects are instances of exception classes.

In [1]:
print(f'{type(BaseException)=}')
print(f'{type(Exception)=}')
print(f'{issubclass(Exception, BaseException)=}')
ex = Exception()
print(f'{isinstance(ex, Exception)=}')
print(f'{isinstance(ex, BaseException)=}')
print(f'{ex.__class__=}')

type(BaseException)=<class 'type'>
type(Exception)=<class 'type'>
issubclass(Exception, BaseException)=True
isinstance(ex, Exception)=True
isinstance(ex, BaseException)=True
ex.__class__=<class 'Exception'>


In [6]:
def square_it(seq, index):
    return seq[index] ** 2

def squares(seq, first_n):
    try:
        for i in range(first_n):
            yield square_it(seq, i)
    except IndexError:
        return
    except TypeError as ex:
        print(ex)
        return

print(list(squares([1, 2, 3, 4, 5], 10)))
print(list(squares([1, 2, '3', 4, 5], 4)))
print(list(squares(1, 4)))
print(list(squares([1, 2, 3], '4')))

[1, 4, 9, 16, 25]
unsupported operand type(s) for ** or pow(): 'str' and 'int'
[1, 4]
'int' object is not subscriptable
[]
'str' object cannot be interpreted as an integer
[]


### Handling exceptions

* It is possible to write programs that handle selected exceptions.
* The `try` statement works as follows.
  * First, the `try` clause (the statement(s) between the `try` and `except` keywords) is executed.
  * If no exception occurs, the `except` clause is skipped and execution of the `try` statement is finished.
  * If an exception occurs during execution of the `try` clause, the rest of the clause is skipped. Then, if its type matches the exception named after the `except` keyword, the `except` clause is executed, and then execution continues after the `try...except` block.
  * If an exception occurs which does not match the exception named in the `except` clause, it is passed on to outer `try` statements; if no handler is found, it is an unhandled exception and execution stops with an error message.
* A `try` statement must have at least one `except` or `finally` clause.
* A `try` statement may have zero or more `except` clauses, to specify handlers for different exceptions.
  * At most one handler will be executed.
  * Arrange the `except` clauses from most specific to least specific, i.e. you should catch more specialized exceptions before more general ones.
  * If you want to handle all exceptions, it's better to use `except Exception:` instead of a plain `except:`. The latter catches even `SystemExit` and `KeyboardInterrupt`, which may not be desired.
* An expression-less `except` clause, if present, must be last.
  * It matches any exception.
  * It is good in certain circumstances.
    * do some cleanup work
    * logging
    * re-raise exception
  * Use `sys.exc_info()` to get info about current exception.
    * return a tuple `(exc_type, exc_value, exc_traceback)`
* An `except` clause may name multiple exceptions as a parenthesized *tuple*.
* A class in an `except` clause is compatible with an exception if it is the same class or a base class thereof.
* When an exception occurs, it may have associated values, also known as the exception's arguments. The presence and types of the arguments depend on the exception type.
  * Standard exceptions have at least two properties:
    * `args`
      * arguments used to create exception objects
      * often error messages
    * `__traceback__`
      * the traceback object
      * use `traceback` module for easier visualization: `print_tb`, `print_exception`
      * it is the same object as last item returned by `sys.exc_info()`
* The `except` clause may specify a variable after the exception name with the `as` keyword.
  * The variable is bound to the exception instance.
  * The variable is cleared at the end of the `except` clause.
    * This means the exception must be assigned to a different name to be able to refer to it after the `except` clause.
* The `except*` clause(s) are used for handling `ExceptionGroups`. The exception type for matching is interpreted as in the case of `except`, but in the case of exception groups we can have partial matches when the type matches some of the exceptions in the group.
* `BaseException` is the common base class of all exceptions.
  * One of its subclasses, `Exception`, is the base class of all the *non-fatal* exceptions.
  * Exceptions which are not subclasses of `Exception` are not typically handled, because they are used to indicate that the program should terminate.
    * They include `SystemExit` which is raised by `sys.exit()`,
    * `KeyboardInterrupt` which is raised when a user wishes to interrupt the program, and
    * `GeneratorExit` which is raised when a generator or coroutine is closed and it is technically not an error.
* The most common pattern for handling `Exception` is to print or log the exception and then re-raise it (allowing a caller to handle the exception as well).
  * A bare `raise` statement re-raises the last caught exception. Whether this occurs within a `try...except` block or not, if there has been a previously caught exception, invoking `raise` will re-raise that same exception.
  * If you encounter a situation where you want to propagate the same exception further up the call stack, you can use `raise` without any arguments.
* The `try...except` statement has an optional `else` clause, which, when present, must follow all `except` clauses hence it requires at least on `except` clause present.
  * The `else` clause is executed if the control flow leaves the `try` suite, no exception was raised, and no `return`, `continue`, or `break` statement was executed.
  * It is useful for code that must be executed if the `try` clause does not raise an exception.
  * The use of the `else` clause is better than adding additional code to the `try` clause because it avoids accidentally catching an exception that wasn't raised by the code being protected by the `try...except` statement.
* The try statement has another optional `finally` clause which is intended to define clean-up actions that must be executed under all circumstances.
  * If a `finally` clause is present, it will execute as the last task before the `try` statement completes.
  * The `finally` clause runs whether or not the `try` statement produces an exception.
  * The `finally` statement works as follows.
    * The `try` clause is executed, including any `except` and `else` clauses.
    * If an exception occurs in any of the clauses and is not handled, the exception is temporarily saved.
    * The `finally` clause is executed. If there is a saved exception it is *re-raised* at the end of the `finally` clause.
    * If the `finally` clause raises another exception, the saved exception is set as the context of the new exception.
    * If the `finally` clause executes a `return`, `break` or `continue` statement, the saved exception is discarded.
  * The following points discuss more complex cases when an exception occurs:
    * If an exception occurs during execution of the `try` clause, the exception may be handled by an `except` clause. If the exception is not handled by an `except` clause, the exception is re-raised after the `finally` clause has been executed.
    * An exception could occur during execution of an `except` or `else` clause. Again, the exception is re-raised after the `finally` clause has been executed.
    * If the `finally` clause executes a `break`, `continue` or `return` statement, exceptions are not re-raised.
    * If the `try` statement reaches a `break`, `continue` or `return` statement, the `finally` clause will execute just prior to the `break`, `continue` or `return` statement's execution.
    * If a `finally` clause includes a `return` statement, the returned value will be the one from the `finally` clause's return statement, not the value from the `try` clause's return statement.
  * In real world applications, the `finally` clause is useful for releasing external resources (such as files or network connections), regardless of whether the use of the resource was successful.
  * Prior to Python 3.8, a `continue` statement was illegal in the `finally` clause due to a problem with the implementation.
* Exception handling can be nested inside the `try` clause, `except`, `else`, and even `finally`.

Handling exceptions vs avoiding exceptions

* It is easier to ask forgiveness than it is to get permission.
  * Sometimes referred to as the EAFP (easier to ask for forgiveness than permission) principle in Python.
  * The saying suggests that you should take action without seeking permission first. If your actions end up upsetting certain people, it's easier to apologize and seek forgiveness afterward rather than waiting for approval initially. However, while the idea seems valid, the philosophy behind it is flawed.
  * The saying implies that it's more convenient to act first and apologize later if necessary. Instead of seeking permission upfront, you proceed with your plans and deal with any consequences afterward.
  * The phrase is attributed to Grace Murray Hopper, a renowned computer scientist. Hopper used this philosophy in her work on computer programs, emphasizing action over hesitation. She believed in making progress by doing what she knew would work, even if it meant asking for forgiveness later.
  * Wisdom or Selfishness:
    * While the saying has some merit, it's not a wise way to live:
      * Repeatedly acting without permission erodes trust.
      * People may perceive you as selfish or irresponsible.
      * Accountability often falls on those who consistently bypass permission.
    * In most cases, it's better to seek permission when necessary rather than relying on forgiveness.

In [6]:
import json

class Person:
    __slots__ = 'name', '_age'

    def __init__(self, name):
        self.name = name

    @property
    def age(self):
        return self._age
    
    @age.setter
    def age(self, value):
        if isinstance(value, int) and value >= 0:
                self._age = value
        else:
            raise ValueError('age must be a non-negative int')
        
    def __repr__(self):
        return f'Person({self.name}, {self._age})'

json_data = """{
    "Alex": {"age": 18},
    "Bryan": {"age": 21, "city": "London"},
    "Guido": {"age": "unknown"}
}"""

data_dict = json.loads(json_data)
persons = []

for name, attributes_dict in data_dict.items():
    try:
        person = Person(name)
        for attribute, value in attributes_dict.items():
            try:
                setattr(person, attribute, value)
            except AttributeError:
                print(f'ignored invalid attribute: Person({person.name}).{attribute}={value}')
    except ValueError as ex:
        print(f'ignored Person({person.name}) with invalid attribute value: {ex}')
    else:
        persons.append(person)

print(persons)

ignored invalid attribute: Person(Bryan).city=London
ignored Person(Guido) with invalid attribute value: age must be a non-negative int
[Person(Alex, 18), Person(Bryan, 21)]


In [10]:
import json

class Person:
    __slots__ = 'name', '_age'

    def __init__(self, name):
        self.name = name

    @property
    def age(self):
        return self._age
    
    @age.setter
    def age(self, value):
        if isinstance(value, int) and value >= 0:
                self._age = value
        else:
            raise ValueError('age must be a non-negative int')
        
    def __repr__(self):
        return f'Person({self.name}, {self._age})'

json_data = """{
    "Alex": {"age": 18},
    "Bryan": {"age": 21, "city": "London"},
    "Guido": {"age": "unknown"}
}"""

data_dict = json.loads(json_data)
persons = []

for name, attributes_dict in data_dict.items():
    person = Person(name)
    skipped = False
    for attribute, value in attributes_dict.items():
        try:
            setattr(person, attribute, value)
        except AttributeError:
            print(f'ignored invalid attribute: Person({person.name}).{attribute}={value}')
        except ValueError as ex:
            print(f'ignored Person({person.name}) with invalid attribute value: {ex}')
            skipped = True
            break
    if not skipped:
        persons.append(person)

print(persons)

ignored invalid attribute: Person(Bryan).city=London
ignored Person(Guido) with invalid attribute value: age must be a non-negative int
[Person(Alex, 18), Person(Bryan, 21)]


In [21]:
def booleanize_int(value):
    if not isinstance(value, int):
        raise TypeError()
    if value not in {0, 1}:
        raise ValueError('integer values 0 or 1 only')
    return bool(value)

def booleanize_str(value):
    if not isinstance(value, str):
        raise TypeError()
    value = value.casefold()
    if value in {'t', 'true', '1'}:
        return True
    if value in {'f', 'false', '0'}:
        return False
    raise ValueError("admissible string values are 'T', 'F', 'True', 'False', '1', '0' only")

def booleanize(value):
    try:
        try:
            booleanized = booleanize_int(value)
        except TypeError:
            try:
                booleanized = booleanize_str(value)
            except TypeError:
                raise TypeError(f'incompatible value type: {type(value).__name__} type cannot be booleanized')
    except ValueError as ex:
        raise ValueError(f'invalid value: {ex}')
    return booleanized

values = ['T', 't', 'true', 'f', 'False', '0', '1', 1, 0, 2, 1.0, -1, 'truthy', ' true']
for value in values:
    try:
        result = booleanize(value)
    except (TypeError, ValueError) as ex:
        print(f'{value!r} = {ex}')
    else:
        print(f'{value!r} = {result}')

'T' = True
't' = True
'true' = True
'f' = False
'False' = False
'0' = False
'1' = True
1 = True
0 = False
2 = invalid value: integer values 0 or 1 only
1.0 = incompatible value type: float type cannot be booleanized
-1 = invalid value: integer values 0 or 1 only
'truthy' = invalid value: admissible string values are 'T', 'F', 'True', 'False', '1', '0' only
' true' = invalid value: admissible string values are 'T', 'F', 'True', 'False', '1', '0' only


### Raising exceptions

* The `raise` statement allows the programmer to force a specified exception to occur.
  * If no expressions are present, `raise` re-raises the exception that is currently being handled, which is also known as the *active exception*. If there isn't currently an active exception, a `RuntimeError` exception is raised indicating that this is an error.
  * If you need to determine whether an exception was raised but don't intend to handle it, the expression-less `raise` statement allows you to re-raise the exception.
  * Otherwise, `raise` evaluates the first expression as the exception object.
    * It must be either a subclass or an instance of BaseException. If it is a class, the exception instance will be obtained when needed by instantiating the class with no arguments.
* The type of the exception is the exception instance's class, the value is the instance itself.
* A traceback object is normally created automatically when an exception is raised and attached to it as the `__traceback__` attribute.
  * You can create an exception and set your own traceback in one step using the `with_traceback()` exception method (which returns the same exception instance, with its traceback set to its argument), like so: `raise Exception("foo occurred").with_traceback(tracebackobj)`

Exception chaining

* If an unhandled exception occurs inside an `except` section, it will have the exception being handled attached to it and included in the error message.
* To indicate that an exception is a direct consequence of another, the `raise` statement allows an optional `from` clause.
  * if given, the second expression must be another exception class or instance.
    * If the second expression is an exception instance, it will be attached to the raised exception as the `__cause__` attribute (which is *writable*).
    * If the expression is an exception class, the class will be instantiated and the resulting exception instance will be attached to the raised exception as the `__cause__` attribute.
* A similar mechanism works implicitly if a new exception is raised when an exception is already being handled. An exception may be handled when an `except` or `finally` clause, or a `with` statement, is used. The previous exception is then attached as the new exception's `__context__` attribute.
* Exception chaining can be explicitly suppressed by specifying `None` in the `from` clause, i.e. automatic exception chaining can be disabled using the `from None` idiom.

In [6]:
class CustomException(Exception):
    """a custom exception class"""

def div(a, b):
    try:
        return a / b
    except ZeroDivisionError as ex:
        raise CustomException(*ex.args)
    
try:
    div(1, 0)
except Exception as ex:
    print(repr(ex))

def div(a, b):
    try:
        return a / b
    except ZeroDivisionError as ex:
        raise CustomException() from ex
    
try:
    div(1, 0)
except Exception as ex:
    print(repr(ex), repr(ex.__cause__))

CustomException('division by zero')
CustomException() ZeroDivisionError('division by zero')


### Custom exceptions

* Programs may name their own exceptions by creating a new exception class.
* Exceptions should typically be derived from the `Exception` class, either directly or indirectly.
* Exception classes can be defined which do anything any other class can do, but are usually kept simple, often only offering a number of attributes that allow information about the error to be extracted by handlers for the exception.
* Most exceptions are defined with names that end in "Error", similar to the naming of the standard exceptions.
* Many standard modules define their own exceptions to report errors that may occur in functions they define.

Creating an exception hierarchy

* Often we have an entire set of custom exceptions.
* To keep the exceptions organized, create a hierarchy of custom exceptions.
* This also allows trapping exceptions at multiple levels.

Multi inheritance

* Custom exception classes can inherit from multiple exceptions, so that such exceptions can be trapped with either of their base classes.

In [ ]:
class WebScraperException(Exception):
    """Base exception for WebScraper"""

class HTTPException(WebScraperException):
    """General HTTP exception for WebScraper"""

class InvalidURLException(HTTPException):
    """Invalid URL"""

class TimeoutException(HTTPException):
    """General timeout exception in HTTP connectivity"""

class PingTimeoutException(TimeoutException):
    """Ping time out"""

class LoadTimeoutException(TimeoutException):
    """Page load time out"""

class ParserException(WebScraperException):
    """General page parsing exception"""

In [ ]:
class APIException(Exception):
    """Base API exception"""

class ApplicationException(APIException):
    """Indicates an application error (not user caused) - 5xx HTTP type error"""

class DBException(ApplicationException):
    """General database exception"""

class DBConnectionError(DBException):
    """Indicates an error connecting to database"""

class ClientException(APIException):
    """Indicates exception that was caused by user, not an internal error"""

class NotFoundError(ClientException):
    """Indicates resource was not found"""

class NotAuthorizedError(ClientException):
    """User is not authorized to perform requested action on resource"""

In [4]:
from http import HTTPStatus
import json
from datetime import datetime, timezone

class APIException(Exception):
    """Base API exception"""
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    internal_err_msg = 'API exception occurred'
    user_err_msg = 'an unexpected internal error occurred'

    def __init__(self, *args, user_err_msg=None):
        if args:
            self.internal_err_msg = args[0]
            super().__init__(*args)
        else:
            super().__init__(self.internal_err_msg)

        if user_err_msg is not None:
            self.user_err_msg = user_err_msg

    def to_json(self):
        err_dict = {'status': self.http_status,
                    'internal_message': self.internal_err_msg,
                    'user_message': self.user_err_msg
                    }
        return json.dumps(err_dict)

    def log_exception(self):
        exception_dict = {
            'exception_type': type(self).__name__,
            'http_status': self.http_status,
            'internal_message': self.internal_err_msg,
            'user_message': self.user_err_msg,
            'args': self.args
        }
        print(f'{datetime.now(timezone.utc).isoformat()}: {exception_dict}')

class ApplicationException(APIException):
    """Indicates an application error (not user caused) - 5xx HTTP type error"""
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    internal_err_msg = 'generic server side exception occurred'
    user_err_msg = 'an unexpected internal error occurred'

class DBException(ApplicationException):
    """General database exception"""
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    internal_err_msg = 'database exception occurred'
    user_err_msg = 'an unexpected internal error occurred'

class DBConnectionError(DBException):
    """Indicates an error connecting to database"""
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    internal_err_msg = 'database connection exception occurred'
    user_err_msg = 'an unexpected internal error occurred'

class ClientException(APIException):
    """Indicates exception that was caused by user, not an internal error"""
    http_status = HTTPStatus.BAD_REQUEST
    internal_err_msg = 'bad request exception occurred'
    user_err_msg = 'server received a bad request'

class NotAuthorizedError(ClientException):
    """User is not authorized to perform requested action on resource"""
    http_status = HTTPStatus.UNAUTHORIZED
    internal_err_msg = 'not authorized exception occurred'
    user_err_msg = 'you are not authorized to perform this request'

class NotFoundError(ClientException):
    """Indicates resource was not found"""
    http_status = HTTPStatus.NOT_FOUND
    internal_err_msg = 'resource not found exception occurred'
    user_err_msg = 'requested resource was not found'

class Account:
    def __init__(self, account_id, account_type):
        self.account_id = account_id
        self.account_type = account_type

    def __repr__(self):
        return f'Account({self.account_id}, {self.account_type})'

def lookup_account_by_id(account_id):
    if not isinstance(account_id, int) or account_id <= 0:
        raise ClientException(f'account number is invalid',
                           f'account_id = {account_id!r}')
    if account_id < 100:
        raise DBConnectionError('permanent failure connecting to database',
                                'db = db001')
    if account_id < 200:
        raise NotAuthorizedError('user does not have permission to read this account',
                                 f'account_id = {account_id}')
    if account_id < 300:
        raise NotFoundError('Account not found',
                            f'account_id = {account_id}')
    return Account(account_id, 'Savings')

def get_account(account_id):
    try:
        account = lookup_account_by_id(account_id)
    except APIException as ex:
        ex.log_exception()
        return ex.to_json()
    else:
        return HTTPStatus.OK, account

get_account('abc')
get_account(50)
get_account(150)
get_account(250)
get_account(350)

2024-02-29T18:49:34.799267+00:00: {'exception_type': 'ClientException', 'http_status': <HTTPStatus.BAD_REQUEST: 400>, 'internal_message': 'account number is invalid', 'user_message': 'server received a bad request', 'args': ('account number is invalid', "account_id = 'abc'")}
2024-02-29T18:49:34.800469+00:00: {'exception_type': 'DBConnectionError', 'http_status': <HTTPStatus.INTERNAL_SERVER_ERROR: 500>, 'internal_message': 'permanent failure connecting to database', 'user_message': 'an unexpected internal error occurred', 'args': ('permanent failure connecting to database', 'db = db001')}
2024-02-29T18:49:34.800469+00:00: {'exception_type': 'NotAuthorizedError', 'http_status': <HTTPStatus.UNAUTHORIZED: 401>, 'internal_message': 'user does not have permission to read this account', 'user_message': 'you are not authorized to perform this request', 'args': ('user does not have permission to read this account', 'account_id = 150')}
2024-02-29T18:49:34.800469+00:00: {'exception_type': 'NotF

(<HTTPStatus.OK: 200>, Account(350, Savings))

### Raising and handling multiple unrelated exceptions

* There are situations where it is necessary to report several exceptions that have occurred.
  * This is often the case in concurrency frameworks, when several tasks may have failed in parallel, but there are also other use cases where it is desirable to continue execution and collect multiple errors rather than raise the first exception.
* The builtin `ExceptionGroup` wraps a list of exception instances so that they can be raised together. It is an exception itself, so it can be caught like any other exception.
* By using `except*` instead of `except`, we can selectively handle only the exceptions in the group that match a certain type, letting all other exceptions propagate to other clauses and eventually to be re-raised.
* Note that the exceptions nested in an exception group must be *instances*, not types. This is because in practice the exceptions would typically be ones that have already been raised and caught by the program.

In [1]:
def f():
    raise ExceptionGroup(
        "group1",
        [
            OSError(1),
            SystemError(2),
            ExceptionGroup(
                "group2",
                [
                    OSError(3),
                    RecursionError(4)
                ]
            )
        ]
    )

try:
    f()
except* OSError as e:
    print("There were OSErrors")
except* SystemError as e:
    print("There were SystemErrors")

There were OSErrors
There were SystemErrors


  + Exception Group Traceback (most recent call last):
  |   File "C:\Users\FREEDEMPIRE-XL\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py", line 3553, in run_code
  |     exec(code_obj, self.user_global_ns, self.user_ns)
  |   File "C:\Users\FREEDEMPIRE-XL\AppData\Local\Temp\ipykernel_68\1308333313.py", line 18, in <module>
  |     f()
  |   File "C:\Users\FREEDEMPIRE-XL\AppData\Local\Temp\ipykernel_68\1308333313.py", line 2, in f
  |     raise ExceptionGroup(
  | ExceptionGroup: group1 (1 sub-exception)
  +-+---------------- 1 ----------------
    | ExceptionGroup: group2 (1 sub-exception)
    +-+---------------- 1 ----------------
      | RecursionError: 4
      +------------------------------------


### Enriching exceptions with notes

* Exceptions have a method `add_note(note)` that accepts a string and adds it to the exception's notes list.
* The standard traceback rendering includes all notes, in the order they were added, after the exception.

In [3]:
try:
    raise TypeError('bad type')
except Exception as e:
    e.add_note('Add some information')
    e.add_note('Add some more information')
    raise

TypeError: bad type

In [4]:
def f():
    raise OSError('operation failed')

excs = []
for i in range(3):
    try:
        f()
    except Exception as e:
        e.add_note(f'Happened in Iteration {i+1}')
        excs.append(e)

raise ExceptionGroup('We have some problems', excs)

  + Exception Group Traceback (most recent call last):
  |   File "C:\Users\FREEDEMPIRE-XL\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py", line 3553, in run_code
  |     exec(code_obj, self.user_global_ns, self.user_ns)
  |   File "C:\Users\FREEDEMPIRE-XL\AppData\Local\Temp\ipykernel_68\1429863547.py", line 12, in <module>
  |     raise ExceptionGroup('We have some problems', excs)
  | ExceptionGroup: We have some problems (3 sub-exceptions)
  +-+---------------- 1 ----------------
    | Traceback (most recent call last):
    |   File "C:\Users\FREEDEMPIRE-XL\AppData\Local\Temp\ipykernel_68\1429863547.py", line 7, in <module>
    |     f()
    |   File "C:\Users\FREEDEMPIRE-XL\AppData\Local\Temp\ipykernel_68\1429863547.py", line 2, in f
    |     raise OSError('operation failed')
    | OSError: operation failed
    | Happened in Iteration 1
    +---------------- 2 ----------------
    | Traceback (most recent call last):
    |   File "C:\Users\FREEDEMP

## Project 6

Suppose we have a Widget online sales application and we are writing the backend for it.

* We want a base `WidgetException` class that we will use as the base class for all our custom exceptions we raise from our Widget application.
* Furthermore we have determined that we will need the following categories of exceptions:
  * Supplier exceptions
    * Not manufactured anymore
    * Production delayed
    * Shipping delayed
  * Checkout exceptions
    * Inventory type exceptions
      * out of stock
    * Pricing exceptions
        * invalid coupon code
        * cannot stack coupons
* Implement the following functionality:
  * implement separate internal error message and user error message
  * implement an http status code associated to each exception type (keep it simple, use a 500 (server error) error for everything except invalid coupon code, and cannot stack coupons, these can be 400 (bad request))
  * implement a logging function that can be called to log the exception details, time it occurred, etc.
  * implement a function that can be called to produce a json string containing the exception details you want to display to your user (include http status code (e.g. 400), the user error message, etc)
* Log the traceback
  * you can use the `TracebackException` class in the `traceback` module
  * In particular, look at the class method `from_exception` and the `format` instance method.

In [1]:
from http import HTTPStatus
import json
from datetime import datetime, timezone
import traceback

class WidgetException(Exception):
    """Base exception for widget application"""
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    internal_err_msg = 'widget exception occurred'
    user_err_msg = 'an unexpected internal error occurred'

    def __repr__(self):
        return f'{type(self).__name__}({self.http_status}, {self.internal_err_msg}, {self.user_err_msg})'

    def log_exception(self, file=None):
        if file is None:
            file = 'log.txt'
        with open(file, 'a+') as f:
            f.write(f'Exception: {datetime.now(timezone.utc)}, {self!r}\n')

            # traceback.print_exception(self, file=f)
            # add indentation to each line of traceback string
            f.writelines(map(lambda row: f'\n    {row[1:]}' if row.startswith('\n') else f'    {row}',
                             traceback.format_exception(self)))
            f.write('\n')

    def to_json(self):
        return json.dumps({
            'exception_type': type(self).__name__,
            'http_status': self.http_status,
            'internal_message': self.internal_err_msg,
            'user_message': self.user_err_msg
        })

class SupplierException(WidgetException):
    """General supplier-end exception"""
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    internal_err_msg = 'supplier exception occurred'
    user_err_msg = 'an unexpected internal error occurred'

class NotManufacturedError(SupplierException):
    """Indicate an error due to manufacturing discontinuation"""
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    internal_err_msg = 'exception caused by manufacturing discontinuation'
    user_err_msg = 'the widget requested is no longer manufactured by supplier'

class ProductionDelayedError(SupplierException):
    """Indicate an error due to production delay"""
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    internal_err_msg = 'error caused by production delay'
    user_err_msg = 'production was delayed by supplier'

class ShippingDelayedError(SupplierException):
    """Indicate an error due to shipping delay"""
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    internal_err_msg = 'error caused by shipping delay'
    user_err_msg = 'shipping was delayed by supplier'

class CheckoutException(WidgetException):
    """General checkout-end exception"""
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    internal_err_msg = 'exception caused by checkout'
    user_err_msg = 'an unexpected internal error occurred'

class InventoryException(CheckoutException):
    """Indicates an exception relating to inventory"""
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    internal_err_msg = 'exception caused by inventory'
    user_err_msg = 'an unexpected internal error occurred'

class OutOfStockError(InventoryException):
    """Indicates an error due to inventory defficiency"""
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    internal_err_msg = 'error caused by stock insufficiency'
    user_err_msg = 'not enough stock for the widget requested'

class PricingException(CheckoutException):
    """Indicates an exception relating to pricing"""
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    internal_err_msg = 'exception caused by pricing'
    user_err_msg = 'an unexpected internal error occurred'

class InvalidCouponCodeError(PricingException):
    """Indicates an error due to invalid coupon code"""
    http_status = HTTPStatus.BAD_REQUEST
    internal_err_msg = 'error caused by invalid coupon code'
    user_err_msg = 'the coupon code provided is invalid'

class NotStackableError(PricingException):
    """Indicates an error due to stacking unsupported coupons"""
    http_status = HTTPStatus.BAD_REQUEST
    internal_err_msg = 'error caused by stacking unsupported coupons'
    user_err_msg = 'the coupons provided cannot be stacked'

try:
    raise PricingException()
except PricingException:
    try:
        raise InvalidCouponCodeError()
    except InvalidCouponCodeError as ex:
        ex.log_exception()
        print(ex.to_json())

try:
    raise OutOfStockError()
except OutOfStockError as ex:
    ex.log_exception()
    print(ex.to_json())

{"exception_type": "InvalidCouponCodeError", "http_status": 400, "internal_message": "error caused by invalid coupon code", "user_message": "the coupon code provided is invalid"}
{"exception_type": "OutOfStockError", "http_status": 500, "internal_message": "error caused by stock insufficiency", "user_message": "not enough stock for the widget requested"}


## Metaprogramming

### Introduction

* Metaprogramming is a programming technique in which computer programs have the ability to treat other programs as their data.
* It means that a program can be designed to read, generate, analyze or transform other programs, and even modify itself while running.
* The basic idea is we can use code to modify code.
* It helps keep code DRY (don't repeat yourself).

Some examples of metaprogramming

* Parsers
  * Programs that analyze and interpret other programs or data formats.
* Interpreters
  * Metaprogramming enables building interpreters for custom languages.
* Compilers
  * Writing compilers involves metaprogramming to translate high-level code into machine code.

Metaprogramming in Python
* Decorators
  * Use code to modify the behavior of another piece of code
  * Decorators themselves can be functions or classes, i.e. function-based decorators and class-based decorators.
  * Function decorators vs class decorators
    * [Function decorators](https://peps.python.org/pep-0318/): decorators for functions and methods, applying transformation to functions or methods
      * Common use cases
        * Logging: add logging statements before and after function execution
        * Timing: measure the execution time of a function
        * Authorization: check user permissions before allowing access to a function
        * Caching: cache function results for performance optimization
    * [Class decorators](https://peps.python.org/pep-3129/): decorators for classes, they are decorators working similarly to function decorators but decorate classes instead of functions or methods
  * Decorator functions vs decorator classes
    * They are different approaches to implement decorators, i.e. function-based or class-based decorators respectively.
    * Function-based decorators: functions that accept another function as an argument and return a new function that incorporates additional functionality
      * Advantages
        * Simplicity: easier to create and use
        * Automatic handling: works seamlessly with methods (functions within classes)
        * Lightweight: minimal overhead
        * Easy to return the original function after modification
    * Class-based decorators: classes that implements the decorator behavior with an `__init__` method to receive the original class (or function) and an `__call__` method to modify its behavior
      * Advantages
        * State management: easier to maintain state (e.g. keeping track of counts, caching)
        * Extensibility: can add methods and properties to the decorated callable object
        * Inheritance: allows creating similar but different decorators
* Descriptors
  * Use code to essentially modify the behavior of the dot operator
* Metaclasses
  * Knowing when to use a metaclass is not easy. Unless you come across a problem where the use of a metaclass is obvious, don't use it.
  * Just because you have a new hammer does not mean everything is a nail.
  * Beware of the "solution in search of a problem" syndrome.
    * solution in search of a problem (*plural* solutions in search of problems):
      * A proposal that does not solve any problem or provide any value; or one that is intended to fill a need which does not really exist.
      * Some teachers complained that the new online grading software was a solution in search of a problem.
  * If you are writing a library or framework, maybe use metaclass; if you are writing application code, probably not.
  * It's still good to understand how metaclasses work.
    * provides deeper insight into Python internal mechanism and how things work
* `type`
  * The statement "everything is an object" isn't entirely true. Here's a better version: Everything in Python is either an instance of a class or an instance of a metaclass, except for `type`.
    * It highlights that most objects in Python fall into one of these categories: instance, class, or metaclass.
    * However, there's a special case with `type`, which serves as both a class and a metaclass.
  * `type` is indeed an object in Python, but it occupies a unique position.
    * Instances: These are objects created from classes.
    * Classes: Classes themselves are instances of metaclasses. When you define a class (e.g. `class MyClass:`), Python implicitly creates an instance of the metaclass (usually `type`) to represent that class.
    * Metaclasses: Metaclasses define the behavior of classes. They are responsible for creating classes. In other words, metaclasses are classes for classes.
    * `type`: `type` is both a class and a metaclass:
      * As a class: When you define a custom class, Python uses `type` as the default metaclass to create that class.
      * As a metaclass: `type` itself is a metaclass that defines how classes (including itself) behave.

### `__new__` method

* `__new__(cls[, ...])`
  * Called to create a new instance of class `cls`.
  * `__new__()` is a *static method* (special-cased so you need not declare it as such) that takes the class of which an instance was requested as its first argument. The remaining arguments are those passed to the object constructor expression (the call to the class). The return value of `__new__()` should be the new object instance (usually an instance of `cls`).
  * Typical implementations create a new instance of the class by invoking the superclass's `__new__()` method using `super().__new__(cls[, ...])` with appropriate arguments and then modifying the newly created instance as necessary before returning it.
  * If `__new__()` is invoked during *object construction* (i.e. calling the class) and it returns an instance of `cls`, then the new instance's `__init__()` method will be invoked like `__init__(self[, ...])`, where `self` is the new instance and the remaining arguments are the same as were passed to the object constructor.
  * If `__new__()` does not return an instance of `cls`, then the new instance's `__init__()` method will not be invoked.
  * `__new__()` is intended mainly to allow subclasses of immutable types (like `int`, `str`, or `tuple`) to customize instance creation. It is also commonly overridden in custom *metaclasses* in order to customize class creation.

In [2]:
class Person:
    def __init__(self, name):
        print('__init__ called')
        self.name = name

    def __new__(cls, name):
        print('__new__ called')
        instance = super().__new__(cls) # object.__new__() takes exactly one argument (the type to instantiate)
        if isinstance(instance, Person): # this process will be done automatically by Python when calling the class
            instance.__init__(name)
        return instance

p = Person.__new__(Person, 'john')
print(p.name)
p = Person('jack')
print(p.name)

__new__ called
__init__ called
john
__new__ called
__init__ called
__init__ called
jack


In [4]:
class Squared(int):
    def __new__(cls, x): # __init__ does not work for int
        if isinstance(x, int):
            return super().__new__(cls, x ** 2)
        raise TypeError('only support int type for instantiation')
    
s = Squared(4)
print(f'{s = }')
print(f'{repr(s) = }')
print(f'{type(s) = }')
print(f'{isinstance(s, int) = }')

s = 16
repr(s) = '16'
type(s) = <class '__main__.Squared'>
isinstance(s, int) = True


In [7]:
class Square:
    def __new__(cls, w, l): # __init__ are not necessary here
        cls.area = lambda self: self.w * self.l
        instance = super().__new__(cls)
        instance.w = w
        instance.l = l
        return instance
    
s = Square(3, 4)
print(s.__dict__)
print(s.area())

{'w': 3, 'l': 4}
12


### How classes are created

The inner mechanics of class creation

1. The class body is extracted (basically just a blob of text, but it is valid code).
2. A new dictionary is created—this will be the namespace of the new class.
3. The body code is executed inside that namespace, thereby populating the namespace.
4. A new `type` instance is created using the name of the class, the base classes, and the populated dictionary.
   * `class type(name, bases, dict, **kwds)`

In [13]:
import math

# class Circle:
#     def __init__(self, radius):
#         self.radius = radius

#     def area(self):
#         return math.pi * self.radius ** 2

#     def __repr__(self):
#         return f'Circle(radius={self.radius})'
    
# use type to create the class above manually

cls_name = 'Circle'
cls_bases = () # an empty tuple indicating the class inherits from object class directly
cls_body = """
def __init__(self, radius):
    self.radius = radius

def area(self):
    return math.pi * self.radius ** 2

def __repr__(self):
    return f'Circle(radius={self.radius})'
"""
cls_dict = {}

exec(cls_body, globals(), cls_dict)

Circle = type(cls_name, cls_bases, cls_dict)

c = Circle(1)
print(Circle)
print(type(Circle))
print(Circle.__name__)
print(c)
print(c.__dict__)
print(c.radius)
print(c.area())

<class '__main__.Circle'>
<class 'type'>
Circle
Circle(radius=1)
{'radius': 1}
1
3.141592653589793


### Inheriting from `type`

* `type` is an object.
  * like any object it inherits from `object`
  * it has `__new__` and `__init__`
* `type` is a class.
  * it is callable
  * calling it creates a new instance of `type`
    * just like calling any class, calls `__new__`
    * `type.__new__(type, name, bases, dict)`
  * we can customize `type` by subclassing it and overriding `__new__`

In [2]:
import math

class CustomType(type): # a subclass of type metaclass, i.e. a metaclass too
    def __new__(cls, name, bases, cls_dict):
        # injecting something to cls_dict
        cls_dict['circumference'] = lambda self: 2 * math.pi * self.radius
        cls_obj = super().__new__(cls, name, bases, cls_dict)
        # or injecting something here to the class instance created
        # cls_obj.circumference = lambda self: 2 * math.pi * self.radius
        return cls_obj
    
cls_name = 'Circle'
cls_bases = ()
cls_body = """
def __init__(self, radius):
    self.radius = radius

def area(self):
    return math.pi * self.radius ** 2

def __repr__(self):
    return f'Circle(radius={self.radius})'
"""
cls_dict = {}

exec(cls_body, globals(), cls_dict)

Circle = CustomType(cls_name, cls_bases, cls_dict)
print(Circle)
print(type(Circle))
c = Circle(1)
print(c)
print(c.radius)
print(c.area())
print(c.circumference())

<class '__main__.Circle'>
<class '__main__.CustomType'>
Circle(radius=1)
1
3.141592653589793
6.283185307179586


### Metaclasses

* Reference: https://docs.python.org/3/reference/datamodel.html#metaclasses
* The class of a class.
* Class definitions create a class name, a class dictionary, and a list of base classes. The metaclass is responsible for taking those three arguments and creating the class.
* Most object oriented programming languages provide a default implementation. What makes Python special is that it is possible to create *custom* metaclasses.
* Metaclasses have been used for logging attribute access, adding thread-safety, tracking object creation, implementing singletons, and many other tasks.
* By default, classes are constructed using `type()`. The class body is executed in a new namespace and the class name is bound locally to the result of `type(name, bases, namespace)`.
* The class creation process can be customized by passing the `metaclass` keyword argument in the class definition line, or by inheriting from an existing class that included such an argument.
* When a class definition is executed, the following steps occur:
  * MRO entries are resolved
  * the appropriate metaclass is determined
  * the class namespace is prepared
  * the class body is executed
  * the class object is created

In [ ]:
class Meta(type):
    pass

class MyClass(metaclass=Meta): # an instance of Meta, unlike normal classes that are instances of type
    pass

class MySubClass(MyClass): # an instance of Meta
    pass

In [8]:
import math

class Meta(type): # a subclass of type metaclass, i.e. a metaclass too
    def __new__(cls, name, bases, cls_dict):
        # injecting something to cls_dict
        cls_dict['circumference'] = lambda self: 2 * math.pi * self.radius
        cls_obj = super().__new__(cls, name, bases, cls_dict)
        # or injecting something here to the class instance created
        # cls_obj.circumference = lambda self: 2 * math.pi * self.radius
        return cls_obj
    
class Circle(metaclass=Meta):
    def __init__(self, radius):
        self.radius = radius

    def __repr__(self):
        return f'Circle(radius={self.radius})'
    
    def area(self):
        return math.pi * self.radius ** 2

print(Circle)
print(type(Circle))
c = Circle(1)
print(c)
print(c.radius)
print(c.area())
print(c.circumference())

<class '__main__.Circle'>
<class '__main__.Meta'>
Circle(radius=1)
1
3.141592653589793
6.283185307179586


### Class decorators

* Class decorators are decorators for classes.
* They are decorators working similarly to function decorators but decorate classes instead of functions or methods.
* A decorator that expects a class as the input and returns the tweaked class.
* Class decorators can be used to create, delete, or modify class attributes, including both plain attributes and methods.
* We can also make class decorators parametrized just like parametrized function decorators, which are actually decorator factories.
* We can use a metaclass instead, but that will be an overkill.

In [4]:
def account_type(type_):
    def decorator(cls):
        cls.account_type = type_
        return cls
    return decorator

@account_type('savings')
class BankAccountSavings:
    pass

@account_type('checking')
class BankAccountChecking:
    pass

print(BankAccountSavings.account_type)
print(BankAccountChecking.account_type)

savings
checking


In [6]:
def hello(cls):
    cls.hello = lambda self: f'{self} says hello'
    return cls

@hello
class Person:
    def __init__(self, name):
        self.name = name

    def __repr__(self):
        return f'Person({self.name})'
    
print(Person('Meta').hello())

Person(Meta) says hello


In [15]:
from functools import wraps

def func_logger(fn):
    @wraps(fn)
    def inner(*args, **kwargs):
        result = fn(*args, **kwargs)
        print(f'Log: {fn.__qualname__}({args}, {kwargs}) = {result}')
        return result
    return inner

def class_logger(cls):
    for name, obj in vars(cls).items():
        if callable(obj) and name != '__repr__':
            print('decorating:', name)
            setattr(cls, name, func_logger(obj))
    return cls

@class_logger
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
    
    def __repr__(self): # need to exclude this method from decorating, because it will be called to get the representation of self, leading to infinite recursion
        return f'Person({self.name}, {self.age})'

    def greet(self):
        return f'Hello, I\'m {self.name}, and I\'m {self.age} years old.'
    
p = Person('John', 18)
print(p)
print(p.greet())

decorating: __init__
decorating: greet
Log: Person.__init__((Person(John, 18), 'John', 18), {}) = None
Person(John, 18)
Log: Person.greet((Person(John, 18),), {}) = Hello, I'm John, and I'm 18 years old.
Hello, I'm John, and I'm 18 years old.


In [13]:
from functools import wraps

def func_logger(fn):
    @wraps(fn)
    def decorator(*args, **kwargs):
        result = fn(*args, **kwargs)
        print(f'Log: {fn.__qualname__}({args}, {kwargs}) = {result}')
        return result
    return decorator

def class_logger(cls):
    for name, obj in vars(cls).items():
        if callable(obj) and name != '__repr__': # if callable is too broad, we can use inspect.isroutine (or isfunction) instead
            print('decorating:', name)
            setattr(cls, name, func_logger(obj))
        elif isinstance(obj, (staticmethod, classmethod)):
            print('decorating:', name)
            setattr(cls, name, obj.__class__((func_logger(obj.__func__))))
        elif isinstance(obj, property):
            print('decorating:', name)
            attrs = ('fget', 'getter'), ('fset', 'setter'), ('fdel', 'deleter')
            for attr, method in attrs:
                if getattr(obj, attr) is not None:
                    obj = getattr(obj, method)(func_logger(getattr(obj, attr)))
            setattr(cls, name, obj)
    return cls

@class_logger
class Person:
    @staticmethod
    def static_method():
        return 'static_method called'

    @classmethod
    def class_method(cls):
        return 'class_method called'

    def __init__(self, name, age):
        self.name = name
        self.age = age

    @property
    def name(self):
        return self._name
    
    @name.setter
    def name(self, name):
        self._name = name

    @property
    def age(self):
        return self._age
    
    @age.setter
    def age(self, age):
        if isinstance(age, int) and age >= 0:
            self._age = age
        else:
            raise ValueError('age must be an non-negative int')
    
    # def __repr__(self): # need to exclude this method from decorating, because it will be called to get the representation of self, leading to infinite recursion
    #     return f'Person({self._name}, {self._age})'

    def greet(self):
        return f'Hello, I\'m {self.name}, and I\'m {self.age} years old.'
    
class Student(Person):
    def study(self):
        return f'Student({self.name}) is studying'
    
print('-' * 40)
p = Person('John', 18)
print('-' * 40)
print(p)
print('-' * 40)
print(p.greet())
print('-' * 40)
print(Person.static_method())
print('-' * 40)
print(Person.class_method())
print('-' * 40)
print(p.name)
print('-' * 40)
p.age = 20
print('-' * 40)
print(p.age)
print('-' * 40)
print('-' * 40)
s = Student('Jack', 12)
print('-' * 40)
print(s.greet())
print('-' * 40)
print(s.study()) # the study method of student is not decorated

decorating: static_method
decorating: class_method
decorating: __init__
decorating: name
decorating: age
decorating: greet
----------------------------------------
Log: Person.name((<__main__.Person object at 0x00000162AB169AF0>, 'John'), {}) = None
Log: Person.age((<__main__.Person object at 0x00000162AB169AF0>, 18), {}) = None
Log: Person.__init__((<__main__.Person object at 0x00000162AB169AF0>, 'John', 18), {}) = None
----------------------------------------
----------------------------------------
Log: Person.name((<__main__.Person object at 0x00000162AB169AF0>,), {}) = John
Log: Person.age((<__main__.Person object at 0x00000162AB169AF0>,), {}) = 18
Log: Person.greet((<__main__.Person object at 0x00000162AB169AF0>,), {}) = Hello, I'm John, and I'm 18 years old.
Hello, I'm John, and I'm 18 years old.
----------------------------------------
Log: Person.static_method((), {}) = static_method called
static_method called
----------------------------------------
Log: Person.class_method(

### Decorator classes

* Decorator classes are classes that can be used as decorators.
* Compare decorator functions and decorator classes
  * calling `my_func` decorated by the decorator function actually calls the `inner` function
  * `my_func` decorated by the decorator class becomes an instance of `Decorator`, calling it actually calls its `__call__` method 
    ```Python
    def decorator(fn):
        def inner(*args, **kwargs):
            return fn(*args, **kwargs)
        return inner

    my_func = decorator(my_func)
    
    class Decorator:
        def __init__(self, fn):
            self.fn = fn

        def __call__(self, *args, **kwargs):
            return self.fn(*args, **kwargs)

    my_func = Decorator(my_func)
    ```

In [7]:
from types import MethodType

class Logger:
    def __init__(self, fn):
        print(f'{fn} decorated with Logger')
        self.fn = fn

    def __call__(self, *args, **kwargs):
        print('__call__ called')
        return self.fn(*args, **kwargs)
    
    def __get__(self, instance, owner): # because methods are actually non-data descriptors, so to correctly decorate methods, we need to make the result of decoration remain a descriptor
        print('__get__ called')
        if instance is None:
            return self
        return MethodType(self, instance) # bound self (the callable instance) to instance

class Person:
    def __init__(self, name):
        self.name = name

    @Logger
    def say_hello(self):
        print(f'{self.name} says hello')

    @classmethod
    @Logger
    def class_method(cls):
        print(f'{cls}\'s class method called')

    @staticmethod
    @Logger
    def static_method():
        print('static method called')

@Logger
def my_func():
    print('my_func called')
    
p = Person('John')
print('-' * 40)
p.say_hello()
print('-' * 40)
p.class_method()
print('-' * 40)
p.static_method()
print('-' * 40)
my_func()

<function Person.say_hello at 0x00000162A9F4F240> decorated with Logger
<function Person.class_method at 0x00000162A9F4DC60> decorated with Logger
<function Person.static_method at 0x00000162A9F4FB00> decorated with Logger
<function my_func at 0x00000162A9F4DEE0> decorated with Logger
----------------------------------------
__get__ called
__call__ called
John says hello
----------------------------------------
__get__ called
__call__ called
<class '__main__.Person'>'s class method called
----------------------------------------
__call__ called
static method called
----------------------------------------
__call__ called
my_func called


### Metaclass vs class decorators

Metaclass|Class decorator
---|---
harder to understand|easier to understand
can only specify a single metaclass|can stack decorators
subclass inherit parent metaclass|subclass do not inherit decorator
only a single metaclass (or a custom metaclass together with the `type` metaclass) can be in play in an inheritance chain|can decorate classes in an inheritance chain with different decorators
offer more control over class creation and behavior|simpler and more limited

In [11]:
# metaclass acting like a class decorator

from functools import wraps

def func_logger(fn):
    @wraps(fn)
    def inner(*args, **kwargs):
        result = fn(*args, **kwargs)
        print(f'Log: {fn.__qualname__}({args}, {kwargs}) = {result}')
        return result
    return inner

class ClassLoggerMeta(type):
    def __new__(mcls, name, bases, dict):
        cls = super().__new__(mcls, name, bases, dict)
        for name, obj in cls.__dict__.items():
            if callable(obj):
                print('decorating:', cls, name)
                setattr(cls, name, func_logger(obj))
        return cls
    
class Person(metaclass=ClassLoggerMeta):
    def __init__(self, name):
        self.name = name

    def greet(self):
        return f'Person({self.name}) says hello'
    
p = Person('John')
print(p.greet())

decorating: <class '__main__.Person'> __init__
decorating: <class '__main__.Person'> greet
Log: Person.__init__((<__main__.Person object at 0x00000162A9F88380>, 'John'), {}) = None
Log: Person.greet((<__main__.Person object at 0x00000162A9F88380>,), {}) = Person(John) says hello
Person(John) says hello


### Metaclass parameters

* We can pass extra parameters to a custom metaclass: `MetaClass.__new__(mcls, name, bases, dict, extra_arg1, extra_arg2, extra_arg3=None)`, since Python 3.6.
* Then we can specify these extra arguments when defining customs classes with the custom metaclass.
* Those extra arguments must be passed as named arguments: `class CustomClass(metaclass=MetaClass, arg1=val1, arg2=val2):`.

In [8]:
class MetaClass(type):
    # def __new__(mcls, name, bases, dict, extra_attrs=None):
    #     if extra_attrs is not None:
    #         print('Creating class with extra attributes:', extra_attrs)
    #         for attr_name, attr_value in extra_attrs:
    #             dict[attr_name] = attr_value
    #     return super().__new__(mcls, name, bases, dict)
    
    def __new__(mcls, name, bases, dict, **extra_attrs):
        # dict.update(extra_attrs)
        dict |= extra_attrs
        return super().__new__(mcls, name, bases, dict)

# class Account(metaclass=MetaClass, extra_attrs=(('account_type', 'savings'), ('apr', 0.5))):
#     pass
class Account(metaclass=MetaClass, account_type='savings', apr=0.5):
    pass

print(vars(Account))
print(Account.__dict__) # same as above


{'__module__': '__main__', 'account_type': 'savings', 'apr': 0.5, '__dict__': <attribute '__dict__' of 'Account' objects>, '__weakref__': <attribute '__weakref__' of 'Account' objects>, '__doc__': None}
{'__module__': '__main__', 'account_type': 'savings', 'apr': 0.5, '__dict__': <attribute '__dict__' of 'Account' objects>, '__weakref__': <attribute '__weakref__' of 'Account' objects>, '__doc__': None}


### `__prepare__` method

* In a metaclass, when the metaclass is called (via `class MyClass(metaclass=MetaClass):`)
  * Python determines and passes to the `__new__` method of the metaclass:
    * the metaclass used to create the class
    * the name of the class to be created
    * the base classes
    * a dictionary used as the class namespace
* Once the appropriate metaclass has been identified, then the class namespace is prepared.
  * `__prepare__` method of the metaclass which is implemented by `type` actually creates the namespace for the class to be created
    * If the metaclass has a `__prepare__` attribute, it is called as `namespace = metaclass.__prepare__(name, bases, **kwds)` (where the additional keyword arguments, if any, come from the class definition).
      * The `__prepare__` method should be implemented as a *classmethod*. The namespace returned by `__prepare__` is passed in to `__new__`, but when the final class object is created the namespace is copied into a new dict.
      * The return value of `__prepare__` must be a mapping.
  * If the metaclass has no `__prepare__` attribute, then the class namespace is initialized as an empty ordered mapping.

In [10]:
class Meta(type):
    def __new__(mcls, name, bases, cls_dict, **kwargs):
        print('Meta.__new__ called...')
        print('\tmcls', mcls, type(mcls))
        print('\tname', name, type(name))
        print('\tbases', bases, type(bases))
        print('\tcls_dict', cls_dict, type(cls_dict))
        print('\tkwargs', kwargs)
        return super().__new__(mcls, name, bases, cls_dict)
    
class CustomClass(metaclass=Meta, extra1='val1', extra2='val2'):
    pass

Meta.__new__ called...
	mcls <class '__main__.Meta'> <class 'type'>
	name CustomClass <class 'str'>
	bases () <class 'tuple'>
	cls_dict {'__module__': '__main__', '__qualname__': 'CustomClass'} <class 'dict'>
	kwargs {'extra1': 'val1', 'extra2': 'val2'}


In [2]:
class Meta(type):
    def __prepare__(name, bases, **kwargs):
        print('Meta.__prepare__ called...')
        print('\tname', name, type(name))
        print('\tbases', bases, type(bases))
        print('\tkwargs', kwargs)
        # return {**kwargs}
        return type.__prepare__(**kwargs)

    def __new__(mcls, name, bases, cls_dict, **kwargs):
        print('Meta.__new__ called...')
        print('\tmcls', mcls, type(mcls))
        print('\tname', name, type(name))
        print('\tbases', bases, type(bases))
        print('\tcls_dict', cls_dict, type(cls_dict))
        print('\tkwargs', kwargs)
        return super().__new__(mcls, name, bases, cls_dict)
    
class CustomClass(metaclass=Meta, extra1='val1', extra2='val2'):
    pass

Meta.__prepare__ called...
	name CustomClass <class 'str'>
	bases () <class 'tuple'>
	kwargs {'extra1': 'val1', 'extra2': 'val2'}
Meta.__new__ called...
	mcls <class '__main__.Meta'> <class 'type'>
	name CustomClass <class 'str'>
	bases () <class 'tuple'>
	cls_dict {'__module__': '__main__', '__qualname__': 'CustomClass'} <class 'dict'>
	kwargs {'extra1': 'val1', 'extra2': 'val2'}


### Classes, metaclasses, and `__call__`

* In declarative creation of classes, Python doing the following steps:
  * extracts name
  * extracts bases
  * creates a class dictionary, by calling `__prepare__`
  * executes the class body within that class dictionary
  * calls the metaclass to create the class
  * assign the created class to the symbol (name) in the scope
* In programmatic creation of classes, we reproduce the above process manually:
  * define name, bases
  * create an initial class dictionary
  * calculate and add `__qualname__`, `__doc__` to the class dictionary
  * execute the code in the context of the class dictionary
  * call metaclass to create the class
* When we make instances of a custom class callable, we implement `__call__` method in it.
  * `__call__(self[, args...])`
    * Called when the instance is "called" as a function; if this method is defined, `x(arg1, arg2, ...)` roughly translates to `type(x).__call__(x, arg1, ...)`.
* But the class itself is callable, so the class created the custom class, i.e. its metaclass (usually `type`), must implement `__call__`.
  * `type` is a class that implements a `__call__` method.
  * The `__call__` method is called when its instance, i.e. a class, is called as a method bound to the class instance.
    * `__call__` calls `__new__` of the class to create a new instance of the class instance
    * then calls `__init__` of the class instance bound to the new instance created by `__new__`
    * returns the new initialized instance of the class instance
* `type`'s metaclass is itself.
  * the `__call__` of `type` is also called when `type` is called.
    * calls `type.__new__` to create a new class instance
    * then calls `__init__` bound to the new class instance
    * returns the class instance

### Metaprogramming application 1

In [1]:
class PointMeta(type):
    def __new__(mcls, name, bases, cls_dict, slots): # slots are passed in as a tuple of underscore prefixed attribute names
        cls = super().__new__(mcls, name, bases, cls_dict) # this is buggy, as the cls has already been created before its __slots__ being set, causing __slots__ meaningless
        if slots is not None:
            cls.__slots__ = slots
            slots_unprefixed = tuple(map(lambda s: s.removeprefix('_'), slots))
            for property_name, slot_name in zip(slots_unprefixed, slots):
                setattr(cls, property_name, property(lambda self, slot=slot_name: getattr(self, slot),
                                                     lambda self, value, slot=slot_name: setattr(self, slot, value)))

        # __eq__ for class instance  
        def dunder_eq(self, other):
            if isinstance(other, cls):
                # method 1
                # return all(map(lambda m: getattr(self, m) == getattr(other, m), vars(self))) # vars() still working, because __slots__ not working

                # method 2
                # for member in vars(self):
                #     if getattr(self, member) != getattr(other, member):
                #         return False
                # return True
            
                # method 3
                # print(vars(self).values(), type(vars(self).values())) # dict_values type
                return tuple(vars(self).values()) == tuple(vars(other).values())
            return False
        
        # __hash__
        def dunder_hash(self):
            # return hash(tuple(map(lambda m: getattr(self, m), vars(self))))
            return hash(tuple(vars(self).values()))
        
        # __repr__
        def dunder_repr(self):
            # method 1
            # return f'{type(self).__name__}{tuple(vars(self).values())}'

            # method 2
            # attr_strings = []
            # for key, value in vars(self).items():
            #     attr_strings.append(f'{key.removeprefix('_')}={value}')

            # method 3
            attr_strings = [f'{key.removeprefix('_')}={value}' for key, value in vars(self).items()]
            return f'{type(self).__name__}({', '.join(attr_strings)})'
        
        # __str__
        def dunder_str(self):
            return f'{type(self).__name__}{tuple(vars(self).values())}'
        
        setattr(cls, '__eq__', dunder_eq)
        setattr(cls, '__hash__', dunder_hash)
        setattr(cls, '__repr__', dunder_repr)
        setattr(cls, '__str__', dunder_str)
        return cls
        
class Point2D(metaclass=PointMeta, slots=('_x', '_y')):
    def __init__(self, x, y):
        self.x = x
        self.y = y

# class Point2D(metaclass=PointMeta, slots=None): # the metaclass still works with slots=None
#     def __init__(self, x, y):
#         self.x = x
#         self.y = y
        
class Point3D(metaclass=PointMeta, slots=('_x', '_y', '_z')):
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z

p1 = Point2D(0, 0)
p2 = Point2D(1, 0)
p3 = Point2D(0, 0)
p4 = Point3D(1, 2, 3)

print(vars(p1))
print(p1)
print(repr(p1))
print(p1.x)
print(p1.y)
p1.x = 1
print(p1)
print(p1 == p2)
print(p1 == p3)
print(p1 == (1, 0))
print(hash(p1) == hash((1, 0)))
print(p4)
print(repr(p4))
print(p1 == p4)
print(hash(p4) == hash((1, 2, 3)))

{'_x': 0, '_y': 0}
Point2D(0, 0)
Point2D(x=0, y=0)
0
0
Point2D(1, 0)
True
False
False
True
Point3D(1, 2, 3)
Point3D(x=1, y=2, z=3)
False
True


In [3]:
class PointMeta(type):
    def __new__(mcls, name, bases, cls_dict, slots): # slots are passed in as a tuple of underscore prefixed attribute names
        cls_dict.update(__slots__=slots) # add __slots__ to cls_dict before the creation of cls
        cls = super().__new__(mcls, name, bases, cls_dict)
        slots_unprefixed = tuple(map(lambda s: s.removeprefix('_'), slots))
        for property_name, slot_name in zip(slots_unprefixed, slots):
            setattr(cls, property_name, property(lambda self, slot=slot_name: getattr(self, slot),
                                                 lambda self, value, slot=slot_name: setattr(self, slot, value)))

        # __eq__ for class instance  
        def dunder_eq(self, other):
            if isinstance(other, cls):
                for member in self.__slots__:
                    if getattr(self, member) != getattr(other, member):
                        return False
                return True
            return False
        
        # __hash__
        def dunder_hash(self):
            return hash(tuple(map(lambda m: getattr(self, m), self.__slots__)))
        
        # __repr__
        def dunder_repr(self):
            attr_strings = [f'{member.removeprefix('_')}={getattr(self, member)}' for member in self.__slots__]
            return f'{type(self).__name__}({', '.join(attr_strings)})'
        
        # __str__
        def dunder_str(self):
            return f'{type(self).__name__}{tuple(map(lambda m: getattr(self, m), self.__slots__))}'
        
        setattr(cls, '__eq__', dunder_eq)
        setattr(cls, '__hash__', dunder_hash)
        setattr(cls, '__repr__', dunder_repr)
        setattr(cls, '__str__', dunder_str)
        return cls
        
class Point2D(metaclass=PointMeta, slots=('_x', '_y')):
    def __init__(self, x, y):
        self.x = x
        self.y = y

class Point3D(metaclass=PointMeta, slots=('_x', '_y', '_z')):
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z

p1 = Point2D(0, 0)
p2 = Point2D(1, 0)
p3 = Point2D(0, 0)
p4 = Point3D(1, 2, 3)

print(Point2D.__slots__)
print(p1)
print(repr(p1))
print(p1.x, p1.y)
p1.x = 1
print(p1)
print(p1 == p2)
print(p1 == p3)
print(p1 == (1, 0))
print(hash(p1) == hash((1, 0)))
print(p4)
print(repr(p4))
print(p1 == p4)
print(hash(p4) == hash((1, 2, 3)))

('_x', '_y')
Point2D(0, 0)
Point2D(x=0, y=0)
0 0
Point2D(1, 0)
True
False
False
True
Point3D(1, 2, 3)
Point3D(x=1, y=2, z=3)
False
True


In [4]:
class PointMeta(type):
    def __new__(mcls, name, bases, cls_dict, slots): # slots are passed in as a tuple of underscore prefixed attribute names
        cls_dict.update(__slots__=slots) # add __slots__ to cls_dict before the creation of cls
        cls = super().__new__(mcls, name, bases, cls_dict)
        slots_unprefixed = tuple(map(lambda s: s.removeprefix('_'), slots))
        for property_name, slot_name in zip(slots_unprefixed, slots):
            setattr(cls, property_name, property(lambda self, slot=slot_name: getattr(self, slot),
                                                 lambda self, value, slot=slot_name: setattr(self, slot, value)))

        # __eq__ for class instance  
        def dunder_eq(self, other):
            if isinstance(other, cls):
                for member in self.__slots__:
                    if getattr(self, member) != getattr(other, member):
                        return False
                return True
            return False
        
        # __hash__
        def dunder_hash(self):
            return hash(tuple(map(lambda m: getattr(self, m), self.__slots__)))
        
        # __repr__
        def dunder_repr(self):
            attr_strings = [f'{member.removeprefix('_')}={getattr(self, member)}' for member in self.__slots__]
            return f'{type(self).__name__}({', '.join(attr_strings)})'
        
        # __str__
        def dunder_str(self):
            return f'{type(self).__name__}{tuple(map(lambda m: getattr(self, m), self.__slots__))}'
        
        setattr(cls, '__eq__', dunder_eq)
        setattr(cls, '__hash__', dunder_hash)
        setattr(cls, '__repr__', dunder_repr)
        setattr(cls, '__str__', dunder_str)
        return cls

def point_meta_struct(slots): # parameterized decorator
    def inner(cls):
        return PointMeta(cls.__name__, cls.__bases__, dict(cls.__dict__), slots=slots)
    return inner

@point_meta_struct(slots=('_x', '_y'))
class Point2D():
    def __init__(self, x, y):
        self.x = x
        self.y = y

@point_meta_struct(slots=('_x', '_y', '_z'))
class Point3D():
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z

p1 = Point2D(0, 0)
p2 = Point2D(1, 0)
p3 = Point2D(0, 0)
p4 = Point3D(1, 2, 3)

print(Point2D.__slots__)
print(p1)
print(repr(p1))
print(p1.x, p1.y)
p1.x = 1
print(p1)
print(p1 == p2)
print(p1 == p3)
print(p1 == (1, 0))
print(hash(p1) == hash((1, 0)))
print(p4)
print(repr(p4))
print(p1 == p4)
print(hash(p4) == hash((1, 2, 3)))

('_x', '_y')
Point2D(0, 0)
Point2D(x=0, y=0)
0 0
Point2D(1, 0)
True
False
False
True
Point3D(1, 2, 3)
Point3D(x=1, y=2, z=3)
False
True


### Metaprogramming application 2

In [5]:
class Singleton100:
    singleton = None

    def __new__(cls):
        if cls.singleton is None: # because __new__ is a static method, must use cls. to reference singleton attribute
            instance = super().__new__(cls)
            setattr(instance, 'name', 'hundred')
            setattr(instance, 'value', 100)
            cls.singleton = instance
        return cls.singleton
    
s1 = Singleton100()
s2 = Singleton100()
print(s1.name, s1.value)
print(s1 is s2)

hundred 100
True


In [9]:
class SingletonMeta(type):
    def __call__(cls_instance): # gets called when cls_instance is called
        if getattr(cls_instance, 'singleton', None) is None:
            cls_instance.singleton = super().__call__() # because __call__ is an instance method, the instance, i.e. cls_instance, is passed into super().__call__(), so the result is an instance of cls_instance
        return cls_instance.singleton

class Hundred(metaclass=SingletonMeta):
    def __init__(self):
        self.name = 'hundred'
        self.value = 100

class Thousand(Hundred):
    def __init__(self):
        self.name = 'thousand'
        self.value = 1000

h1 = Hundred()
h2 = Hundred()
print(h1.name, h1.value)
print(h1 is h2)

th1 = Thousand()
print(th1.name, th1.value) # does not work correctly in inheritance

hundred 100
True
hundred 100


In [11]:
class SingletonMeta(type):
    def __call__(cls_instance): # gets called when cls_instance is called
        if 'singleton' not in cls_instance.__dict__: # checking singleton attribute in cls_instance.__dict__ directly, avoiding referencing from base classes
            cls_instance.singleton = super().__call__() # because __call__ is an instance method, the instance, i.e. cls_instance, is passed into super().__call__(), so the result is an instance of cls_instance
        return cls_instance.singleton

class Hundred(metaclass=SingletonMeta):
    def __init__(self):
        self.name = 'hundred'
        self.value = 100

class Thousand(Hundred):
    def __init__(self):
        self.name = 'thousand'
        self.value = 1000

class Ten(metaclass=SingletonMeta):
    def __init__(self):
        self.name = 'ten'
        self.value = 10

h1 = Hundred()
h2 = Hundred()
print(h1.name, h1.value)
print(h1 is h2)

th1 = Thousand()
th2 = Thousand()
print(th1.name, th1.value) # works correctly in inheritance now
print(th1 is th2)

t1 = Ten()
t2 = Ten()
print(t1.name, t1.value)
print(t1 is t2)

hundred 100
True
thousand 1000
True
ten 10
True


### Metaprogramming application 3

In [1]:
with open('prod.ini', 'w') as prod, open('dev.ini', 'w') as dev:
    prod.write('[Database]\n')
    prod.write('db_host=prod.network.com\n')
    prod.write('db_name=prod_db\n')
    prod.write('\n[Server]\n')
    prod.write('port=8080\n')

    dev.write('[Database]\n')
    dev.write('db_host=dev.network.com\n')
    dev.write('db_name=dev_db\n')
    dev.write('\n[Server]\n')
    dev.write('port=3000\n')

In [51]:
import configparser

class Config:
    def __init__(self, env='dev'):
        filename = f'{env}.ini'
        print(f'loading config from {filename} file')
        config = configparser.ConfigParser()
        config.read(filename)

        # manually setting the attributes
        # self.db_host = config['Database']['db_host']
        # self.db_name = config['Database']['db_name']
        # self.port = config['Server']['port']

        # setting the attributes programmatically
        for section in config.sections():
            for key, value in config[section].items():
                setattr(self, key, value)

config = Config()
print(config.__dict__)

loading config from dev.ini file
{'db_host': 'dev.network.com', 'db_name': 'dev_db', 'port': '3000'}


In [52]:
import configparser

class Config:
    def __init__(self, env='dev'):
        filename = f'{env}.ini'
        config = configparser.ConfigParser()
        config.read(filename)

        self.sections = config.sections()
        # setting the attributes using Section class
        for section in self.sections:
            setattr(self, section, Section(config[section]))

class Section:
    def __init__(self, section):
        for key, value in section.items():
            setattr(self, key, value)

config = Config()
print(config.__dict__)
print(config.sections)
print(config.Server.port)

{'sections': ['Database', 'Server'], 'Database': <__main__.Section object at 0x000001C1DB4A9910>, 'Server': <__main__.Section object at 0x000001C1D9CD20F0>}
['Database', 'Server']
3000


In [53]:
import configparser

class Config:
    def __init__(self, env='dev'):
        filename = f'{env}.ini'
        config = configparser.ConfigParser()
        config.read(filename)

        self.sections = config.sections()
        # setting the attributes using Section class
        for section in self.sections:
            SectionClass = SectionMeta(f'{section.capitalize()}Section', (), {},
                                       section_name=section.lower(), section_dict=config[section])
            setattr(self, section.lower(), SectionClass) # no need to instantiate SectionClass, because attributes are set on class

class SectionMeta(type):
    def __new__(mcls, name, bases, cls_dict, section_name, section_dict):
        cls_dict['__doc__'] = f'configs for {section_name} section'
        cls_dict['section_name'] = section_name
        for key, value in section_dict.items():
            cls_dict[key] = value
        return super().__new__(mcls, name, bases, cls_dict)

# two ways of create instance classes of SectionMeta
    
# class DatabaseSection(metaclass=SectionMeta, section_name='Database',
#                       section_dict={'db_host': 'host', 'db_name': 'name'}):
#     pass
    
# DatabaseSection = SectionMeta('DatabaseSection', (), {}, section_name='Database',
#                               section_dict={'db_host': 'host', 'db_name': 'name'})

config = Config()
print(config.__dict__)
print(config.sections)
print(config.database.db_name)
print(config.server.port)
print(config.server.section_name)

help(config.server)

{'sections': ['Database', 'Server'], 'database': <class '__main__.DatabaseSection'>, 'server': <class '__main__.ServerSection'>}
['Database', 'Server']
dev_db
3000
server
Help on class ServerSection in module __main__:

class ServerSection(builtins.object)
 |  configs for server section
 |
 |  Data descriptors defined here:
 |
 |  __dict__
 |      dictionary for instance variables
 |
 |  __weakref__
 |      list of weak references to the object
 |
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |
 |  port = '3000'
 |
 |  section_name = 'server'



In [32]:
import configparser

class ConfigMeta(type):
    def __new__(mcls, name, bases, cls_dict, env):
        filename = f'{env}.ini'
        config = configparser.ConfigParser()
        config.read(filename)

        cls_dict['__doc__'] = f'configs for {env} environment'
        cls_dict['env_name'] = env
        cls_dict['sections'] = config.sections()

        for section in cls_dict['sections']:
            SectionClass = SectionMeta(f'{section.capitalize()}Section', (), {},
                                       section_name=section.lower(), section_dict=config[section])
            cls_dict[section.lower()] = SectionClass
        return super().__new__(mcls, name, bases, cls_dict)

class SectionMeta(type):
    def __new__(mcls, name, bases, cls_dict, section_name, section_dict):
        cls_dict['__doc__'] = f'configs for {section_name} section'
        cls_dict['section_name'] = section_name
        for key, value in section_dict.items():
            cls_dict[key] = value
        return super().__new__(mcls, name, bases, cls_dict)

class DevConfig(metaclass=ConfigMeta, env='dev'):
    pass

class ProdConfig(metaclass=ConfigMeta, env='prod'):
    pass

print(DevConfig.__dict__)
print(DevConfig.sections)
print(DevConfig.database.db_name)
print(DevConfig.server.port)
print(DevConfig.server.section_name)
print()

help(DevConfig)
help(DevConfig.server)

{'__module__': '__main__', '__doc__': 'configs for dev environment', 'env_name': 'dev', 'sections': ['Database', 'Server'], 'database': <class '__main__.DatabaseSection'>, 'server': <class '__main__.ServerSection'>, '__dict__': <attribute '__dict__' of 'DevConfig' objects>, '__weakref__': <attribute '__weakref__' of 'DevConfig' objects>}
['Database', 'Server']
dev_db
3000
server

Help on class DevConfig in module __main__:

class DevConfig(builtins.object)
 |  configs for dev environment
 |
 |  Data descriptors defined here:
 |
 |  __dict__
 |      dictionary for instance variables
 |
 |  __weakref__
 |      list of weak references to the object
 |
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |
 |  database = <class '__main__.DatabaseSection'>
 |      configs for database section
 |
 |
 |  env_name = 'dev'
 |
 |  sections = ['Database', 'Server']
 |
 |  server = <class '__main__.ServerSection'>
 |      configs f

### Attribute read accessor

* In Python, attribute values of an object could be living in any number of places:
  * instance dictionary
  * descriptor class attribute
  * plain class attribute
  * in a parent class
* `__getattribute__(self, name)`
  * Called unconditionally to implement attribute accesses for instances of the class, i.e. when an attribute is accessed on an instance of a class it is invoked.
  * If the class also defines `__getattr__()`, the latter will not be called unless `__getattribute__()` either calls it explicitly or raises an `AttributeError`.
  * This method should return the (computed) attribute value or raise an `AttributeError` exception.
  * In order to avoid infinite recursion in this method, its implementation should always call the base class method with the same name to access any attributes it needs, for example, `object.__getattribute__(self, name)`.
  * This method may still be bypassed when looking up special methods as the result of implicit invocation via language syntax or built-in functions.
  * For certain sensitive attribute accesses, raises an auditing event `object.__getattr__` with arguments obj and name.
  * It is very easy to get into infinite recursion when overriding `__getattribute__`.
    * use `super().__getattribute__` to bypass your own overrides
* `__getattr__(self, name)`
  * Called when the default attribute access fails with an `AttributeError` (either `__getattribute__()` raises an `AttributeError` because `name` is not an instance attribute or an attribute in the class tree for `self`; or `__get__()` of a `name` property raises `AttributeError`).
  * This method should either return the (computed) attribute value or raise an `AttributeError` exception.
  * `object` does not have attribute `__getattr__` by default.
  * If the attribute is found through the normal mechanism, `__getattr__()` is not called.
    * This is an intentional asymmetry between `__getattr__()` and `__setattr__()`.
    * This is done both for efficiency reasons and because otherwise `__getattr__()` would have no way to access other attributes of the instance.
  * At least for instance variables, you can fake total control by not inserting any values in the instance attribute dictionary (but instead inserting them in another object).
* Default attribute lookup flow:
  * `obj.attr`
    * `obj.__getattribute__('attr')` (overridable, often using delegation to `super`)
    * in class (include parents) dict?
      * yes
        * data descriptor?
          * yes -> `__get__`
          * no
            * in instance dict?
              * yes -> return it
              * no
                * non-data descriptor?
                  * yes -> `__get__`
                  * no -> return class attr
      * no
        * in instance dict?
          * yes -> return it
          * no -> `AttributeError`
            * `__getattr__` -> `AttributeError` (default implementation, overridable)
* To override access for class attributes, override `__getattribute__` and `__getattr__` in the metaclass, since classes are instances of metaclasses.

In [35]:
class Person:
    def __getattr__(self, name):
        print(f'__getattribute__ did not find attribute {name!r}')
        return f'attribute {name!r} not found'
    
p = Person()
print(p.name)

__getattribute__ did not find attribute 'name'
attribute 'name' not found


In [40]:
class Person:
    def __init__(self, name):
        self._name = name

    def __getattr__(self, name):
        alt_name = f'_{name}'
        try:
            return super().__getattribute__(alt_name) # the base class's __getattribute__ does not call __getattr__ in its subclass directly, ensuring no recursion
        except AttributeError:
            raise AttributeError(f'could not find attribute {name!r} or {alt_name!r}') from None
    
p = Person('John')
print(p.name)
print(p.age)

John


AttributeError: could not find attribute 'age' or '_age'

In [44]:
class DefaultAttributeClass:
    def __init__(self, default_value=None):
        self._default_value = default_value

    def __getattr__(self, name):
        setattr(self, name, self._default_value)
        return getattr(self, name)

dac = DefaultAttributeClass()
print(dac.name)
print(dac.age)
print(dac.__dict__)

class Person(DefaultAttributeClass):
    def __init__(self, name, age, default_value):
        super().__init__(default_value)
        self.name = name
        self.age = age

p = Person('John', 'Smith', 'n/a')
print(p.name, p.age, p.profession)
print(p.__dict__)

None
None
{'_default_value': None, 'name': None, 'age': None}
John Smith n/a
{'_default_value': 'n/a', 'name': 'John', 'age': 'Smith', 'profession': 'n/a'}


In [45]:
class AttributeNotFoundLogger:
    def __getattr__(self, name):
        err_msg = f'{self.__class__.__name__} object does not have attribute {name!r}'
        print(f'log: {err_msg}')
        raise AttributeError(err_msg)
    
class Person(AttributeNotFoundLogger):
    def __init__(self, name, age):
        self.name = name
        self.age = age

p = Person('John', 18)
try:
    p.hobby
except AttributeError as ex:
    print('AttributeError:', ex)

log: Person object does not have attribute 'hobby'
AttributeError: Person object does not have attribute 'hobby'


In [47]:
class Person:
    def __init__(self, name, age):
        self._name = name
        self._age = age

    def __getattribute__(self, name):
        if name.startswith('_'):
            raise AttributeError(f'cannot read private attribute {name!r}')
        return super().__getattribute__(name)
    
    @property
    def name(self): # provide access to attribute '_name' by using 'name'
        return super().__getattribute__('_name')
    
p = Person('John', 18)
print(p.name)
try:
    p._age
except AttributeError as ex:
    print('AttributeError:', ex)

John
AttributeError: cannot read private attribute '_age'


In [48]:
class ClassAttributeLoggerMeta(type):
    def __getattribute__(self, name):
        print(f'class attribute {name!r} accessed')
        return super().__getattribute__(name) # here super().__getattribute__ still calls __getattr__ from subclass if AttributeError occurs
    
    def __getattr__(self, name):
        print(f'class attribute {name!r} not found')
        return 'n/a'
    
class Account(metaclass=ClassAttributeLoggerMeta):
    apr = 4

print(Account.apr)
print(Account.name)

class attribute 'apr' accessed
4
class attribute 'name' accessed
class attribute 'name' not found
n/a


In [49]:
class MyClass:
    def __getattribute__(self, name):
        print(f'__getattribute__ called for attribute {name!r}')
        return super().__getattribute__(name)
    
    def __getattr__(self, name):
        print(f'__getattr__ called for attribute {name!r}')
        raise AttributeError(f'attribute {name!r} does not exist')
    
    def say_hello(self):
        print('Hello!')

mc = MyClass()
mc.say_hello()
try:
    mc.say_hi()
except AttributeError as ex:
    print('AttributeError:', ex)

__getattribute__ called for attribute 'say_hello'
Hello!
__getattribute__ called for attribute 'say_hi'
__getattr__ called for attribute 'say_hi'
AttributeError: attribute 'say_hi' does not exist


### Attribute write accessor

* `__setattr__(self, name, value)`
  * Called when an attribute assignment is attempted. This is called instead of the normal mechanism (i.e. store the value in the instance dictionary). `name` is the attribute name, `value` is the value to be assigned to it.
  * If `__setattr__()` wants to assign to an instance attribute, it should call the base class method with the same name, for example, `object.__setattr__(self, name, value)`.
  * For certain sensitive attribute assignments, raises an auditing event `object.__setattr__` with arguments `obj`, `name`, `value`.
* There's no `__setattribute__` like with the attribute read accessors.
* If setting an attribute that does not exist, it is created in the object's `__dict__`, unless there'e no `__dict__` in which case an exception is raised.
* Default attribute setter flow:
  * `obj.attr = value`
    * `obj.__setattr__('attr', value)`
      * in class (include parent) dict?
        * yes
          * data descriptor?
            * yes -> `__set__`
            * no
              * `obj.__dict__` available?
                * yes -> insert / update it
                * no -> `AttributeError`
        * no
          * `obj.__dict__` available?
            * yes -> insert / update it
            * no -> `AttributeError`

In [54]:
class Person:
    def __setattr__(self, name, value):
        print(f'__setattr__ called for attribute {name!r}')
        return super().__setattr__(name, value)
    
p = Person()
p.name = 'John'
Person.cls_attr = 'cls_attr' # __setattr__ not called for setting class attribute, because the attribute is not in the class's __dict__

__setattr__ called for attribute 'name'


In [55]:
class Meta(type):
    def __setattr__(self, name, value):
        print(f'__setattr__ called for attribute {name}')
        return super().__setattr__(name, value)
    
class Person(metaclass=Meta):
    pass

p = Person()
p.name = 'John'
Person.cls_attr = 'cls_attr' # __setattr__ called for class attribute

__setattr__ called for attribute cls_attr


In [58]:
class NonDataDescriptor:
    def __get__(self, instance, owner=None):
        print('__get__ called on non-data descriptor')

class DataDescriptor:
    def __get__(self, instance, owner=None):
        print('__get__ called on data descriptor')

    def __set__(self, instance, value):
        print('__set__ called on data descriptor')

class MyClass:
    non_data_descriptor = NonDataDescriptor()
    data_descriptor = DataDescriptor()

    def __setattr__(self, name, value):
        print(f'__setattr__ called for attribute {name}')
        super().__setattr__(name, value)
    
mc = MyClass()
mc.non_data_descriptor = 'non data descriptor'
mc.data_descriptor = 'data descriptor'
print(mc.__dict__)
print(MyClass.__dict__)

__setattr__ called for attribute non_data_descriptor
__setattr__ called for attribute data_descriptor
__set__ called on data descriptor
{'non_data_descriptor': 'non data descriptor'}
{'__module__': '__main__', 'non_data_descriptor': <__main__.NonDataDescriptor object at 0x000001C1DB4F7E00>, 'data_descriptor': <__main__.DataDescriptor object at 0x000001C1DB4F69F0>, '__setattr__': <function MyClass.__setattr__ at 0x000001C1DB92B7E0>, '__dict__': <attribute '__dict__' of 'MyClass' objects>, '__weakref__': <attribute '__weakref__' of 'MyClass' objects>, '__doc__': None}


### Accessors application

* A useful application of `__getattr__` and `__setattr__` is dealing with objects where we may not know the attributes in advance.
* Consider this scenario where we have a database with various tables and fields. We want to create a class that allows us to retrieve data from these tables.

In [68]:
# dictionary for simulating database
DB = {
    'Person': {
        1: {'first_name': 'Isaac', 'last_name': 'Newton', 'born': 1642, 'country_id': 1},
        2: {'first_name': 'Gottfried', 'last_name': 'von Leibniz', 'born': 1646, 'country_id': 5},
        3: {'first_name': 'Joseph', 'last_name': 'Fourier', 'born': 1768, 'country_id': 3},
        4: {'first_name': 'Bernhard', 'last_name': 'Riemann', 'born': 1826, 'country_id': 5},
        5: {'first_name': 'David', 'last_name': 'Hilbert', 'born': 1862 , 'country_id': 5},
        6: {'first_name': 'Srinivasa', 'last_name': 'Ramanujan', 'born': 1887, 'country_id': 4},
        7: {'first_name': 'John', 'last_name': 'von Neumann', 'born': 1903, 'country_id': 2},
        8: {'first_name': 'Andrew', 'last_name': 'Wiles', 'born': 1928, 'country_id': 6}
    },
    'Country': {
        1: {'name': 'United Kingdom', 'capital': 'London', 'continent': 'Europe'},
        2 :{'name': 'Hungary', 'capital': 'Budapest', 'continent': 'Europe'},
        3: {'name': 'France', 'capital': 'Paris', 'continent': 'Europe'},
        4: {'name': 'India', 'capital': 'New Delhi', 'continent': 'Asia'},
        5: {'name': 'Germany', 'capital': 'Berlin', 'continent': 'Europe'},
        6: {'name': 'USA', 'capital': 'Washington DC', 'continent': 'North America'}
        }
}

class DatabaseRecord:
    def __init__(self, record_dict):
        super().__setattr__('_record', record_dict)

    def __getattr__(self, name):
        record = super().__getattribute__('_record')
        if name in record:
            return record[name]
        raise AttributeError(f'field name {name!r} does not exist')
    
    def __setattr__(self, name, value):
        record = super().__getattribute__('_record')
        if name in record:
            record[name] = value
        else:
            raise AttributeError(f'field name {name!r} does not exist')
        
    @property
    def record(self):
        return self._record
    
    @property
    def fields(self):
        return tuple(self._record.keys())

class DatabaseTable:
    def __init__(self, database, table_name):
        if not database:
            raise ConnectionError(f'database {database!r} establishing failed')
        if table_name not in database:
            raise ValueError(f'table {table_name!r} does not exist')
        self._table_name = table_name
        self._table = database[table_name]

    @property
    def table_name(self):
        return self._table_name
    
    def __call__(self, record_id): # make DatabaseTable instance a callable
        if record_id in self._table:
            return DatabaseRecord(self._table[record_id])
        raise ValueError(f'record (id={record_id}) does not exist in table {self._table_name}')

person_table = DatabaseTable(DB, 'Person')
country_table = DatabaseTable(DB, 'Country')
person = person_table(1)
print(person_table.table_name)
print(person.fields)
print(person.record)
print(person.first_name)
person.first_name = 'Jinx'
print(person.first_name)
country = country_table(person.country_id)
print(country.name)
print(country.capital)

Person
('first_name', 'last_name', 'born', 'country_id')
{'first_name': 'Isaac', 'last_name': 'Newton', 'born': 1642, 'country_id': 1}
Isaac
Jinx
United Kingdom
London
